In [ ]:
import requests
import json

# via gemini :)
def download_file(url, filename):
  """Downloads file from the given URL and saves it locally."""
  response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}) # bypass 403
  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
    print(f"Downloaded {filename} successfully.")
  else:
    print(f"Failed to download {filename}. Status code: {response.status_code}")

def load_json(filename):
  """Loads a JSON file from the given path."""
  with open(filename, 'r') as f:
    data = json.load(f)
  return data

def load_txt(filename):
  """Loads a tab-delimited text file into a dictionary."""
  data = {}
  with open(filename, 'r') as f:
    for line in f:
      ticker, cik = line.strip().split('\t')
      data[ticker] = cik
  return data

def ticker_to_cik(ticker, json_data):
  """Maps a ticker symbol to its corresponding CIK from the JSON data."""
  for key, value in json_data.items():
    if value['ticker'] == ticker:
      return value['cik_str']
  return None

# Download files
download_file("https://www.sec.gov/files/company_tickers.json", "company_tickers.json")
download_file("https://www.sec.gov/include/ticker.txt", "ticker.txt")

# Load data
json_data = load_json("company_tickers.json")
txt_data = load_txt("ticker.txt")

# Example usage
cik = ticker_to_cik("AAPL", json_data)
if cik:
  print(f"CIK for AAPL: {cik}")
else:
  print("Ticker symbol not found in JSON data.")

Downloaded company_tickers.json successfully.
Downloaded ticker.txt successfully.
CIK for AAPL: 320193


In [ ]:
import requests
import pandas as pd

# Define base URL and CIK
base_url = "https://data.sec.gov/submissions/"
cik = "CIK0000320193"

# Get main JSON data
response = requests.get(f"{base_url}{cik}.json", headers={"User-Agent": "Mozilla/5.0"})
print(response)
data = response.json()

# Extract recent filings data
recent_filings = data["filings"]["recent"]

# Convert recent filings data to pandas DataFrame
df = pd.DataFrame(recent_filings)

# Print DataFrame head
print(df.head())

print(df.describe())

# Loop through recent filings and download individual files
for filing in recent_filings:
  print(filing)
  # filing_id = filing["filing_id"]
  # filing_url = f"{base_url}{cik}-{filing_id}-submissions-001.json"

  # # Download individual filing data
  # response = requests.get(filing_url)
  # filing_data = response.json()

  # # Save individual filing data locally
  # with open(f"{filing_id}.json", "w") as f:
  #   json.dump(filing_data, f)


<Response [200]>
        accessionNumber  filingDate  reportDate        acceptanceDateTime act  \
0  0001193125-24-036431  2024-02-14              2024-02-14T16:47:05.000Z  34   
1  0001104659-24-020009  2024-02-13              2024-02-13T16:55:49.000Z  34   
2  0001086364-24-006980  2024-02-12              2024-02-12T16:17:19.000Z  34   
3  0001096906-24-000181  2024-02-09              2024-02-09T16:26:35.000Z  34   
4  0000320193-24-000022  2024-02-05  2024-02-01  2024-02-05T18:37:14.000Z       

       form fileNumber filmNumber items    size  isXBRL  isInlineXBRL  \
0  SC 13G/A  005-33632   24639680        600487       0             0   
1  SC 13G/A  005-33632   24627065         13130       0             0   
2  SC 13G/A  005-33632   24620156         13991       0             0   
3   PX14A6G  001-36743   24615426        177443       0             0   
4         4                                8450       0             0   

                      primaryDocument  \
0               

In [ ]:
df["filingDate"] = pd.to_datetime(df["filingDate"])

In [ ]:
# Define cutoff date (based on input)
cutoff_date = pd.to_datetime("2020-03-14") # March 2020

filtered_df = df[(df['form'] == "10-K") & (df['filingDate'] < cutoff_date)]

filtered_df = df[(df['filingDate'] < cutoff_date)]

grouped_data = filtered_df.groupby('form')[['filingDate', 'accessionNumber']].max()

In [ ]:
grouped_data

,filingDate,accessionNumber
form,,
10-K,2019-10-31,0001628280-16-020309
10-Q,2020-01-29,0001628280-17-004790
25,2019-03-14,0001193125-19-074874
3,2019-02-07,0001209191-15-087193
3/A,2019-04-17,0000320193-19-000058
4,2020-02-28,0001628280-17-006558
424B2,2019-11-08,0001193125-19-288412
8-A12B,2019-11-15,0001193125-19-292825
8-K,2020-02-27,0001628280-17-004663


In [ ]:
accession_number = '0001628280-16-020309'.replace('-','')
doc_url = f"https://www.sec.gov/Archives/edgar/data/{cik[3:]}/{accession_number}/Financial_Report.xlsx"

response = requests.get(doc_url, headers={"User-Agent": "Mozilla/5.0"})
print(response.content)

if response.status_code == 200:
  # Download successful, save content to file
  with open("Financial_Report.xlsx", "wb") as f:
    f.write(response.content)
  print("Downloaded Financial_Report.xlsx successfully!")
else:
  print(f"Failed to download document. Status code: {response.status_code}")

b'PK\x03\x04\x14\x00\x00\x00\x08\x00n\x85ZI\xa6\x05\x14\xd61\x02\x00\x00\xf3+\x00\x00\x13\x00\x00\x00[Content_Types].xml\xcd\xda\xcdn\xd3@\x10\x07\xf0W\x89|E\xf1f?\r\xa8\xe9\x85r\x85J\xf0\x02\x8b=\x89\xad\xd8\xde\xd5\xee6M\xdf\x9e\xb5\xd3"\x88R\xd4B#\xfd/q\x9c\xd9\x9d\x19{\xec\xdf)W\xdf\x1f<\xc5\xc5a\xe8\xc7\xb8.\xda\x94\xfcG\xc6b\xdd\xd2`c\xe9<\x8d9\xb2qa\xb0)\x9f\x86-\xf3\xb6\xde\xd9-1\xb1Z\x19V\xbb1\xd1\x98\x96i\xcaQ\\_}\xddS\x08]C\x8bO\xc7\xc0\x94{]X\xef\xfb\xae\xb6\xa9s#\xdb\x8f\xcdI\xd6\xa5\xdbl\xba\x9a\x1aW\xdf\ryK\x99riz\x97\xe3\xc5\xe2\xd6\x86\xf4\xc5\x0e9\x05;\xf4l\x0e\x1c?y9\xc5\xd9\xdb\x14\x8c>\x90mbK\x94\x86\xbe\x8c\xe9\xa1\xa7x\xae\xfe1\xf2T\xf9\x866\xf6\xaeO\xaf*\xfcx\xef\xca@\xfd\xbc&\xb6\x9d\x7f,\xf5\xf9\x90\xb3\xc4\xfc\xdb\xba\xc8\xd1\xf8\xa2\n\xa7\x1b\xff\xd6Y7LC\xf3\xe3\xf6\x8f\x1d\xd3\xf9?^\xcb\xc9M\xdc\x0f\xfdM\xb0\xf7\xddI\x81\xfd\xc5\xc64\x1d\xcb\xc1v\xe3\xb9Q\xdd\xbb\xb0\xfb\xe1\xdc\xee-\x1f\x13\x9a\xae\xaa\xa1f\xe9C^\x18Rw\xe6)\xc9\x8bos4\xb2\x9c\xfa\xbfj?=)\x

In [ ]:
import requests
from bs4 import BeautifulSoup

# Define URL and element selectors
url = "https://www.sec.gov/Archives/edgar/data/320193/000162828016020309/"
target_link_selector = "tr:nth-child(3) a"
inner_link_selector = "#formDiv:nth-child(3) tr:nth-child(2) a"
text_selector = "div:nth-child(74)"

# Send GET request and get HTML content
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.content, "html.parser")

# Find the target link and extract its href
target_link = soup.select_one(target_link_selector)
print(target_link)

inner_link_url = target_link["href"]

# Send GET request for the inner link and get HTML content
inner_response = requests.get(inner_link_url, headers={"User-Agent": "Mozilla/5.0"})
inner_soup = BeautifulSoup(inner_response.content, "html.parser")

# Extract the text from the desired element
text = inner_soup.select_one(text_selector).text

# Print the extracted text
print(text)

None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#https://www.sec.gov/Archives/edgar/data/0000320193/000162828016020309/

In [ ]:
cik

'CIK0000320193'

In [ ]:
most_recent_filings

form
10-K        320
10-Q        304
25          356
3           370
3/A         351
4           285
424B2       315
8-A12B      312
8-K         291
8-K/A       425
CERT        311
CERTNYS     529
CORRESP     774
DEF 14A     309
DEFA14A     308
DFAN14A     974
FWP         317
NO ACT      614
PRE 14A     969
PX14A6G     306
S-3ASR      389
S-8         405
S-8 POS     319
SC 13G      367
SC 13G/A    293
SD          296
UPLOAD      773
Name: filingDate, dtype: int64

In [ ]:
filtered_df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
320,0000320193-19-000119,2019-10-31,2019-09-28,2019-10-30T18:12:36.000Z,34,10-K,001-36743,191181423,,12861616,1,1,a10-k20199282019.htm,10-K
390,0000320193-18-000145,2018-11-05,2018-09-29,2018-11-05T08:01:40.000Z,34,10-K,001-36743,181158788,,12275572,1,0,a10-k20189292018.htm,10-K
478,0000320193-17-000070,2017-11-03,2017-09-30,2017-11-03T08:01:37.000Z,34,10-K,001-36743,171174673,,14071062,1,0,a10-k20179302017.htm,10-K
623,0001628280-16-020309,2016-10-26,2016-09-24,2016-10-26T16:42:16.000Z,34,10-K,001-36743,161953070,,13277662,1,0,a201610-k9242016.htm,10-K
740,0001193125-15-356351,2015-10-28,2015-09-26,2015-10-28T16:31:09.000Z,34,10-K,001-36743,151180619,,9594425,1,0,d17062d10k.htm,FORM 10-K
869,0001193125-14-383437,2014-10-27,2014-09-27,2014-10-27T17:11:55.000Z,34,10-K,000-10030,141175110,,12082626,1,0,d783162d10k.htm,10-K
978,0001193125-13-416534,2013-10-30,2013-09-28,2013-10-29T20:38:28.000Z,34,10-K,000-10030,131177575,,11043722,1,0,d590790d10k.htm,FORM 10-K


## Setup

In [ ]:
ticker_dict = {}

with open('ticker.txt', 'r') as f:
  for line in f:
      info = line.strip().split('\t')
      if len(info) == 2:
        ticker, cik = info
        ticker_dict[ticker] = cik

print(ticker_dict)

{'aapl': '320193', 'msft': '789019', 'brk-b': '1067983', 'unh': '731766', 'jnj': '200406', 'v': '1403161', 'tsm': '1046179', 'xom': '34088', 'wmt': '104169', 'spy': '884394', 'jpm': '19617', 'pg': '80424', 'nvda': '1045810', 'lvmuy': '824046', 'ma': '1141391', 'cvx': '93410', 'tsla': '1318605', 'lly': '59478', 'hd': '354950', 'bac': '70858', 'pfe': '78003', 'ko': '21344', 'abbv': '1551152', 'baba': '1577552', 'nvo': '353278', 'pep': '77476', 'cost': '909832', 'mrk': '310158', 'tmo': '97745', 'avgo': '1730168', 'dhr': '313616', 'dis': '1744489', 'tm': '1094517', 'mcd': '63908', 'asml': '937966', 'orcl': '1341439', 'abt': '1800', 'csco': '858877', 'nvs': '1114448', 'azn': '901832', 'tmus': '1283699', 'wfc': '72971', 'acn': '1467373', 'vz': '732712', 'nee': '753308', 'nke': '320187', 'ups': '1090727', 'bmy': '14272', 'ms': '895421', 'cmcsa': '1166691', 'crm': '1108524', 'cop': '1163165', 'txn': '97476', 'adbe': '796343', 'pm': '1413329', 'lin': '1707925', 'unp': '100885', 'qcom': '804328'

In [ ]:
with open('ticker.txt', 'w') as f:
  f.write(ticker_map)

In [ ]:
!pip install sec-edgar-api

In [ ]:
from sec_edgar_api import EdgarClient

edgar = EdgarClient(user_agent="ENGS15.18 engs2524w@gmail.com")

In [ ]:
edgar

In [ ]:
edgar.get_submissions(cik="0000320193")

TypeError: unhashable type: 'slice'

In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup

cik="0000320193"

url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type=10-K&dateb=&owner=exclude&count=40'
# add this
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
response = requests.get(url, headers=headers)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

<Response [200]>
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ENG">
<head>
<title>EDGAR Search Results</title>
<link href="/include/interactive.css" rel="stylesheet" type="text/css"/>
<link href="/edgar/search/css/smartSearchCompanyLanding.css" rel="stylesheet" type="text/css"/>
<script language="JavaScript" src="/edgar/search/js/smartSearchCompanyLanding.js" type="text/javascript"></script>
<link href="/cgi-bin/browse-edgar?action=getcompany&amp;CIK=0000320193&amp;type=10-K%25&amp;dateb=&amp;owner=exclude&amp;count=40&amp;output=atom" rel="alternate" title="ATOM" type="application/atom+xml"/>
</head>
<body style="margin: 0">
<!-- SEC Web Analytics - For information please visit: https://www.sec.gov/privacy.htm#collectedinfo -->
<noscript><iframe height="0" src="//www.googletagmanager.com/ns.html?id=GTM-TD3BKV" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<script>(function(w,d,s,l,i){w[l]=w[

### JSON


## Tickers

In [ ]:
ticker_map = """
aapl	320193
msft	789019
brk-b	1067983
unh	731766
jnj	200406
v	1403161
tsm	1046179
xom	34088
wmt	104169
spy	884394
jpm	19617
pg	80424
nvda	1045810
lvmuy	824046
ma	1141391
cvx	93410
tsla	1318605
lly	59478
hd	354950
bac	70858
pfe	78003
ko	21344
abbv	1551152
baba	1577552
nvo	353278
pep	77476
cost	909832
mrk	310158
tmo	97745
avgo	1730168
dhr	313616
dis	1744489
tm	1094517
mcd	63908
asml	937966
orcl	1341439
abt	1800
csco	858877
nvs	1114448
azn	901832
tmus	1283699
wfc	72971
acn	1467373
vz	732712
nee	753308
nke	320187
ups	1090727
bmy	14272
ms	895421
cmcsa	1166691
crm	1108524
cop	1163165
txn	97476
adbe	796343
pm	1413329
lin	1707925
unp	100885
qcom	804328
ry	1000275
schw	316709
cvs	64803
bhp	811809
tte	879764
low	60667
amgn	318154
rtx	101829
asti	1350102
hsbc	1089113
hdb	1144967
mdt	1613103
hon	773840
td	947263
axp	4962
t	732717
intu	896878
intc	50863
eqnr	1140625
ul	217410
qqq	1067839
lmt	936468
pypl	1633917
ibm	51143
amt	1053507
gs	886982
de	315189
sbux	829224
nflx	1065280
bp	313807
sny	1121404
deo	835403
sap	1000184
cat	18230
adp	8670
rtntf	887028
c	831001
ci	1739940
blk	1364742
jd	1549802
amd	2488
cslly	1274152
bti	1303523
sony	313838
rio	863064
el	1001250
ba	12927
now	1373715
pbr	1119639
pld	1045609
bud	1668717
pdd	1737806
syk	310764
enb	895728
cni	16868
spgi	64040
mdlz	1103982
cb	896159
so	92122
duk	1326160
gild	882095
ibn	1103838
bkng	1075531
abnb	1559720
mmc	62709
tgt	27419
amat	6951
gsk	1131399
adi	6281
tjx	109198
mo	764180
bam	1001085
noc	1133421
regn	872589
infy	1067491
ge	40545
glncy	1524684
zts	1555280
isrg	1035267
eadsy	1378580
pgr	80661
bdx	10795
vrtx	875320
eog	821189
wm	823768
pbcry	1463157
cp	16875
idexy	1438656
cci	1051470
pnc	713676
fisv	798354
cme	1156375
googl	1652044
usb	36104
chtr	1091667
bx	1393818
mmm	66740
csx	277948
mufg	67088
bns	9631
d	715957
hca	860730
sftby	1065521
uber	1543151
tfc	92230
amzn	1018724
team	1650372
hum	49071
gd	40533
cnq	1017413
cl	21665
vale	917851
bmo	927971
aon	315293
dmlry	1067318
itw	49826
oxy	797468
snow	1640147
bsx	885725
atvi	718877
cta-pa	30554
pxd	1038357
f	37996
ew	1099800
mpc	1510295
gm	1467858
mu	723125
shw	89800
lrcx	707549
nsc	702165
ubs	1610520
ntes	1110646
epd	1061219
etn	1551182
dg	29534
slb	87347
abb	1091587
eqix	1101239
ice	1571949
met	1099219
mrna	1682852
apd	2969
psa	1393311
sre	1032208
tri	1075124
fdx	1048911
amov	1129137
mar	1048286
kdp	1418135
rbgly	1420798
mco	1059556
fis	1136893
mck	927653
klac	319201
ntr	1725964
cnc	1071739
aep	4904
itub	1132597
enlay	1096200
race	1648416
relx	929869
meli	1099590
prndy	899108
snps	883241
cmg	1058090
emr	32604
vmw	1124610
adm	7084
dvn	1090012
trp	1232384
orly	898173
ecl	31462
mnst	865752
rsg	1060391
su	311337
gis	40704
ctva	1755672
hsy	47111
cdns	813672
abev	1565025
stz	16918
aig	5272
bidu	1329099
san	891478
lhx	202058
edu	1372920
adsk	769397
azo	866787
lulu	1397187
cof	927628
fcx	831259
lng	3570
aph	820313
ngg	1004315
vlo	1035002
payx	723531
bsbr	1471055
hmc	715153
nxpi	1413447
stla	1605484
khc	1637459
bce	718940
exc	1109357
enph	1463101
tak	1395064
kmi	1506307
ctas	723254
smfg	1022837
crwd	1535527
e	1002242
kmb	55785
syy	96021
tel	1385157
rop	882835
trv	86312
gld	1222333
msi	68505
a	1090872
wmb	107263
iqv	1478242
xel	72903
afl	4977
mrvl	1835632
jci	833444
lyg	1160106
hes	4447
wday	1327811
wcn	1318220
et	1276187
ajg	354190
gpn	1123360
cve	1475260
tt	1466258
hlt	1585689
msci	1408198
ing	1039765
all	899051
bk	1390777
bf-a	14693
scco	1001838
nwg	844150
anet	1596532
o	726728
psx	1534701
mchp	827054
bbd	1160330
kr	56873
ea	712515
rivn	1874178
carr	1783180
pru	1137774
bntx	1776985
dow	1751788
ed	1047862
ph	76334
apo	1858681
nio	1736541
vod	839923
se	1703399
nem	1164727
yum	1041061
twtr	1418091
peg	788784
mplx	1552000
bcs	312069
mfc	1086888
alb	915913
nue	73309
sbac	1034054
alc	1167379
ctsh	1058290
rmd	943819
spg	1063761
tdg	1260221
stm	932787
bbva	842180
ttd	1671933
imo	49938
ifnny	1107457
cpng	1834584
sq	1512673
wec	783325
dlr	1297996
well	766704
danoy	1048515
rost	745732
ilmn	1110803
gfs	1709048
amp	820027
dltr	935703
iau	1278680
biib	875045
puk	1116578
lvs	1300514
keys	1601046
es	72741
cmi	26172
sqm	909037
ddog	1561550
pcar	75362
idxx	874716
vrsk	1442145
dfs	1393612
wba	1618921
bax	10456
tlk	1001807
pcg	1004980
odfl	878927
tu	868675
on	1097864
otis	1781335
cht	1132924
hpq	47217
abc	1140859
ibkr	1381197
kkr	1404912
mfg	1335730
ppery	1461748
hcmly	948696
dell	1571996
csgp	1057352
ppg	79879
mtd	1037646
dd	1666700
rok	1024478
expgy	1376891
avb	915912
caixy	1535628
ame	1037868
baesy	895564
ceg	1868275
dnbby	1498632
oke	1039684
glw	24741
wit	1123799
nok	924613
gww	277135
eqr	906107
rweoy	939380
crh	849395
fast	815556
lcid	1811210
gold	756894
veev	1393052
awk	1410636
iff	51253
oran	1038143
cqp	1383650
ebay	1065088
li	1791706
hal	45012
frc	1132979
cprt	900075
lyb	1489393
cbre	1138118
tsn	100493
aptv	1521332
stt	93751
eix	827052
clr	732834
sgen	1060736
slf	1097362
trow	1113169
nu	1691493
dte	936340
dia	1041130
rblx	1315098
ferg	1832433
ctra	858470
exr	1289490
alny	1178670
hrl	48465
it	749251
dhi	882184
zs	1713683
mdy	936958
fitb	35527
fang	1539838
zbh	1136869
mtb	36270
k	55067
wegzy	1445166
gmab	1434265
arzgy	1379235
zm	1585521
caj	16988
siri	908937
fmx	1061736
aee	1002910
wy	106535
epam	1352010
are	1035443
etr	65984
ckhuy	1637120
ftv	1659166
fe	1031296
cdw	1402057
ulta	1403568
zto	1677250
vwdry	1330306
efx	33185
eric	717826
wtw	1140536
fnv	1456346
len	920760
hig	874766
rjf	720005
sivb	719739
lgfry	1554394
gwlif	850918
invh	1687229
dash	1792789
tef	814052
gpc	40987
lnngy	1447629
cm	1045520
luv	92380
ccep	1650107
cf	1324404
ppl	922224
yumc	1673358
vmc	1396009
net	1477333
lh	920148
ec	1444406
tsco	916365
rf	1281761
dal	27904
rci	733099
bkr	1701605
ttm	926042
payc	1590955
anss	1013462
mlm	916076
codyy	1012037
hban	49196
mkc	63754
dre	783280
uri	1067701
mro	101778
vici	1705696
lyv	1335258
pfg	1126328
poahy	1450346
fts	1666175
wst	105770
cnp	1130310
coin	1679788
tscdy	885834
ir	1699150
moh	1179929
ste	1757898
mt	1243429
spot	1639920
algn	1097149
mos	1285785
ntrs	73124
argx	1697862
pwr	1050915
pba	1546066
vrsn	1014473
mpwr	1280452
cms	811156
hei	46619
chd	313927
db	1159508
ptaiy	1455967
snap	1564408
maa	912595
br	1383312
agr	1634997
atasy	1437775
foxa	1754301
vtr	740260
gib	1061574
ix	1070304
ui	1511737
cah	721371
lpla	1397911
qsr	1618756
pcrfy	63271
jbht	728535
acgl	947484
eqt	33213
rprx	1802768
amcr	1748790
wat	1000697
podd	1145197
syf	1601712
aes	874761
panw	1327567
zi	1794515
anyyy	1700171
sui	912593
dov	29905
rol	84839
bby	764478
tdy	1094285
bro	79282
clx	21076
teck	886986
bgne	1651308
holx	859737
xyl	1524472
iclr	1060955
ess	920522
dri	940944
hpe	1645590
plug	1093691
elp	1041792
fds	1013237
tcom	1269238
key	91576
tsgty	939383
grmn	1121788
pins	1506293
mga	749098
pki	31791
flt	1175454
swks	4127
trgp	1389170
cnhi	1567094
sedg	1419612
expd	746515
wab	943452
mtch	891103
expe	1324424
bill	1786352
mkl	1096343
ts	1190723
lbrda	1611983
bmrn	1048477
mgm	789570
cag	23217
cinf	20286
cfg	759944
pltr	1321655
swmay	1011218
vfc	103379
chkp	1015922
zbra	877212
ryaay	1038683
csl	790051
umc	1033767
iex	832101
snmry	1437771
evrg	1711269
mdb	1441816
phg	313216
wpc	1025378
hldcy	811250
irm	1020569
j	52988
ip	51434
ntap	1002047
ctlt	1596783
lsxma	1560385
dgx	1022079
incy	879169
chwy	1766502
ato	731802
stld	1022671
lnt	352541
sjm	91419
azpn	1897982
rkt	1805284
trmb	864749
tw	1758730
tyl	860731
avy	8818
splk	1353283
coo	711404
ttwo	946581
omc	29989
hwm	4281
smpny	1669414
sgioy	1281721
hznp	1492426
akam	1086222
hubs	1404655
jkhy	779152
wpm	1323404
iep	813762
lkq	1065696
etsy	1370637
jrony	1438077
cpb	16732
bio	12208
avtr	1722482
txt	217346
xpev	1810997
znh	1041668
l	60086
fslr	1274494
wmg	1319161
chrw	1043277
swk	93556
nvr	906163
pcty	1591698
stx	1137789
udr	74208
twlo	1447669
peak	765880
kb	1445930
kmx	1170010
fmc	37785
tpl	1811074
ssnc	1402436
wolf	895419
prdsy	1527249
ptc	857005
ter	97210
ebr	1439124
ovv	1792580
viv	1066119
cpt	906345
pool	945841
gnrc	1474735
shg	1263043
tru	1552033
apa	1841666
ares	1176948
kof	910631
els	895417
ndsn	72331
ldos	1336920
rcl	884887
hst	1070750
bxp	1037540
ctxs	877890
aci	1646972
rgen	730272
bg	1144519
frfhf	915191
nice	1003935
cs	1159510
kim	879101
ccl	815097
dpz	1286681
cnttq	1652499
cboe	1374310
fhn	36966
sjr	932872
gddy	1609711
entg	1101302
bip	1406234
pkg	75677
ual	100517
nlok	849399
pkx	889132
fico	814547
ar	1433270
cck	1219601
tech	842023
bah	1443646
ben	38777
atlky	748954
vst	1692819
chk	895126
dar	916540
wlk	1262823
abmd	815094
hubb	48898
wrb	11544
mas	62996
eqh	1333986
docu	1261333
amh	1562401
htht	1483994
rpm	110621
glpi	1575965
wdc	106040
emraf	1127248
ce	1306830
ssl	314590
erie	922621
ptbry	1504122
vtrs	1792044
wtrg	78128
y	775368
bouyy	1445475
ccj	1009001
sna	91440
asx	1122411
wes	1423902
jxhly	1487533
cea	1030475
ni	1111711
aer	1378789
lw	1679273
fnf	1331875
ally	40729
suz	909327
cuk	1125259
re	1095073
cg	1527166
rs	861884
nfe	1749723
u	1810806
afg	1042046
dqjcy	1446597
tap	24545
grab	1855612
dt	1773383
mmp	1126975
nrg	1013871
ipg	51644
xp	1787425
emn	915389
tfx	96943
ggg	42888
icl	941221
sbny	1288784
snn	845982
has	46080
mgddy	896076
nmr	1163653
bldr	1316835
nly	1043219
bep	1533232
wso	105016
ewbc	1069157
cma	28412
bki	1627014
ihg	858446
aap	1158449
zshgy	1495686
bj	1531152
cna	21175
bntgy	1533945
gl	320335
crl	1100682
h	1468174
tost	1650164
uhal	4457
ateyy	1158838
qgen	1015820
ndaq	1120193
nwsa	1564708
acm	868857
wrk	1732845
bap	1001290
nbix	914475
wpp	806968
lbtya	1570585
wsm	719955
dox	1062579
aem	2809
sci	89089
swav	1642545
reg	910606
nlsn	1492633
hsic	1000228
czr	1590895
uthr	1082554
roku	1428439
dks	1089063
bhvn	1689813
jhx	1159152
rrx	82811
phm	822416
fms	1333141
agl	1831097
morn	1289419
teva	818686
bsy	1031308
gfl	1780232
mktx	1278021
okta	1660134
armk	1584509
gshn	1639327
lsi	944314
srpt	873303
bbwi	701985
dbrg	1679688
wms	1604028
lamr	1090425
cube	1298675
nvcr	1645113
asxfy	1449003
dva	927066
wsc	1647088
ttc	737758
qrvo	1604778
hood	1783879
aeg	769218
lnvgy	932477
zg	1617640
bch	1161125
jazz	1232524
deck	910521
mtn	812011
hii	1501585
tfii	1588823
zen	1463172
burl	1579298
aal	6201
zion	109380
ffiv	1048695
lumn	18926
lii	1069202
dish	1001082
tpr	1116132
car	723612
app	1751008
cday	1725057
jnpr	1043604
bdnny	1446457
glob	1557860
bz	1842827
rexr	1571283
cfr	39263
aqn	1174169
htz	1657853
vivhy	1127055
lnc	59558
bwa	908255
aiz	1267238
manh	1056696
whr	106640
lu	1816007
kep	887225
rga	898174
dxcm	1093557
arcc	1287750
dscsy	1671750
snx	1177394
stor	1538990
paa	1070423
eslt	1027664
gtlb	1653482
rhi	315213
wfg	1402388
glpey	1385535
rdy	1135951
masi	937556
cbsh	22356
remyy	1552653
jbl	898293
flex	866374
g	1398659
fnd	1507079
aos	91142
prymy	1437672
oc	1370946
slvyy	930826
alle	1579241
jll	1037976
tme	1744676
dbx	1467623
pnw	764622
pstg	1474432
unm	5513
ftnt	1262039
uso	1327068
adooy	1489079
brkr	1109354
pag	1019849
kkoyy	1436794
fcnca	798941
axon	1069183
lea	842162
swch	1710583
jef	96223
oge	1021635
logi	1032975
mpw	1287865
fbhs	1519751
rrc	315852
bsmx	1698287
leco	59527
aa	1675149
knx	1492691
grfs	1438569
avlr	1348036
agco	880266
path	1734722
casy	726958
uelmo	100826
chng	1756497
uhs	352915
dcp	1338065
otex	1002638
midd	769520
hrb	12659
dlo	1846832
nxst	1142417
ugi	884614
aseky	1450206
cms-pb	201533
pso	938323
dkng	1883685
tixt	1825155
estc	1707753
frt	34903
oln	74303
swn	7332
skm	1015650
mat	63276
run	1469367
nnn	751364
chdn	20212
seic	350894
wal	1212545
five	1177609
clf	764065
ymm	1838413
olpx	1868726
celh	1341766
gtls	892553
pfgc	1618673
miufy	1447113
pen	1321732
ivz	914208
cold	1455863
wynn	1174922
achc	1520697
see	1012100
pcor	1611052
clvt	1764046
alsmy	1062066
ggb	1073404
pac	1347557
s	1583708
cgnx	851205
che	19584
dlb	1308547
lad	1023128
bsac	1027552
jpxgy	1600520
mdiby	1379481
ogn	1821825
bery	1378992
ohi	888491
an	350698
mtlhy	1446418
rba	1046102
krtx	1771917
musa	1573516
pnr	77360
xray	818479
cacc	885550
ttek	831641
vips	1529192
elan	1739104
nwl	814453
stwd	1465128
voya	1535929
exas	1124140
cflt	1699838
levi	94845
rnr	913144
gpk	1408075
bepc	1791863
bili	1723690
kbr	1357615
lscc	855658
osh	1564406
arw	7536
cpri	1530721
rkuny	1294591
dxc	1688568
atr	896622
isday	1224306
nep	1603145
roll	1324948
mtdr	1520006
nov	1021860
rl	1037038
mhk	851968
adt	1703056
gfi	1172724
yndx	1513845
ori	74260
afrm	1820953
cib	1071371
pii	931015
ions	874015
wcc	929008
usfd	1665918
legn	1801198
fr	921825
crcby	1553915
wex	1309108
pb	1068851
mssmy	1710864
xm	1747748
cvna	1690820
oukpy	1436786
clh	822818
edr	1766363
accyy	1049183
chh	1046311
cien	936395
asr	1123452
dseey	1481045
futu	1754581
egp	49600
alv	1034670
nfg	70145
nclh	1513761
caci	16058
cabo	1632127
airc	1820877
st	1477294
iot	1642896
thc	70318
pnfp	1115055
dci	29644
chpt	1777393
bpop	763901
sbs	1170858
csan	1430162
eme	105634
hog	793952
coty	1024305
sbsw	1786909
bokf	875357
dnb	1799208
cnxc	1803599
apls	1492422
wh	1722684
txrh	1289460
brx	1581068
pdce	77877
sf	720672
mur	717423
ppc	802481
gmed	1237831
trq	1158041
shlx	1610466
asnd	1612042
gntx	355811
itt	216228
vac	1524358
gil	1061894
son	91767
uwmc	1783398
mndy	1845338
docs	1516513
mtz	15615
ffin	36029
cybr	1598110
hqy	1428336
fybr	20520
phi	78150
mdu	67716
byd	906553
rcm	1910851
wf	1264136
qlys	1107843
slv	1330568
rgld	85535
kt	892450
gh	1576280
wwd	108312
halo	1159036
irdm	1418819
infa	1868778
agnc	1423689
snv	18349
fsk	1422183
vmi	102729
dsgx	1050140
fivn	1288847
reyn	1786431
alk	766421
krc	1025996
lyft	1759509
dooo	1748797
opch	1014739
hun	1307954
cw	26324
nvst	1757073
onon	1858985
crsp	1674416
sndl	1766600
ssb	764038
mp	1801368
exls	1297989
axta	1616862
nvt	1720635
xpo	1166003
plnt	1637207
gbci	868671
timb	1826168
ash	1674862
skx	1065837
hcp	1720671
lnth	1521036
bc	14930
bld	1633931
tx	1342874
wu	1365135
iac	1800227
cccs	1818201
dds	28917
ayi	1144215
faf	1472787
dtm	1842022
cae	1173382
rh	1528849
knsl	1669162
lstr	853816
pycr	1839439
drvn	1804745
ingr	1046257
cc	1627223
tmbby	1531740
au	1067428
ccoey	1551467
flo	1128928
gwre	1528396
lite	1633978
x	1163302
hli	1302215
post	1530950
cr	25445
synh	1610950
medp	1668397
fsv	1637810
stag	1479094
insp	1609550
sm	893538
exel	939767
ida	1057877
ozk	1569650
tpx	1206264
lthm	1742924
stn	1131383
osk	775158
cx	1076378
civi	1509589
lfus	889331
ora	1296445
prgo	1585364
saic	1571123
trex	1069878
coup	1385867
fcn	887936
slab	1038074
penn	921738
tmx	1428875
slgn	849869
dava	1656081
src	1308606
wwe	1091907
nyt	71691
tol	794170
pags	1712807
ntnx	1618732
adc	917251
lhcg	1303313
sofi	1818874
cade	1299939
nati	935494
tdoc	1477449
ntla	1652130
pri	1475922
amg	1004434
amc	1411579
site	1650729
glbe	1835963
w	1616707
mksi	1049502
ehc	785161
vno	899689
bncdy	1667075
qs	1811414
orcc	1655888
amn	1142750
tnet	937098
ubsi	729986
gxo	1852244
zlab	1704292
saia	1177702
m	794367
vvv	1674910
sgfy	1828182
bwxt	1486957
enlc	1592000
wtfc	1015328
cigi	913353
lysdy	1444327
am	1623925
rli	84246
hxl	717605
sigi	230557
swx	1692115
leg	58492
vrt	1674101
tsem	928876
tkr	98362
bkh	1130464
thg	944695
vly	714310
asgn	890564
hgv	1674168
ntra	1604821
bxmt	1061630
smar	1366561
oled	1005284
hp	46765
tgna	39899
asan	1477720
sim	887153
ryn	52827
stvn	1849853
expo	851520
colm	1050797
ensg	1125376
docn	1582961
bkgfy	1447137
msa	66570
mfa	1055160
asai	1834048
mtg	876437
rog	84748
rhp	1040829
lpx	60519
jsgry	1442656
bmbl	1830043
dna	1830214
ipgp	1111928
crk	23194
gds	1526125
iaa	1745041
srcl	861878
exp	918646
jscpy	1449005
lanc	57515
omf	1584207
ivr	1437071
embk	1827980
cytk	1061983
ryan	1849253
cmc	22444
spwr	867773
esi	1590714
axs	1214816
nvei	1765159
mrtx	1576263
crox	1334036
irtc	1388658
ufpi	912767
eeft	1029199
slm	1032033
be	1664703
dv	1819928
ayx	1689923
novt	1076930
sfbs	1430723
prva	1759655
por	784977
nycb	910073
adrzy	1436467
amkr	1047127
chx	1723089
spsc	1092699
trno	1476150
phys	1477049
aqua	1604643
cere	1805387
mituy	1446420
wts	795403
lkncy	1767582
hwc	750577
val	314808
wen	30697
gbt	1629137
bak	1071438
ntco	1776967
wix	1576789
staa	718937
pltk	1828016
umbf	101382
wbs	801337
esnt	1448893
cwen	1567683
eva	1592057
crus	772406
bpmc	1597264
seas	1564902
shc	1822479
msm	1003078
algm	866291
azihy	1533475
psn	275880
den	945764
aso	1817358
njr	356309
dq	1477641
wb	1595761
syna	817720
athm	1527636
mq	1522540
cwst	911177
shop	1594805
ogs	1587732
unvr	1494319
ubsfy	1446702
tho	730263
bxsl	1736035
seb	88121
pinc	1577916
tenb	1660280
ab	825313
ftch	1740915
curlf	1756770
fti	1681459
rng	1384905
cig	1157557
mgy	1698990
nsa	1618563
he	354707
auy	1264089
wtm	776867
cbt	16040
is	1837430
kgc	701818
atus	1702780
irt	1466085
wing	1636222
beam	1745999
etrn	1747009
itci	1567514
al	1487712
frsh	1544522
krg	1286043
bl	1666134
powi	833640
bcpc	9326
wns	1356570
pnm	1108426
jhg	1274173
cpg	1545851
mxchy	1435969
sam	949870
twks	1866550
ait	109563
vet	1293135
coke	317540
kex	56047
iart	917520
nsp	1000753
four	1794669
owl	1823945
trtn	1660734
cnm	1856525
erf	1126874
r	85961
kss	885639
cuz	25232
avt	8858
box	1372612
awi	7431
hae	313143
ptcay	1547873
bhf	1685040
man	871763
newr	1448056
bfam	1437578
ihcpf	1508633
fele	38725
fl	850209
umpq	1077771
amed	896262
fizz	69891
woof	1826470
sndr	1692063
ati	1018963
homb	1331520
fnb	37808
mtsi	1493594
tfsl	1381668
dnp	806628
act	1823529
alks	1520262
ncr	70866
azta	933974
laz	1311370
pgny	1551306
abcm	1492074
pvh	78239
duol	1562088
sji	91928
becn	1124941
fcfs	840489
lpl	1290109
amrc	1488139
dnli	1714899
ebc	1810546
ptct	1070081
peco	1476204
ambp	1845097
altr	1701732
msgs	1636519
ago	1273813
txg	1770787
rdn	890926
tndm	1438133
mms	1032220
esgr	1363829
flr	1124198
fn	1408710
atkr	1666138
nari	1531048
oaofy	1058255
uaa	1336917
abg	1144980
alsn	1411207
gt	42582
indb	776901
dei	1364250
arwr	879407
igt	1619762
grp-un	1564538
cnx	1070412
zim	1654126
lac	1440972
evo	1412558
ay	1601072
pwsc	1835681
slfpy	1370418
tnl	1361658
hhc	1498828
golf	1672013
ccxi	1340652
atco	1794846
axnx	1603756
olli	1639300
foxf	1424929
eurn	1604481
evr	1360901
alit	1809104
calx	1406666
aple	1418121
ssd	920371
spt	1517375
fls	30625
trip	1526520
ncno	1902733
fix	1035983
bcucy	1581285
icui	883984
tal	1499620
bipc	1788348
sr	1126956
ajrd	40888
envx	1828318
vc	1111335
zd	1084048
crc	1609253
ucbi	857855
pton	1639825
appf	1433195
wk	1445305
apg	1796209
cvbf	354647
doc	1574540
chffy	1601994
hgty	1840776
mli	89439
dfihy	871459
mmyt	1495153
avnt	1122976
evh	1628908
joby	1819848
irdey	1530256
btg	1429937
qdel	1906324
bb	1070235
tv	912892
pbf	1534504
tdc	816761
ael	1039828
sem	1320414
cns	1284812
cef	1726122
brp	1781755
sid	1049659
gatx	40211
go	1771515
cbu	723188
alrm	1459200
nea	1195737
vnt	1786842
hr	1360604
lope	1434588
vlpny	1378984
viav	912093
appn	1441683
bsm	1621434
gps	39911
lfst	1845257
fhb	36377
mmsi	856982
pk	1617406
onto	704532
bku	1504008
sun	1552275
hri	1364479
smci	1375365
coop	933136
lazr	1758057
chgg	1364954
sum	1621563
abcb	351569
txmd	25743
esmt	1863105
twnk	1644406
gsat	1366868
paas	771992
pega	1013857
inst	1841804
gtes	1718512
asmvy	1447945
dy	67215
onem	1404123
wsfs	828944
brze	1676238
frhc	924805
caty	861842
fhi	1056288
aur	1828108
hasi	1561894
zws	1439288
ful	39368
banf	760498
unf	717954
pacw	1102112
rare	1515673
cd	1807192
cpa	1345105
awr	1056903
cwk	1628369
glpg	1421876
boh	46195
pfsi	1745916
dufry	1546354
egfey	1437178
gef	43920
sqsp	1496963
arlp	1086600
eqc	803649
aeis	927003
srad	1836470
asb	7789
mnmd	1813814
sauhy	1451279
knbe	1664998
pten	889900
wd	1497770
epr	1045450
tcbi	1077428
mrvi	1823239
aval	1504764
cwt	1035201
sxt	310142
vsbc	1697884
yeti	1670592
neu	1282637
sbra	1492298
nova	1772695
sfm	1575515
adnt	1670541
hiw	921082
vrns	1361113
ale	66756
nomd	1651717
ppbi	1028918
abcl	1703057
nni	1258602
jxn	1822993
pch	1338749
sig	832988
slg	1040971
kos	1509991
cri	1060822
fold	1178879
bnl	1424182
rpd	1560327
aaon	824142
sanm	897723
livn	1639691
hbi	1359841
iijiy	1090633
bzzuy	1437774
we	1813756
alhc	1832466
kmpr	860748
glng	1207179
delky	1361371
cvi	1376139
kwr	81362
fbp	1057706
diod	29002
jbt	1433660
sky	90896
vsco	1856437
nwe	73088
smg	825542
acad	1070494
matx	3453
stne	1745431
jwn	72333
cort	1088856
bowl	1840572
ottr	1466593
pzza	901491
rezi	1740332
aaww	1135185
bdc	913142
ibtx	1564618
gpi	1031203
psec	1287032
eye	1710155
mcw	1853513
hi	1417398
ssrm	921638
cwan	1866368
cvet	1752836
nsit	932696
taipy	1716957
insm	1104506
tmhc	1562476
ulcc	1670076
pegry	1455633
smpl	1702744
nhi	877860
ava	104918
cvt	1827075
lspd	1823306
xrx	1770450
pslv	1494728
lcii	763744
vicr	751978
tcn	1584425
nvg	1090116
impjy	1437538
azek	1782754
ofc	860546
jbgs	1689796
btegf	1279495
calm	16160
onb	707179
omab	1378239
spr	1364885
tkc	1071321
rnw	1848763
agi	1178819
mlco	1381640
dbc	1328237
vsat	797721
xmtr	1657573
abm	771497
jks	1481513
main	1396440
jamf	1721947
ghc	104889
nad	1083839
kfy	56679
lxp	910108
aca	1739445
btu	1064728
rmbs	917273
bhc	885590
env	1337619
cxm	1569345
ain	819793
ugp	1094972
fsr	1720990
vsh	103730
rlay	1812364
mgee	1161728
cnmd	816956
sgml	1848309
mc	1596967
iboc	315709
nvmi	1109345
mth	833079
bmi	9092
crvl	874866
sfnc	90498
dorm	868780
lbrt	1694028
hele	916789
trup	1371285
mrcy	1049521
mntk	1826600
gkos	1192448
flyw	1580560
out	1579877
sitc	894315
jjsf	785956
vir	1706431
ngvt	1653477
mxl	1288469
wern	793074
iq	1722608
aciw	935036
hlf	1180262
fult	700564
eprt	1728951
ens	1289308
hrmy	1802665
fro	913290
bco	78890
ypf	904851
hpk	1792849
sgry	1638833
root	1788882
stng	1483934
ggal	1114700
uilcy	1449580
hubg	940942
meoh	886977
gdrx	1809519
fibk	860413
fwrd	912728
spxc	88205
hlne	1433642
arvn	1655759
fate	1434316
shls	1831651
wdfc	105132
pdco	891024
vrrm	1682745
cbrl	1067294
jblu	1158463
yelp	1345016
wor	108516
open	1801169
pbh	1295947
klic	56978
ccoi	1158324
ibrx	1326110
dkl	1552797
aub	883948
si	1312109
plxs	785786
whd	1699136
lesl	1821806
ahco	1725255
virt	1592386
vnom	1602065
rusha	1012019
save	1498710
cpe	928022
mog-a	67887
fun	811532
nog	1104485
ozon	1822829
inmd	1742692
pagp	1581990
gtbif	1795139
ifs	1615903
nuva	1142596
exg	1379438
ibp	1580905
sage	1597553
mgpi	835011
amr	1704715
sfrgy	1571549
kw	1408100
vrnt	1166388
tac	1144800
simo	1329394
nvax	1000694
stem	1758766
unfi	1020859
utf	1275617
focs	1651052
ths	1320695
blkb	1280058
albky	902102
sho	1295810
ipar	822663
mstr	1050446
arry	1820721
ax	1299709
evtc	1559865
ntct	1078075
kbh	795266
csiq	1375877
eqrx	1843762
wire	850460
pcrx	1396814
comm	1517228
amba	1280263
colb	887343
enic	1659939
roiv	1635088
cert	1827090
rrr	1653653
kd	1867072
hayw	1834622
iba	1044896
fss	277509
wfrd	1603923
lth	1869198
shoo	913241
aimc	1374535
manu	1549107
ping	1679826
cno	1224608
clbk	1723596
avav	1368622
tex	97216
thnpy	1792045
usm	821130
pi	1114995
axsm	1579428
cvlt	1169561
bpxxy	1567024
bcc	1328581
lfg	1823766
gcp	1644440
peb	1474098
gbdc	1476765
bcrx	882796
tr	98677
navi	1593538
ttec	1013880
agti	1749704
cixx	1829948
rklb	1819994
evcm	1853145
ameh	1083446
trn	99780
brfs	1122491
cbz	944148
arnc	1790982
lgf-a	929351
wsbc	203596
ceix	1710366
yumm	1073748
zgn	1877787
brc	746598
qtwo	1410384
cpk	19745
csq	1275214
sabr	1597033
abr	1253986
plmr	1761312
itri	780571
rxrx	1601830
rxdx	1718852
bldp	1453015
prmw	884713
frme	712534
qfin	1741530
ihs	1876183
nmrk	1690680
zip	1617553
prft	1085869
fa	1210677
tcnnf	1754195
gme	1326380
utz	1739566
oi	812074
scl	94049
nex	1688476
verx	1806837
hcm	1648257
iipr	1677576
orc	1518621
vzio	1835591
yy	1530238
iosp	1054905
ken	1611005
upst	1647639
etwo	1800347
nkla	1731289
rig	1451505
trox	1530804
pd	1568100
upwk	1627475
atrc	1323885
joe	745308
elf	1600033
sblk	1386716
laur	912766
dan	26780
prk	805676
payo	1845815
hth	1265131
itcb	1276671
ne	1458891
apam	1517302
form	1039399
ailih	18654
frog	1800667
cmtg	1666291
ktb	1760965
banr	946673
udmy	1607939
isee	1410939
urbn	912615
hl	719413
mgrc	752714
smtc	88941
sstk	1549346
itgr	1114483
acls	1113232
dnut	1857154
sdgr	1490978
utg	1263994
safe	1688852
nvee	1532961
lgih	1580670
ffbc	708955
mdrx	1124804
unit	1620280
wafd	936528
esba	1553079
epc	1096752
fcpt	1650132
cnne	1704720
nxe	1698535
spb	109177
ese	866706
xene	1582313
rlj	1511337
gpor	874499
lz	1286139
vtscy	1880866
ssu	1869858
mdc	773141
twst	1581280
ster	1645070
mfgp	1359711
hpp	1482512
verv	1840574
allo	1737287
kai	886346
trmk	36146
sg	1477815
six	701374
dk	1694426
ag	1308648
lkfn	721994
sono	1314727
icfi	1362004
fbc	1033012
imcr	1671927
wmk	105418
gnw	1276520
cswi	1624794
shak	1620533
atsg	894081
thrm	903129
sjw	766829
arch	1037676
rely	1782170
carg	1494259
teo	932470
ctre	1590717
ccu	888746
prct	1588978
pipr	1230245
wgo	107687
hep	1283140
mtx	891014
asz	1839191
boot	1610250
msge	1795250
dh	1861795
amps	1828723
mac	912242
sah	1043509
rlx	1828365
ety	1340736
atgsy	1437541
lyel	1806952
erj	1355444
fbk	1649749
atge	730464
goos	1690511
roic	1407623
prgs	876167
mpln	1793229
arcb	894405
drh	1298946
cent	887733
arhs	1875444
rvnc	1479290
pl	1836833
psmt	1041803
mcy	64996
ceqp	1136352
vtyx	1851194
pax	1825570
rcus	1724521
ftdr	1727263
cvco	278166
crct	1828962
webr	1857951
skin	1818093
kmt	55242
furcf	1559444
omi	75252
htlf	920112
you	1856314
gdv	1260729
vctr	1570827
enr	1632790
npo	1164863
iova	1425205
fg	1934850
odp	800240
sbcf	730708
ctkb	1831915
gms	1600438
dsey	1831617
slcjy	1523541
ptve	1527508
rnst	715072
xhr	1616000
supn	1356576
agio	1439222
tell	61398
blmn	1546417
gff	50725
nus	1021561
expi	1495932
cim	1409493
gogl	1029145
rqi	1157842
mlkn	66382
sitm	1451809
flng	1772253
xpel	1767258
myov	1679082
bmez	1785971
evt	1253327
bbu	1654795
sybt	835324
aeo	919012
play	1525769
nuv	812801
ampl	1866692
frpt	1611647
bur	1714174
rvlv	1746618
nabl	1834488
cvac	1809122
nmih	1547903
pfs	1178970
ue	1611547
tds	1051512
tsp	1823593
ivt	1307748
irwd	1446847
fhtf	1764443
sasr	824410
gtn	43196
efsc	1025835
nwbi	1471265
trmd	1655891
door	893691
krys	1711279
ttgt	1293282
crs	17843
csgs	1005757
adx	2230
ccyc	1681769
jack	807882
ptxky	1545057
otly	1843586
talo	1724965
bros	1866581
tph	1561680
gpre	1309402
bbio	1743881
nzf	1137887
usac	1522727
task	1829864
algt	1362468
neog	711377
hain	910406
ftai	1590364
gsbd	1572694
napa	1835256
hive	1720424
cake	887596
rcii	933036
crto	1576427
mygn	899923
hope	1128361
cstm	1563411
nvro	1444380
ns	1110805
step	1796022
apps	317788
bdj	1332283
extr	1078271
tgsgy	1437731
lrn	1157408
nac	1074952
frey	1844224
avdx	1858257
olk	1835539
snex	913760
kahc	1843212
mwa	1350593
figs	1846576
slvm	1856485
cvii	1828248
nbtb	790359
mnso	1815846
wre	104894
pjt	1626115
insw	1679049
vrtv	1599489
ntb	1653242
ctos	1709682
ltc	887905
cour	1651562
hlio	1024795
lnn	836157
xncr	1326732
tvtx	1438533
int	789460
bke	885245
cnk	1385280
mrtn	799167
or	1627272
two	1465740
angi	1705110
ako-a	925261
res	742278
aat	1500217
sbgi	912752
mtrn	1104657
dct	1160951
htgc	1280784
gab	794685
vsto	1616318
get	1867325
wrby	1504776
evri	1318568
bcat	1809541
tlry	1731348
stra	1013934
semr	1831840
b	9984
imkta	50493
gshd	1726978
xpro	1575828
mir	1809987
rfp	1393066
evop	1704596
kar	1395942
loma	1711375
msex	66004
cyrx	1124524
irbt	1159167
arco	1508478
ari	1467760
silk	1397702
maxr	1121142
sbh	1368458
bstz	1768666
bvn	1013131
ten	1024725
fcel	886128
ptn	911216
etv	1322436
hlmn	1822492
ai	1577526
tbk	1539638
praa	1185348
xper	1803696
skt	899715
prm	1880319
mux	314203
lob	1462120
gcmg	1819796
alg	897077
ttmi	1116942
btt	1528437
www	110471
rvmd	1628171
dea	1622194
modv	1220754
ltghy	1569963
evtv	1563568
lmnd	1691421
lila	1712184
inbt	1657803
aatp	1713210
me	1804591
md	893949
lgnd	886163
pgre	1605607
egbn	1050441
oxm	75288
swtx	1773427
pty	1190935
bgcp	1094831
ktos	1069258
myrg	700923
road	1718227
sovo	1856608
hmn	850141
pcvx	1649094
esta	1688757
mnro	876427
jbi	1839839
rvt	804116
ecpg	1084961
ihrt	1400891
kymr	1815442
pdm	1042776
mmi	1578732
osis	1039065
nwn	1733998
ccs	1576940
phr	1412408
pay	1841156
hcc	1691303
vbtx	1501570
tgh	1413159
wabc	311094
sats	1415404
dht	1331284
lxu	60714
wow	1701051
pacb	1299130
sfl	1289877
adus	1468328
tmdx	1756262
ng	1173420
jps	1176433
schl	866729
alkt	1529274
cgc	1737927
forty	1045986
wbx	1866501
erii	1421517
ipof	1818873
hurn	1289848
th	1712189
pdi	1510599
eaf	931148
acva	1637873
amk	1591587
fna	1531978
spce	1706946
gel	1022321
ptnr	1096691
bhlb	1108134
evbg	1437352
mei	65270
cldx	744218
ty	99614
air	1750
vgr	59440
vrts	883237
pam	1469395
med	910329
dawn	1845337
pmt	1464423
cprx	1369568
tcbk	356171
opk	944809
amph	1297184
tdcx	1803112
krnt	1625791
vvnt	1713952
prta	1559053
vivo	794172
bdn	790816
idcc	1405495
vcyt	1384101
crncy	909531
usa	799195
zeta	1851003
clne	1368265
dvax	1029142
ramp	733269
bxmx	1298699
cts	26058
pdo	1798618
ladr	1577670
vorb	1843388
bpiry	1437441
forg	1543916
dac	1369241
mlnk	1834494
kro	1257640
cwh	1669779
aiv	922864
ahl-pc	1267395
kind	1846069
bamr	1837429
gty	1052752
clfd	796505
swi	1739942
gnl	1526113
sana	1770121
uan	1425292
gogo	1537054
ofg	1030469
px	1841968
cmp	1227654
gsg	1332174
tbbk	1295401
akr	899629
frg	1528930
fbnc	811589
baly	1747079
tslx	1508655
rock	912562
sndx	1395937
nxrt	1620393
dada	1793862
stc	94344
exfy	1476840
hmy	1023514
cmre	1503584
mnrl	1745797
sos	1346610
afya	1771007
crsr	1743759
dfin	1669811
eat	703351
codi	1345126
clmt	1340122
hni	48287
rily	1464790
meg	1643615
cohu	21535
etg	1270523
chef	1517175
neo	1077183
kfrc	930420
buse	314489
chco	726854
saft	1172052
lc	1409970
gva	861459
ptra	1820630
bgs	1278027
finv	1691445
din	49754
jwsm	1831359
hrt	1859285
fcf	712537
alex	1545654
gsm	1639877
arko	1823794
atax	1059142
dcom	846617
dsgn	1807120
cano	1800682
sava	1069530
tmci	1630627
comp	1563190
evtl	1867102
eoss	1651958
psfe	1833835
dnow	1599617
tigo	912958
cint	1868995
oscr	1568651
gdev	1848739
fvrr	1762301
ias	1842718
cffn	1490906
esrt	1541401
aac	1829432
ebs	1367644
nmfc	1496099
mara	1507605
ssdt	1673504
bynd	1655210
pct	1830033
kn	1587523
cgau	1854640
uctt	1275014
hims	1773751
flgt	1674930
olo	1431695
lpro	1806201
gdyn	1743725
vcel	887359
taro	906338
radi	1810739
enva	1529864
vre	924901
sbsi	705432
veru	863894
cls	1030894
uvv	102037
dgii	854775
pgti	1354327
ocsl	1414932
dsl	1566388
bigc	1626450
ngm	1426332
ssp	832428
ocfc	1004702
kyn	1293613
lpi	1528129
cnxn	1050377
dbi	1319947
sand	1434614
ecvt	1708035
stba	719220
mag	1230992
adpt	1478320
mdgl	1157601
bfly	1804176
mcg	1846510
plus	1022408
srce	34782
cron	1656472
imgn	855654
holi	1357450
tgs	931427
cdmo	704562
ddd	910638
usna	896264
kref	1631596
kdny	1435049
enta	1177648
kalu	811596
nbhc	1475841
fdp	1047340
ostk	1130713
evv	1222922
alx	3499
nxgn	708818
dcph	1654151
amrs	1365916
fsly	1517413
swir	1111863
sbr	710752
blu	1259942
mbuu	1590976
cxw	1070985
rgnx	1590877
agys	78749
zntl	1725160
hlit	851310
htld	799233
wrmk	1609471
hcyt	1591165
rxt	1810019
gden	1071255
krp	1657788
clm	814083
mcri	907242
grbk	1373670
vrnof	1848416
patk	76605
modn	1118417
tnc	97134
avns	1606498
vadp	1700849
posh	1825480
fgen	921299
azul	1432364
agm	845877
aroc	1389050
hees	1339605
mho	799292
spns	885740
bst	1616678
ccrn	1141103
nfj	1260563
trs	842633
ionq	1824920
tmp	1005817
hyt	1222401
nssc	69633
lmat	1158895
btrs	1774155
edit	1650664
clov	1801170
ffwm	1413837
gdot	1386278
pra	1127703
camt	1109138
indi	1841925
slp	1023459
nmz	1266585
caap	1717393
prg	1808834
fpf	1567569
maxn	1796898
stkl	351834
morf	1679363
hbm	1322422
mntv	1739936
rdnt	790526
oflx	1317945
avo	1802974
cbd	1038572
atri	701288
ande	821026
svc	945394
banc	1169770
blfs	834365
nbr	1163739
hera	1842912
lzb	57131
scs	1050825
cubi	1488813
exai	1865408
avid	896841
bchpy	1887388
dfh	1825088
epac	6955
rdwr	1094366
pbfx	1582568
ryi	1481582
rnp	1224450
qrtea	1355096
csr	798359
prlb	1443669
sptn	877422
eras	1761918
dmlp	1172358
cpuh	1828608
nhc	1047335
gic	945114
infn	1138639
dnn	1063259
brsp	1717547
goss	1728117
tdw	98222
sph	1005210
nymt	1273685
cown	1466538
amwl	1393584
sncy	1743907
uuuu	1385849
dm	1754820
prim	1361538
sldp	1844862
ges	912463
momo	1610601
reta	1358762
tgls	1534675
bma	1347426
tgtx	1001316
ssys	1517396
negg	1474627
vlrs	1520504
flnc	1868941
pta	1793882
myte	1831907
hcsg	731012
ego	918608
eig	1379041
wti	1288403
inta	1565687
bbn	1493683
plya	1692412
licy	1828811
parr	821483
arqq	1859690
zuo	1423774
axl	1062231
uec	1334933
sxi	310354
crnx	1658247
ddl	1854545
nrc	70487
mrus	1651311
azz	8947
eqx	1756607
apog	6845
veco	103145
cash	907471
oec	1609804
rckt	1281895
pro	1392972
ccsi	1866633
dcbo	1829959
pfbc	1492165
sumo	1643269
fmtx	1538927
dcgo	1822359
aten	1580808
amcx	1514991
cnob	712771
mttr	1819394
rna	1599901
mgni	1595974
geo	923796
usph	885978
glp	1323468
cdna	1217234
kids	1425450
auph	1600620
btz	1379384
blnk	1429764
hska	1038133
amtb	1734342
snbr	827187
slca	1524741
peri	1338940
brkl	1049782
kura	1422143
pfc	946647
tse	1519061
bfs	907254
aus	1838207
bcor	1068875
sa	1231346
bkd	1332349
asix	1673985
himx	1342338
obnk	1516912
bhg	1671284
tnk	1419945
rlmd	1553643
pll	1728205
astl	1860805
qqqx	1608741
etw	1322435
hesm	1789832
oii	73756
mgi	1273931
fnko	1704711
noah	1499543
hfwa	1046025
brbr	1772016
ra	1655099
cal	14707
siga	1010086
plab	810136
rc	1527590
arqt	1787306
shen	354963
asle	1754170
mnkd	899460
ero	1853860
cet	18748
imax	921582
skyw	793733
bcsf	1655050
qure	1590560
gbx	923120
pubm	1422930
gern	886744
bv	1734713
pdfs	1120914
lwlg	1325964
kros	1664710
agro	1499505
akro	1744659
pump	1680247
nvgs	1581804
rgr	95029
cdre	1860543
geni	1834489
jeld	1674335
nrk	1195739
amsf	1018979
bhe	863436
crlbf	1832928
gof	1380936
cutr	1162461
sii	1512920
valn	1836564
ulh	1308208
fwrg	1789940
splp	1452857
imxi	1683695
ta	1378453
chy	1222719
sboey	1578955
jrvr	1620459
gpro	1500435
dwac	1849635
mtl	1302362
crhc	1818212
umh	752642
vnet	1508475
bbby	886158
eos	1308335
vld	1825079
star	1095651
gam	40417
acrs	1557746
abtx	1642081
cco	1334978
vist	1762506
aosl	1387467
merc	1333274
atec	1350653
par	708821
chct	1631569
ac	1642122
vrex	1681622
om	1484612
schn	912603
adtn	926282
next	1612720
nuvl	1861560
mqy	890196
inva	1080014
acel	1698991
jbss	880117
plrx	1746473
riot	1167419
lpsn	1102993
brog	1774983
wash	737468
pntm	1830392
paya	1819881
rdfn	1382821
ango	1275187
hqh	805267
thry	1556739
apgb	1838337
anf	1018840
kamn	54381
lgv	1832300
accd	1481646
ptlo	1871509
prtc	1782999
osw	1758488
lbai	846901
sgmo	1001233
adv	1776661
qcrh	906465
gb	1799983
vmeo	1837686
dole	1857475
cyxt	1794905
htd	1260041
krny	1617242
clbt	1854587
brmk	1784797
aste	792987
stgw	876883
igr	1268884
pds	1013605
mlab	724004
pebo	318300
twi	899751
orgn	1802457
awf	906013
aod	1379400
opi	1456772
hfro	1710680
rbcaa	921557
enfn	1868912
cmco	1005229
proc	1863362
snpo	1856430
sli	1537137
nmm	1415921
wmc	1465885
clb	1000229
cndt	1677703
zeal	1674988
hbnc	706129
by	1702750
mgic	876779
chi	1171471
efc	1411342
rpay	1720592
repl	1737953
este	10254
hibb	1017480
cii	1278895
wlkp	1604665
thq	1604522
argo	1091748
rocc	77159
vhi	59255
alpc	1616736
ha	1172222
chrs	1512762
rwt	930236
hwkn	46250
spnt	1576018
bcx	1506289
wttr	1693256
pbt	319654
mrc	1439095
fubo	1484769
inn	1497645
vcsa	1874944
fviv	1828183
alec	1653087
drq	1042893
avpt	1777921
gsbc	854560
shyf	743238
cmpr	1262976
silv	1659520
vtex	1793663
ccf	830524
mod	67347
jpc	1216583
utl	755001
capl	1538849
nvta	1501134
amwd	794619
twoh	1494413
ichr	1652535
rapt	1673772
vtol	1525221
ntst	1798100
karo	1828102
mcb	1476034
gprk	1464591
invz	1835654
giii	821002
prdo	1046568
wina	908315
wrld	108385
arce	1740594
cnvy	1787640
crbu	1619856
veon	1468091
rpt	842183
wkme	1847584
cars	1683606
dice	1645569
ahh	1569187
apgt	1353538
scwx	1468666
alp-pq	3153
nrds	1625278
imos	1123134
wnc	879526
hhr	1721181
pnt	1811764
orla	1680056
revg	1687221
dlx	27996
boac	1818089
vvi	884219
matw	63296
ufcs	101199
gol	1291733
cbtx	1473844
tls	320121
fmcb	1085913
hcci	1403431
smp	93389
achr	1824502
avan	1819608
svfa	1828478
jmia	1756708
phar	1828316
sp	1059262
eocw	1843862
ctbi	350852
cgbd	1544206
bsig	1748824
rskd	1851112
myi	883412
clvly	1290877
btwn	1815086
land	1495240
tbph	1583107
brg	1442626
forr	1023313
ysg	1819580
hafc	1109242
coco	1482981
law	1625641
mbin	1629019
hone	1769617
hiii	1823034
tcpc	1370755
nfbk	1493225
eim	1176984
nrix	1549595
cde	215466
audc	1086434
ffc	1174164
lung	1127537
ebix	814549
crnc	1768267
til	1789769
eb	1475115
avxl	1314052
hngr	722723
bcyc	1761612
oxlc	1495222
amal	1823608
dyn	1818794
wetf	880631
igms	1496323
knsa	1730430
uvsp	102212
aeri	1337553
tbi	768899
vygg	1822877
cstl	1447362
pdt	855886
nstg	1401708
cepu	1717161
renn	1509223
itos	1808865
tmst	1598428
scoa	1828985
bry	1705873
dao	1781753
plym	1515816
lgac	1836337
scsc	918965
lifx	1581760
msda	1846996
slam	1838162
lmaca	1831992
anab	1370053
fc	886206
jqc	1227476
hmst	1518715
mor	1340243
lev	1834974
mirm	1759425
mytay	1047564
blnd	1855747
mvis	65770
ubp	1029800
nexa	1713930
krus	1772177
evgo	1821159
nvts	1821769
htbk	1053352
trtx	1630472
sens	1616543
strl	874238
wish	1822250
innv	1834376
rovr	1826018
pzn	1399249
bw	1630805
cresy	1034957
iea	1652362
nwbo	1072379
cers	1020214
huya	1728190
hstm	1095565
hzo	1057060
fpac	1829426
ufpt	914156
gwh	1819438
domo	1505952
sd	1349436
tgi	1021162
ecom	1169652
hlx	866829
sohu	1734107
brw	826020
nrgx	1756908
mov	72573
mvst	1760689
phk	1219360
fax	790500
plow	1287213
crgy	1866175
imtx	1809196
good	1234006
wpcb	1836074
ccvi	1828250
atex	1304492
dly	1788399
nesr	1698514
fsm	1341335
slgc	1837412
nktr	906709
cac	750686
gsev	1828096
gsl	1430725
mtls	1091223
bto	925683
pcok	1452936
nwli	1635984
gmre	1533615
rick	935419
safly	1538028
acet	1720580
brfh	1487197
uht	798783
clw	1441236
goev	1750153
zh	1835724
cfb	1458412
base	1845022
cenx	949157
grc	42682
hbia	732417
ghld	1821160
titn	1409171
atni	879585
atai	1840904
kvsc	1846068
vray	1597313
ambc	874501
arvl	1835059
gco	18498
nrbo	1638287
fhai	1309251
valu	717720
fnma	310522
mcrb	1609809
bjri	1013488
yext	1614178
vmo	884152
gthx	1560241
nx	1423221
leu	1065059
fbms	947559
hzon	1821788
cogt	1622229
ntgr	1122904
lbc	1475348
fedu	1709819
ghrs	1855129
seat	1856031
nndm	1643303
vinp	1826286
fpi	1591670
mhd	1034665
vrna	1657312
agen	1098972
ndmo	1793129
trst	357301
pml	1170299
gldd	1372020
idt	1005731
mye	69488
sva	1084201
bbar	913059
srne	850261
dffn	1053691
slrc	1418076
hlly	1822928
ceva	1173489
scvl	895447
denn	852772
bbdc	1379785
lilm	1855756
osbc	357173
coll	1267565
ffic	923139
ymab	1722964
cpf	701347
pgc	1050743
cfiv	1825249
nlst	1282631
curv	1792781
mbi	814585
rere	1838957
vslax	853180
sibn	1459839
ingn	1294133
big	768835
alcc	1849056
cnsl	1304421
egle	1322439
avd	5981
crai	1053706
bxc	1301787
ccv	1812234
qnst	1117297
chs	897429
eoi	1300391
indt	1037390
untc	798949
wiw	1267902
tcda	1595585
fmbh	700565
acre	1529377
cvgw	1133470
fsnb	1840225
swim	1833197
cldt	1476045
acqr	1837393
rgp	1084765
ruth	1324272
lca	1844642
nid	1557915
mit	1826058
cmps	1816590
tuya	1829118
bbsi	902791
bfc	1746109
anip	1023024
stok	1623526
cchwf	1776738
nkx	1195738
mxct	1287098
nnox	1795251
boe	1320375
nie	1383441
twou	1459417
ewcz	1856236
nfgc	1840616
blx	890541
cmax	1813914
for	1406587
pmvp	1699382
rmgc	1838108
ctic	891293
uslm	82020
abeo	318306
biox	1769484
bit	1562818
sbow	351817
uis	746838
etj	1395325
ipi	1421461
yorw	108985
borr	1715497
thff	714562
arr	1428205
ccap	1633336
casi	895051
lpg	1596993
iesc	1048268
gnk	1326200
cion	1534254
cbl	910612
nat	1000177
iim	885601
nrp	1171486
bhk	1160864
aio	1778114
tile	715787
edn	1395213
ccd	1602584
imvt	1764013
trmr	1849396
crf	33934
tigr	1756699
ngd	800166
lqdt	1235468
aezs	1113423
ldp	1548717
aeva	1789029
agcb	1830232
shcr	1816233
pahc	1069899
faro	917491
etd	896156
ipod	1818876
csii	1180145
pfn	1296250
smwb	1842731
ellky	1437194
ipsc	1850119
kelya	55135
aiki	12239
csv	1016281
vvr	1059386
iag	1203464
alt	1326190
cgem	1789972
swbi	1092796
avir	1593899
hcat	1636422
vaqc	1842386
bird	1653909
thr	1489096
tspq	1838219
hckt	1057379
dba	1383082
revh	1841389
vnda	1347178
eols	1570562
pgen	1356090
gtpb	1842046
lnd	1499849
can	1780652
jout	788329
trns	99302
inbx	1739614
catc	711772
nxdt	1356115
sjt	319655
onl	1873923
smrt	1837014
tpic	1455684
gbl	1060349
imgo	1623715
user	1557127
notv	720154
pixy	1675634
cccc	1662579
cara	1346830
diax	1608742
msbi	1466026
dpg	1515671
janx	1817713
irs	933267
srga	1760173
haac	1824013
mrsn	1442836
prpc	1821329
thw	1635977
tti	844965
dx	826675
alvr	1754068
ccbg	726601
rptx	1808158
iiin	764401
fcbc	859070
trvg	1683825
emd	1227862
hyln	1759631
yala	1794350
pcn	1160990
dhil	909108
clar	913277
fthy	1810523
kc	1795589
zumz	1318008
abst	1071058
bme	1314966
vgm	880892
bzun	1625414
mx	1325702
sfix	1576942
vkq	877463
ccb	1437958
pv	1818787
knte	1797768
gte	1273441
agx	100591
adma	1368514
ytpg	1840859
hsii	1066605
frph	844059
ttcf	1741231
dgnu	1827076
seer	1726445
amswa	713425
eght	1023731
knop	1564180
cdxs	1200375
enmpy	1630511
rada	761238
ggn	1313510
ebf	33002
srg	1628063
desp	1703141
kzr	1645666
arow	717538
tbla	1840502
anzu	1840877
mass	1555279
artna	863110
hbt	775215
traq	1841230
srlp	1525287
tbcp	1815753
hci	1400810
dco	30305
opra	1737450
icpt	1270073
eqd	1818221
ctaq	1823634
lind	1512499
athx	1368148
fbrt	1562528
sxc	1514705
hlth	1628945
acco	712034
lwel	1567098
peo	216851
finm	1823855
nuvb	1811063
snrh	1823854
hhla	1824185
ulnv	1605810
iqi	885125
ble	1176194
alxo	1810182
agac	1833909
olp	712770
vivk	1450704
wkhs	1425287
joff	1824149
ohpa	1836129
sklz	1801661
bhil	1830210
plce	1041859
pdot	1841845
mcbs	1747068
bgb	1546429
gevo	1392380
cass	708781
ephy	1827248
mbwm	1042729
hipo	1828105
fsbc	1275168
mcs	62234
tcva	1845580
exk	1277866
pacx	1829797
sbfm	1402328
hy	1173514
sri	1043337
eqbk	1227500
bgy	1393299
cool	1829953
sgla	1433551
blue	1293971
bmrc	1403475
vrdn	1590750
biol	811240
rway	1653384
fcax	1823733
itrn	1337117
hrtx	818033
bgry	1824734
dxpe	1020710
max	1818383
hyzn	1716583
gcbc	1070524
ldi	1831631
mmd	1518557
spfi	1163668
nstb	1834518
rbot	1812173
tlga	1827871
mmx	1773250
nstd	1835814
nstc	1835817
mbac	1839175
vitl	1579733
amrn	897448
nxj	1087786
aaqc	1838883
hps	1215913
bwb	1341317
myd	879361
kod	1468748
uk	1821424
cio	1593222
sdc	1775625
vsec	102752
axgn	805928
ke	1606757
gut	1080720
hayn	858655
impx	1820191
newt	1587987
wsr	1175535
cvlg	928658
nbb	1478888
onew	1772921
vuzi	1463972
cem	1488775
czoo	1859639
lasr	1124796
ddi	1799567
api	1802883
akya	1711933
bui	1528988
nktx	1787400
vtnr	890447
jfr	1276533
ocul	1393434
xpof	1802156
cnbx	1343009
kgeif	1477081
kop	1315257
blw	1233681
rad	84129
amyt	1783010
fmnb	709337
tetc	1840920
naac	1830063
pflt	1504619
cmbm	1738177
avtx	1534120
ifn	917100
mkfg	1816613
ocgn	1372299
acam	1846125
airs	1870940
hvt	216085
mofg	1412665
drd	1023512
espr	1434868
muc	1051004
lhc	1824153
asts	1780312
wdh	1823986
pbi	78814
nvcn	1399708
gain	1321741
puyi	1750264
mpb	879635
dbo	1383058
gpmt	1703644
ezpw	876523
gim	828803
chuy	1524931
ghy	1554697
crmt	799850
bfst	1624322
smbc	916907
vcv	895531
npk	80172
hpi	1176199
orgo	1661181
bfk	1137393
fdus	1513363
higa	1823776
vera	1831828
care	1829576
iiiv	1728688
dsx	1318885
idya	1676725
tcx	909494
grin	1725293
bvs	1665988
arlo	1736946
jpi	1547994
trda	1689375
ewtx	1710072
aehr	1040470
avk	1219120
atlc	1464343
mvf	835948
pets	1040130
dgica	800457
pntg	1766400
vsta	1792829
fcuv	1590418
mmu	886043
cfvi	1830081
aspn	1145986
enpc	1816261
ino	1055726
rgc	1829667
ilpt	1717307
ficv	1855693
ccne	736772
cpar	1848410
sgh	1616533
pcpc	1824993
cstr	1676479
rbbn	1708055
tpgy	1819399
vpg	1487952
icnc	1858351
rmni	1635282
tup	1008654
mui	1232860
twlv	1819498
eres	1814287
doyu	1762417
rlyb	1739410
adxs	1100397
tpb	1290677
cpaa	1841137
pnnt	1383414
ndac	1837067
tree	1434621
arct	1768224
nqp	870780
srdx	924717
luxa	1823767
dsu	1051003
etb	1308927
senea	88948
cndb	1851961
rmr	1644378
cpsi	1169445
frxb	1840161
mua	901243
chw	1396277
pspc	1843716
igd	1285890
gfx	1823896
amot	46129
jcic	1822312
scob	1838431
flic	740663
gtx	1735707
lxfr	1096056
wuyi	1687065
trc	96869
twni	1835881
abml	1576873
ontf	1110611
ibcp	39311
giix	1841080
tnxp	1430306
cnta	1847903
fact	1838987
glbl	1838615
cswc	17313
lega	1834628
ross	1841610
gsqd	1837207
dhhc	1830188
tnp	1166663
tioa	1846163
hcic	1829455
stk	1471420
oepw	1824677
wsbf	1569994
rxra	1840463
repx	1001614
ofix	884624
mitk	807863
acii	1838614
vtru	1805012
gate	1838513
gnty	1058867
sol	1417892
spgs	1839127
slac	1834755
coli	1829615
acah	1836274
ptoc	1838238
scle	1838697
myps	1823878
macc	1840148
imab	1778016
atra	1604464
fmiv	1842916
qsi	1816431
calt	1795579
trtl	1847112
aerg	879911
huma	1818382
vln	1863006
nfys	1850502
smbk	1038773
zt	1849408
aura	1501796
meso	1345099
evn	1074540
twnd	1814215
strc	1826681
kodk	31235
brdg	1854401
acro	1847891
adct	1771910
ride	1759546
cmcm	1597835
litt	1848364
mcft	1638290
cpac	1221029
bcsa	1873441
bhb	743367
poww	1015383
vlta	1819584
fra	1259708
best	1709505
legh	1436208
cifr	1819989
hcar	1824846
pack	1712463
tyg	1268533
byts	1842566
adse	1879248
rbb	1499422
ghg	1724755
isd	1534880
altg	1759824
dtoc	1839998
cdlx	1666071
ston	1753886
fhtx	1822462
claa	1837739
hsc	45876
rsvr	1824403
trin	1786108
flws	1084869
adtx	1726711
alrs	903419
piai	1819175
brt	14846
up	1819516
ead	1210123
opy	791963
faca	1839550
prsr	1825473
awp	1390195
nml	1562051
pfdr	1839132
bvh	315858
metc	1687187
dfp	1559991
cveo	1590584
mgtx	1735438
kmf	1500096
hlgn	1840292
nl	72162
cydy	1175680
lxrx	1062822
myn	882150
rmax	1581091
lgo	1400438
kvsa	1841873
psag	1833235
ww	105319
bbcp	1703956
ecc	1604174
dcrd	1848959
svm	1340677
fisi	862831
ptgx	1377121
evc	1109116
plmi	1840317
sb	1434754
cybe	768411
tpvg	1580345
irmd	1325618
leo	818972
csta	1834032
cyh	1108109
anac	1829558
slcr	1826553
kpti	1503802
niu	1744781
sdac	1837248
egrx	827871
bh-a	1726173
lgsxy	1447221
gotu	1768259
band	1514416
nerv	1598646
body	1826889
avte	1798749
view	1811856
hcne	1843205
mdxg	1376339
sier	1844135
miy	890393
ht	1063344
tk	911971
mvbf	1277902
sln	1479615
bmdpf	1002788
aan	1821393
emo	1517518
aggr	1842329
ngms	1821349
aldx	1341235
ssaa	1830547
vki	908993
fsd	1494530
argu	1865377
spwh	1132105
dhca	1838163
dnmr	1779020
hmco	1829042
twcb	1845618
rmt	912147
acdi	1850316
muj	1053988
esm	1841420
vel	1692376
sify	1094324
plpc	80035
krnl	1832950
qtrx	1503274
ppt	827773
dxlg	813298
bzh	915840
fsrx	1834336
rrac	1860879
hpf	1189740
fmcc	1026214
nept	1401395
pmo	900422
wtba	1166928
ospn	1044777
jmkj	1624140
altu	1822366
hio	910068
bedu	1696355
eto	1281926
hnst	1530979
zyxi	846475
ocft	1780531
arbe	1861841
lmdx	1685428
igic	1794338
cznc	810958
sbnc	703904
htpa	1828817
jun	1838814
cook	1857853
gpac	1831979
tipt	1393726
wald	1840199
cova	1837160
chaa	1838293
tyra	1863127
tsvt	1860782
hudi	1791725
btai	1720893
dske	1642453
glue	1826457
absi	1672688
htbi	1538263
weav	1609151
fei	1556336
tsib	1832737
fulc	1680581
nc	789933
sdhy	1812923
newp	1369085
pfhd	1630856
gxii	1826669
uve	891166
bsky	1846804
feng	1509646
grvy	1313310
nnmx	1473579
itrm	1659323
hql	884121
ebtc	1018399
shbi	1035092
blua	1831006
gogn	1852940
kalv	1348911
scwo	933972
boom	34067
doug	1878897
bmea	1840439
bkuh	1440153
ree	1843588
kiii	1826059
djco	783412
rrbi	1071236
dray	1852771
ciig	1841338
ske	1713748
rfi	891290
tinv	1820144
amnb	741516
aomr	1766478
tzps	1823481
lgto	1883814
smmf	811808
qiwi	1561566
avah	1832332
klaq	1823323
spkb	1826435
prld	1678660
pfis	1056943
clrm	1831937
lcaa	1841024
dhbc	1838176
avnw	1377789
dnz	1830374
cnd	1824301
ipvi	1839611
hut	1731805
aptm	1845550
arrw	1835972
flme	1831481
ptsi	798287
jri	1539337
inbk	1562463
ampi	1835636
albo	1322505
tpc	77543
heps	1850235
cncn	1191334
pei	77281
lptv	1643988
amrx	1723128
wpca	1836075
hugs	1836894
rxst	1111485
vtiq	1823884
avdl	1012477
cgnt	1824814
lgvc	1879297
cir	1091883
skyt	1819974
groy	1834026
lx	1708259
krt	1758021
ktf	839533
fmao	792966
stre	1838361
kinz	1820875
gbab	1495825
ztr	836412
fwac	1847874
vxrt	72444
ung	1376227
cbay	1042074
gbli	1494904
rmbl	1596961
tdf	919893
supv	1517399
bnr	1792267
tmkr	1821606
tci	733590
toi	1799191
ffie	1805521
asc	1577437
cece	3197
biogy	1572957
kcgi	1865407
aurc	1835856
ggmc	1818880
kair	1824171
pow	1829427
chmx	1657045
bark	1819574
rfmz	1817159
nvsa	1838433
cptk	1827899
ncv	1214935
eh	1759783
rev	887921
epwr	1825720
tgb	878518
disa	1838831
nan	1074769
sgu	1002590
swet	1828914
mdh	1823143
sfst	1090009
eac	1832765
adex	1830029
bfz	1137391
cnda	1851959
roni	1845437
ftcv	1829328
ftvi	1844336
picc	1835800
pmgm	1837607
ftaa	1832696
ftev	1839569
jugg	1842609
prvb	1695357
ader	1822912
iiii	1829889
bgr	1306550
axti	1051627
baca	1869673
gtpa	1837105
jro	1289213
eft	1288992
glad	1143513
hawel	46207
ftpa	1839972
plm	866028
cpz	1717457
onyx	1849548
lvra	1841383
tg	850429
rlgt	1171155
cbnk	1419536
cyd	932695
loco	1606366
blde	1779128
evlv	1805385
joan	1834585
gilt	897322
usws	1670349
hdsn	925528
belfa	729580
ncac	1849475
txcb	1006840
isrl	1915328
nmg	1649752
ffa	1291334
bhr	1574085
tars	1819790
rkta	1844363
anik	898437
fsp	1031316
soi	1697500
grpn	1490281
aawh	1756390
opad	1825024
pwp	1777835
ipvf	1839610
min	826735
cste	1504379
rsi	1793659
sght	1531177
bpt	850033
mhn	1038186
puck	1836100
utmd	706698
pond	1842430
ctv	1835378
srrk	1727196
advv	1698519
glop	1598655
efr	1258623
ctlp	896429
mpx	1129155
ioac	1854275
mtal	1853021
pegr	1847241
cffe	1839530
neti	1587264
mcbc	1053584
blfy	1846017
gaxy	1127993
blbd	1589526
nlcp	1854964
aris	1865187
ogi	1620737
cosg	1706509
dtc	1870600
jmsb	1710482
ipax	1844452
noa	1368519
reax	1862461
ormp	1176309
ftci	1828161
caf	1368493
ipva	1839608
cru	1825497
hlah	1826814
hbcp	1436425
civb	944745
clas	1826855
tblt	1668370
fssi	1822862
iacc	1844579
ssti	1351636
pfta	1853580
wetg	1784970
ello	946394
vpcb	1840792
hmpt	1830197
bsrr	1130144
mtw	61986
imrx	1790340
phat	1783183
cang	1725123
bcab	1826892
hpx	1809353
hix	1058239
flya	1844650
afb	1162027
slvr	1842644
bym	1181187
tngx	1819133
rm	1519401
asaq	1819510
abus	1447028
dnad	1850272
cmtl	23197
dnab	1850271
tekk	1822027
fnlc	765207
npab	1837929
rclf	1833498
lokm	1841585
whf	1552198
frst	1325670
sor	91847
jushf	1909747
dhc	1075415
nwpx	1001385
gbio	1733294
mtac	1826667
lqda	1819576
aht	1232582
co	1467808
ibex	1720420
cplp	1392326
dzsi	1101680
hcii	1823033
mon	1828325
hmlp	1603016
pmm	844790
phvs	1830487
vlgea	103595
bcbp	1228454
scaq	1821812
acmr	1680062
cmpx	1738021
acb	1683541
fof	1375340
rgls	1505512
aftr	1865975
gci	1579684
esq	1531031
hwel	1845013
ooma	1327688
rmm	1771226
adal	1853928
amrk	1591588
trssf	1778129
nhhs	1503707
gpp	1635650
prth	1653558
msb	65172
oia	835333
asg	786035
gecc	1675033
czfs	739421
tcs	1411688
afmd	1608390
prts	1378950
nrac	1831964
crd-a	25475
gwrs	1434728
casa	1333835
dsm	855887
gia	1844505
osur	1116463
dmys	1858327
aeac	1827392
clim	1835713
pfl	1244183
zvia	1854139
bwmx	1788257
petz	1684425
fdmt	1650648
skil	1774675
yell	716006
bwc	1854863
inaq	1862463
strs	885508
nca	818851
ml	1807846
meka	1870258
upld	1505155
omga	1850838
pmx	1181506
ryam	1597672
thcp	1843993
acev	1813658
dmrc	1438231
gwii	1850487
mcaa	1856995
ahpa	1846253
sbii	1842729
fen	1284940
toac	1860482
svfb	1837238
sbt	1680379
akic	1826574
mgu	1326390
pcct	1844149
optn	1494650
htaq	1850038
dtrt	1865537
usct	1860514
vaxx	1851657
ampy	1533924
fron	1842223
clrb	1279704
ardc	1515324
dsaq	1871745
lndc	1005286
cors	1842659
osg	75208
sar	1377936
inse	1615063
cbrg	1845149
dhx	1393883
kaii	1825962
svna	1850699
wrac	1855168
irrx	1854795
dpcs	1857803
pcsb	1691337
rcfa	1870143
flag	1855485
inve	1036044
aveo	1325879
nen	746514
ictsf	1010134
edd	1388141
shca	1848558
mpra	1853436
fgpr	922358
wwac	1853044
lhaa	1841948
fiac	1854480
arbg	1823826
noac	1824888
xfin	1852749
rei	1384195
leju	1596856
acnb	715579
lcw	1847577
paqc	1830531
ldha	1842373
afcg	1822523
ptic	1821075
ixaq	1852019
vii	1826011
iqmd	1850051
itq	1805508
tris	1852736
biot	1825413
ngvc	1547459
prch	1784535
unty	920427
petq	1668673
allk	1564824
bhac	1851612
fvt	1824434
gnac	1832250
shac	1820160
blts	1827328
zy	1645842
tmac	1835236
zeus	917470
orrf	826154
ostr	1834226
zwrk	1828438
vlat	1843091
pgss	1861541
boas	1838544
iif	916618
vcxa	1848898
velo	1832371
nn	1865631
fbiz	1521951
hwkz	1841144
fanh	1413855
bkcc	1326003
osi	1832136
mcr	851170
hyi	1497186
pmf	1140392
clpt	1285550
pstl	1759774
thrn	1844280
pcq	1140411
eex	1579214
spxx	1338561
amtx	738214
nath	69733
alto	778164
smlr	1554859
le	799288
tdup	1484778
oprx	1448431
pcb	1423869
scpl	1760717
cala	1496671
ffntf	1783875
ois	1121484
egy	894627
cwco	928340
urg	1375205
mpaa	918251
edap	1041934
eggf	1843973
phci	1620749
naut	1808805
afaq	1841661
giw	1848795
shqa	1844908
bkt	832327
dibs	1600641
tgan	1715768
powl	80420
mist	1408443
ff	1337298
nodk	1681206
frsg	1842883
mci	275694
skya	1847152
fzt	1839824
insg	1022652
uxin	1729173
glo	1350869
itic	720858
grts	1656634
pipp	1822835
glre	1385613
atro	8063
bcov	1313275
twoa	1843988
eypt	1314102
eqha	1826729
glha	1841734
kron	1741830
ciso	1777319
mulg	1629665
mmt	809173
nr	71829
epm	1006655
uscb	1901637
erc	1227073
bgfv	1156388
xpax	1853397
bgh	1521404
aadi	1422142
maca	1835416
stro	1382101
sunl	1821850
celu	1752828
yi	1738906
aimd	1014763
cix	1049606
thrx	1745020
trca	1840353
jgh	1615905
scm	1551901
eot	1454741
grwg	1604868
fvcb	1675644
azre	1633438
nrim	1163370
pkbk	1315399
fct	1282850
wavc	1849580
avac	1836478
ocn	873860
atom	1420520
iii	1371489
bbln	1866390
sgsvf	1398832
mqt	887394
dbd	28823
aef	846676
frw	1825739
vktx	1607678
vhaq	1823857
asa	1230869
opt	1815620
bny	1137390
dbvt	1613780
pvbc	1778784
otlk	1649989
kbal	55772
gamb	1839799
ener	1855555
dlca	1831928
prty	1592058
eigr	1305253
conn	1223389
kltr	1432133
uroy	1711570
silc	916793
jrs	1158289
ednc	1864891
fif	1513789
ueic	101984
atcx	1751143
refi	1867949
idex	837852
hov	357294
wqga	1851174
pyr	1743344
vygr	1640266
phyt	1848756
mlr	924822
ghac	1806156
jrvs	1556801
rchg	1817786
gvci	1858503
adap	1621227
smih	1839185
mlai	1851625
dmb	1565381
bite	1831270
nvec	724910
hplt	1863181
crzn	1844635
plnhf	1813452
arte	1839990
zyme	1403752
apac	1844981
gne	1528356
rngr	1699039
locc	1848323
cdaq	1851909
bgt	1287480
ncz	1227857
xair	1641631
abos	1576885
lian	1831283
dals	1863294
pulm	1574235
brbs	842717
omer	1285819
srsa	1821682
tcbx	1781730
lsea	1721386
qfta	1830795
alco	3545
hrow	1360214
opa	1843121
tbsa	1843764
seda	1846975
hrzn	1487428
iren	1878848
locl	1840780
vrm	1580864
mue	1071899
oria	1847416
bgsx	1838666
ssbk	1689731
mrns	1267813
pbfs	1769663
jynt	1612630
piii	1832511
pear	1835567
ever	1640428
vpv	895528
bli	1689657
ags	1593548
veri	1615165
bwfg	1505732
kio	1515940
vldr	1745317
annx	1528115
nuw	1450445
bkn	894242
xos	1819493
ayrwf	1847462
myfw	1327607
curo	1711291
zom	1684144
kmph	1434647
mmat	1431959
egan	1066194
cnlhn	23426
omic	1850906
smfr	1818331
npv	897421
qd	1692705
psf	1498612
catn	1632275
bmac	1848020
imaq	1846235
atip	1815849
bitf	1812477
nrdy	1819404
vcka	1820190
cnfn	1352952
tei	909112
gacq	1846288
aip	1667011
alpn	1626199
wewa	1616156
fll	891482
fgbi	1408534
spne	1637761
dlth	1649744
pcyo	276720
tsi	809559
cian	1867752
fxlv	1788717
tmc	1798562
scu	1403256
acbm	1622996
fph	1574197
apph	1807707
fdbc	1098151
smti	714256
cnce	1367920
bbw	1113809
rtsl	1575659
wilc	1030997
crge	1277250
tron	1847513
mn	1524223
vmd	1729149
pbax	1870404
lmnr	1342423
pcnt	1504239
maqc	1844419
pana	1828989
scd	1270131
pshif	1538169
rbtk	1594204
vtdrf	1465872
vmga	1875943
dbl	1525201
nsrcf	1302084
tsbk	1046050
arl	1102238
wia	1254370
panl	1606909
hglb	1622148
uone	1041657
wldn	1370450
pstx	1661460
mvt	897269
pmts	1641614
lvlu	1780201
nuo	878200
stks	1399520
weys	106532
apen	1251769
gpja	41091
acaq	1869141
rigl	1034842
rdvt	1720116
pke	76267
hffg	1680873
jakk	1009829
ndls	1275158
nxp	883618
cnty	911147
meac	1849902
rner	1854461
tga	736744
xers	1867096
fund	825202
swss	1838000
fsbw	1530249
cpss	889609
tpba	1847090
bksy	1753539
essa	1382230
bnft	1576169
cvcy	1127371
batl	1282648
oxus	1852973
lctx	876343
loop	1504678
lyts	763532
selb	1453687
evm	1177161
vstm	1526119
kbrs	1411059
kmda	1567529
obt	1754226
tlys	1524025
snce	1819113
gapa	1839121
thac	1863685
pmvc	1807765
aka	1865107
isaa	1845601
cenq	1841425
vff	1584549
fnvt	1857855
acv	1636289
oust	1816581
jce	1385763
arck	1855886
nwfl	1013272
pck	1170300
asax	1824211
roc	1884516
afac	1849489
mfm	801961
briv	1844211
apn	1849011
thtx	1512717
mgnx	1125345
dwin	1812360
lcnb	1074902
ob	1454938
pgrw	1833213
wlfc	1018164
inka	1826671
fami	1701261
swkh	1089907
etx	1563696
phic	1825724
ger	1612875
ctr	1547341
ksi	1843351
bsvn	1746129
xela	1620179
ptmn	1372807
lidr	1818644
ll	1396033
uti	1261654
sdsya	1163609
grph	1815776
wism	1756640
sptk	1836259
eod	1386067
vbiv	764195
bzfd	1828972
real	1573221
ntg	1490286
dune	1817232
ngc	1843249
urgn	1668243
apyx	719135
lvox	1723648
arbk	1841675
mogu	1743971
clbr	1847064
gfor	1845459
xcur	1698530
rent	1468327
istr	1602658
virx	1061027
fins	1745059
ajx	1614806
necb	1847398
moln	1745114
quot	1115128
xoma	791908
bsaq	1851908
actg	934549
evbn	842518
mimo	1823882
mxf	65433
gnft	1757064
mhi	1223026
sev	1840416
aaci	1844817
xxii	1347858
ggt	921671
stsa	1692830
nksh	796534
niobf	1512228
bbai	1836981
cato	18255
enj	71508
ipha	1598599
gnus	1355848
sama	1843100
tusk	1679268
xflt	1703079
atlo	1132651
pht	1166258
mixt	1576914
ihit	1682811
rell	355948
pine	1786117
efl	1701167
hqi	1140102
nbh	1178839
chmg	763563
aft	1502573
aku	1776197
hsaq	1814114
ftf	1233087
akts	1584754
emp	66901
autl	1730463
dbb	1383084
ufi	100726
bglc	1737523
mav	1258943
slqt	1794783
aciu	1651625
dhy	1061353
lgi	1278211
bcml	1730984
emx	1285786
astr	1814329
aray	1138723
vtn	883265
bnnr	1852332
wve	1631574
fosl	883569
incr	1857030
seovf	1491434
tig	1801754
teaf	1704299
glt	41719
jaqc	1817868
rose	1870129
vwe	1834045
cmrx	1117480
ft	833040
tcfc	855874
ccrd	320340
opp	1678130
ctgo	1502377
immr	1058811
scor	1158172
cvgi	1290900
lazy	1721741
prpl	1643953
alpa	1842939
gdo	1472341
em	1834253
ataq	1841004
cmls	1058623
blng	1842384
pfsw	1095315
wrn	1364125
frgi	1534992
hie	1519505
abgi	1833764
hil	1287808
ttsh	1552800
agil	1790625
aryd	1838821
odc	74046
cvly	806279
dlhc	785557
wneb	1157647
aagh	1098009
blsa	1823200
mhld	1412100
arye	1852432
fxe	1328598
ltrx	1114925
racb	1819724
emf	809708
ngl	1504461
tbnk	1447051
nro	1261166
ikna	1835579
uoli	1081834
ocax	1820175
ram	1841867
cban	711669
nsl	1093428
clnn	1822791
ivh	1568292
cnf	1733868
riv	1501072
quad	1481792
pkoh	76282
mfin	1000209
sppi	831547
aif	1526697
vor	1817229
frbk	834285
bprn	1913971
amci	1843724
cloe	1849058
qsam	1310527
mlac	1807594
doma	1722438
ccai	1846968
ltch	1826000
mita	1847440
evi	65312
piivx	1557265
mrbk	1750735
aenz	1572621
nyxh	1857190
igi	1462586
bwmn	1847590
grx	1391437
nref	1786248
lcut	874396
syn	894158
adth	1838672
fdeu	1646109
apmi	1857662
gnln	1743745
mnsb	1693577
vect	1836379
itegy	1579597
envb	890821
wprt	1370416
ytra	1516899
rcky	895456
bbq	1021270
mrcc	1512931
oxsq	1259429
clsn	749647
kvhi	1007587
ela	701719
arec	1590715
ctxr	1506251
rani	1856725
sgii	1869824
cvm	725363
mlp	63330
aprn	1701114
cia	24090
rgco	1069533
glta	1852767
hnrg	788965
voxx	807707
camp	730255
sbtx	1671858
atnm	1388320
gsqb	1849280
mrmd	1522767
clvs	1466301
thca	1773087
bsl	1486298
rdw	1819810
gato	1517006
prbm	1848165
metx	1796514
asps	1462418
flac	1828326
pnrg	56868
anvs	1477845
bbxia	1814974
qk	1769256
hyw	1785680
vbf	5094
omeg	1832010
pim	830622
opbk	1722010
wmpn	1828376
twn	804123
isba	842517
samg	1549966
pzc	1181504
vabk	1572334
artl	1621221
bset	10329
hoft	1077688
fenc	1211583
natr	275053
crnt	1119769
alya	1734520
ljaq	1825437
cclp	1449488
plby	1803914
psnl	1527753
free	1753706
rmbi	1767837
jatt	1855644
jof	859796
linc	1286613
basi	1330016
soph	1840706
nviv	1292519
onds	1646188
true	1327318
slno	1484565
mta	1722606
iber	1835205
pxs	1640043
oyst	1720725
insi	30125
phun	1665300
calb	1752036
kfs	1072627
grrb	1126961
nunz	886093
svra	1160308
pepl	1873324
mhh	1437226
flc	1245648
impl	1445499
jill	1687932
dgl	1383055
rain	1724979
tdac	1926599
icvx	1786255
wimi	1770088
plbc	1168455
howl	1785530
clsk	827876
scma	1863428
adn	1744494
when	943535
cvrx	1235912
mpa	891038
ptrs	832090
bwac	1821146
rex	744187
brqs	1650575
raas	1804583
tcmd	1027838
cfnb	803016
niq	1564584
tnya	1858848
fncb	1035976
ntp	829365
prph	868278
nmmc	1819157
olma	1750284
esea	1341170
dmaq	1857086
mvo	1371782
mepw	1627452
kndi	1316517
dmtk	1651944
talk	1803901
eglx	1854233
blze	1462056
twin	100378
lybc	816332
cbh	1701809
fsfg	1435508
rdcm	1016838
nbst	1831978
sgc	95574
enx	1177162
mbcn	836147
cntq	1847986
spir	1816017
meoa	1859310
rcmt	700841
tern	1831363
yj	1759614
akus	1722271
cofs	803164
ebac	1841258
bsbk	1787414
rcel	1762303
fnwb	1556727
cctc	1445109
hyb	825345
dhf	1057861
epsn	1726126
lion	1845991
ctrn	1318484
mcn	1289868
tals	1827506
gtii	356590
altb	1642365
spe	897802
vntr	1705682
pwod	716605
solo	1637736
owlt	1816708
luna	1239819
ehi	1228509
emyb	1449794
smhi	1690334
mots	1686850
cmct	908311
ide	1417802
apei	1201792
jnce	1640455
kore	1855457
hurc	315374
ccts	1865861
iva	1756594
extn	1635881
dbe	1383062
pdlb	1874071
hbb	1709164
prds	1822711
nny	818850
xl	1772720
zev	1802749
ppta	1526243
sga	886136
cwgl	1562151
ohaa	1870778
glg	1556266
infu	1337013
kulr	1662684
fpl	1589420
bcli	1137883
naz	892992
stxs	1289340
lsac	1869365
vnja	1532383
dkdca	1849380
oprt	1538716
ardx	1437402
bene	1828735
bkkt	1820302
sj	1753673
maps	1779474
mtnb	1582554
evoj	1834342
rpid	1380106
ovly	1431567
asys	720500
mbtc	1837344
gmda	1600847
nbtx	1760854
nlit	1854963
cik	810766
ssss	1509470
hwbk	893847
pbpb	1195734
brez	1817640
clls	1627281
dhac	1864531
ata	1825254
rlfty	1854078
rmti	1041024
smap	1863990
smsi	948708
rpmt	1437283
liby	1880151
ovid	1636651
kwac	1823086
glli	1888734
byrn	1354866
grcl	1826492
pays	1496443
rvsb	1041368
rocg	1855447
qipt	1540013
glq	1316463
evg	1287498
heq	1496749
genc	64472
sevn	1452477
gdnsf	1771706
bbig	1717556
iga	1332943
allt	1365767
bmaq	1867757
edtx	1817153
nrt	72633
bgx	1504234
pono	1855631
lmst	1358356
inmb	1711754
hrbr	899394
tmrc	1445942
sang	1753368
mmlp	1176334
oxac	1861622
fat	1705012
cps	1320461
mtry	1860663
giac	1853314
sqns	1383395
tffp	1733413
qrhc	1442236
bta	1343793
issc	836690
troo	1412095
esca	33488
atxg	1650101
dnif	1059213
winv	1854463
fatp	1865045
goro	1160791
tgvc	1865191
kldi	1752474
fet	1401257
nak	1164771
avya	1418100
spok	1289945
bned	1634117
adra	1823584
msc	1713334
ftco	1828377
bwg	1504545
zuus	1687926
erlff	1271554
bkgm	1527383
drrx	1082038
pebk	1093672
capr	1133869
mhf	830487
immp	1506184
rkly	1852117
srt	1031029
cdro	1866782
caas	1157762
ofs	1487918
csse	1679063
amsc	880807
ofed	1501078
fraf	723646
sttk	1680367
jyac	1824119
crt	881787
acp	1503290
mg	1436126
jmac	1874259
utsi	1030471
tjbh	1499785
reve	1853651
bq	1815021
mtrx	866273
ffnw	1401564
dbp	1383057
cfbk	1070680
cdzi	727273
mec	1766368
dbtx	1656536
amao	1843656
ehth	1333493
iti	350868
asxc	876378
inm	1728328
glsi	1799788
smmt	1599298
snda	1043000
sach	1682220
fusn	1805890
ghl	1282977
pfmt	1550695
tsq	1499832
rndb	1667161
msd	904112
pfd	868578
fath	1836176
glee	1843248
seel	1017491
myna	1850453
avbh	1443575
snfca	318673
eucr	1822929
aeha	1869974
aout	1808997
slgl	1684693
dmf	839122
anix	715446
plg	1095052
avgr	1506928
lmao	1831868
sesn	1485003
fxy	1353613
ovbc	894671
fcco	932781
atha	1620463
dakt	915779
eose	1805077
crmd	1410098
jhs	759866
chn	845379
clpr	1649096
oric	1796280
otec	1846809
dmo	1478102
vsac	1883983
wndw	1071840
jfin	1743102
ebmt	1478454
fstx	1566373
mpv	831655
banx	1578987
goco	1808220
dpls	866439
smwpy	1455634
jpt	1679033
czwi	1367859
mymd	1321834
aty	1861233
orrcf	1390352
gldg	1538847
frla	1849294
ltrpa	1606745
hysr	1481028
bfin	1303942
mitt	1514281
dms	1725134
ngs	1084991
opof	740971
ae	2178
aphe	855787
scph	1604950
bctx	1610820
prax	1689548
usx	923571
arc	1305168
usdp	1610682
grnq	1597846
efsi	880641
hear	1493761
beem	1398805
vtaq	1822145
brid	14177
fkys	737875
finw	1856365
mram	1438423
fnch	1733257
wstg	945983
fstr	352825
hbio	1123494
gdlc	1729997
tmpm	1823524
wea	1163792
lark	1141688
hyfm	1695295
dlng	1578453
avhi	1844507
smid	924719
sppp	1539190
ctrm	1720161
atxs	1454789
zepp	1720446
fmn	1199004
bgsf	1474903
nhs	1487610
trhc	1651561
xbit	1626878
frevs	36840
prtk	1178711
fnwd	919864
sbfg	767405
cpkf	847538
jsd	1509253
infi	1113148
ifrx	1708688
cfcx	1489803
ecf	793040
cbat	1117171
jfu	1619544
tela	1561921
pbhc	1609065
brds	1861449
vacc	1828185
adoc	1824884
rrgb	1171759
dla	1101396
cwbc	1051343
gf	858706
sncr	1131554
cbca	1554906
celc	1603454
qh	1781193
leat	1456189
tsha	1806310
eml	31107
mlvf	1550603
tkno	1850902
mdxh	1872529
etnb	1785173
achl	1830749
gan	1799332
ivac	1001902
trx	1173643
rphm	1637715
hgtxu	862022
fnrn	1114927
csbr	771856
voc	1505413
cto	23795
pfo	882071
tglo	1066684
via	1606268
agd	1362481
qoeg	1439237
flnt	1460329
alpp	1606698
cmcl	766011
atos	1488039
cyrn	1084577
erh	1279014
mreo	1719714
gass	1328919
ltmaq	1047716
tsat	1845840
srl	16859
kplt	1785424
szc	1506488
wulf	1083301
phx	315131
mnp	894351
mnts	1781162
func	763907
msac	1836517
zhybf	1672886
nanx	883107
cxe	845606
jhi	759828
smlp	1549922
fora	1829280
abit	1095146
kopn	771266
axdx	727207
inte	1850262
ffmgf	1641229
nim	890119
tggi	1891791
sotk	806172
cbfv	1605301
pai	75398
cgo	1285650
ctg	23111
xyf	1725033
ubfo	1137547
evlo	1694665
rdi	716634
vra	1495320
seznl	1662991
chmi	1571776
pflc	1093728
pvl	1520048
ribt	1063537
litb	1523836
kf	748691
lbao	1586452
pdsb	1472091
phd	1305767
vate	1006837
mgf	811922
sfbc	1541119
cybn	1833141
gdl	1378701
sirc	1756704
gnss	924383
lax	1847846
gau	1377757
fvam	1823465
unb	706863
alzn	1677077
gwso	1430300
lee	58361
rcat	748268
sunw	1172631
intg	69422
enz	316253
armc	1415813
sbi	882300
jg	1737339
asrt	1808665
pcx	1855751
nnbr	918541
dcf	1627854
xnet	1510593
prof	1628808
ctmx	1501989
irnt	1777946
advm	1501756
mnov	1226616
dit	928465
krmd	704440
nkg	1177219
very	1575434
enob	1527728
cydvf	1394039
nine	1532286
prt	1724009
pwfl	1774170
ckpt	1651407
galt	1133416
flxs	37472
rbkb	1751783
cphc	1672909
xspa	1410428
ntic	875582
asur	884144
utgn	832480
pcyg	50471
nxtc	1661059
prov	1010470
silef	1545224
imux	1280776
rmi	1746967
iaf	779336
prtg	1095435
pbyi	1401667
nmt	897419
vrca	1660334
bcv	9521
ihd	1496292
lptx	1509745
hmnf	921183
cue	1645460
csbb	880417
kxin	1713539
duo	1750593
fixx	1661998
sup	95552
fbio	1429260
ih	1814423
rrts	1440024
mgta	1690585
mynz	1874252
enbp	1437479
afbi	1823406
gmgi	1437925
ship	1448397
jdvb	866643
crkm	1700357
trtk	1560905
tast	809248
clay	1855467
bgi	1179821
nuvr	71557
fxnc	719402
cmtv	718413
fosyf	1336835
btbt	1710350
rfm	1790177
cdxc	1386570
qnbc	750558
dtil	1357874
earn	1560672
exd	1480999
elmd	1488917
fthm	1753162
asmb	1426800
gcv	845611
ghm	716314
wejo	1864448
gdrzf	1072725
pirs	1583648
mntx	1302028
aaic	1209028
swz	813623
smts	1705259
lgvn	1721484
epix	1633932
pavm	1624326
npce	1528287
cadl	1841387
canb	1509957
azyo	1708527
vcif	1517767
fcrd	1464963
reph	1588972
vrdr	1506929
eaco	784539
fpocf	1177439
ggz	1585855
enlv	1596812
move	1734750
evf	1070732
hson	1210708
farm	34563
strt	933034
ds	1175483
bhse	1759186
fonr	355019
bfgx	1741257
ksm	846596
ubcp	731653
life	1339970
gnta	1838716
watt	1575793
fdoc	1459188
bdtx	1701541
sy	1758530
codx	1692415
ipa	1715925
bnix	1845942
jemd	1708261
pni	1170311
mays	54187
hnw	1388126
gth	1782594
acu	2098
dtii	1533357
ueec	1096938
pfx	1490349
hhs	45919
expr	1483510
cmt	1026655
tcrr	1750019
mpir	815577
whg	1165002
pcti	1057083
vgi	1528811
slng	1043186
pxlw	1040161
axr	6207
jls	1472215
hsto	1383701
ecrp	1696898
mgyr	1337068
lway	814586
knit	1696195
fcap	1070296
alot	8146
foa	1828937
htoo	1819794
gsmg	1738758
cizn	1075706
bixt	1445815
lake	798081
cmu	809844
bksc	1007273
oily	1367408
ebon	1799290
thm	1134115
mict	854800
yrd	1631761
vhc	1082324
idba	1824036
mcag	1859035
intt	1036262
itp	1358190
curi	1776909
sfdl	818677
szsmf	1548536
cen	1576340
ftk	928054
gramf	1876945
ggrou	1489874
aneb	1815974
mpvdf	1004530
augg	1448597
acr	1332551
cidm	1173204
tour	1597095
miro	1527096
cgen	1119774
asre	1231339
gceh	748790
ariz	1882078
pasg	1787297
inkt	1840229
yq	1821468
tpz	1408201
nobh	72205
ped	1141197
brag	1867834
emcf	858800
inod	903651
augx	1769804
cjjd	1856084
aubn	750574
gfgy	1181749
hcwb	1828673
dex	1396167
armp	921114
chra	1730346
nmi	830271
glv	1288795
xlo	1840233
gnt	1438893
oiim	1095348
ncmi	1377630
qmco	709283
cih	1749797
viot	1742770
gree	1844971
nsrpf	1536674
glu	1282957
clgn	1631487
uph	1770141
pnckf	877019
elym	1768446
bcrd	1496690
pmhg	1586454
dare	1401914
hrtg	1598665
hypr	1833769
nxc	885732
cntb	1835268
pgz	1557523
hiti	1847409
iobt	1865494
occi	1716951
ctso	1175151
cris	1108205
qubt	1758009
airg	1272842
ocx	1642380
orn	1402829
pgp	1318025
stcn	914712
ffbw	1787384
cull	1845799
intz	736012
pbts	1754323
nms	1607997
mogo	1602842
sqz	1604477
agfs	1592016
fmbm	740806
srv	1400897
lchd	1715433
tlsa	1723069
irme	1839133
dtf	879535
bfnh	1310488
mfd	1276469
pfbx	770460
ipwr	1507957
ulbi	875657
mugh	1746119
trvi	1563880
lvo	1491419
spk	1848097
gnpx	1595248
ncna	1709626
quik	882508
byfc	1001171
ntz	900391
lmb	1606163
sybx	1527599
bcel	1532346
inzy	1693011
snd	1529628
clsd	1539029
cptp	202947
lkco	1487839
iso	1615055
lqmt	1141240
fvti	1626745
pme	1517130
apcx	1070050
pyxs	1782223
mtbc	1582982
cyt	1662244
vnce	1579157
ap	6176
mesa	810332
gbrg	1822792
stim	1227636
lyra	1327273
tcrx	1783328
blrx	1498403
tedu	1592560
mchx	1224133
bsga	1853084
rmo	1757932
vtvt	1641489
iae	1385632
cmpo	1823144
xgn	1274737
novv	1858028
pte	1076682
usas	1286973
job	40570
emkr	808326
juvf	714712
isdr	843006
hhgc	1822886
auid	1534154
wrap	1702924
lsbk	1341318
triq	1514056
virc	751365
pphp	1863460
pafo	1858414
lucd	1799011
bmtx	1725872
gstx	1497649
dila	1843608
ppih	914122
crawa	47307
aspa	1854583
znog	1131312
mcaf	1853774
flgc	1790169
lnkb	1756701
lbti	1384135
vrar	1854445
lfmd	948320
blgo	880242
pkkw	1657642
lbph	1832168
nvno	1661053
ccm	1472072
atto	1606457
naii	787253
foxw	1829999
evok	1403708
essc	1760683
rvp	946563
aqb	1603978
cev	1074692
refr	793524
vbfc	1290476
crdl	1702123
cei	1309082
igny	1834645
gsit	1126741
bafn	1649739
axla	1633070
sund	1171838
mpac	1841993
arkr	779544
jeq	866095
freq	1703647
fuv	1558583
klxe	1738827
tzoo	1133311
oncy	1129928
edf	1501103
pnf	1140410
laaa	1848000
lghl	1806524
hexo	1690947
jhaa	1753217
gp	1584547
lumo	1126234
ptbs	925173
srzn	1824893
gpco	1841209
sldb	1707502
pdex	788920
mcae	1853775
crdf	1213037
clst	1849867
acrhf	1762359
lov	1705338
lhdx	1652724
usap	931584
zenv	1836934
cwbhf	1750155
tcbc	1850398
bcow	1847360
erdcf	1398972
pcf	810943
vlt	846671
sfe	86115
dpsi	1505611
scyx	1178253
lngb	1602706
drio	1533998
ivda	1397183
mrzm	1413754
aaoi	1158114
strm	1008586
sal	1060219
hnna	1145255
hcmc	844856
ahg	1702318
icmb	1578348
bolt	1641281
bmtm	1568385
fhld	1386044
rail	1320854
asrv	707605
ibio	1420720
ncsm	1692427
aeye	1362190
dyai	1213809
gaia	1089872
trrb	1498232
areb	1648087
meip	1262104
nls	1078207
ttp	1526329
koss	56701
hofv	1708176
crym	1533030
sbev	1553788
vtsi	1085243
oss	1394056
gifi	1031623
cxh	847411
bway	1505065
lgl	61004
brlt	1866757
klr	1719489
scx	93676
sft	1762322
bnox	1191070
bvfl	1302387
loan	1080340
rfil	740664
akba	1517022
viri	1818844
ades	1515156
vihd	1676580
brli	1787518
ear	1719395
eq	1746466
drrsf	1421570
boxd	1828672
cga	857949
ddf	896923
staf	1499717
imra	1672619
fsea	1769267
glxz	13156
gtbp	109657
mmnff	1776932
acba	1844389
bivi	1580149
bfi	1723580
hfbl	1500375
vena	1800392
bnet	875729
icad	749660
wscrf	795800
wfcf	1360565
cbkm	1006830
apto	882361
eyen	1682639
lkcru	1156174
ruby	1709401
dnay	1850079
bstg	1563665
turn	893739
benh	1568139
adag	1818838
cbio	1124105
pesi	891532
gsvrf	865400
aqms	1621832
ortx	1748907
glto	1800315
mktw	1805651
imnm	1472012
kffb	1297341
aame	8177
snse	1829802
nbw	1178840
geos	1001115
srax	1538217
vgz	783324
bhat	1759136
gmtx	1816736
dsp	1828791
ktel	845819
jrsh	1696558
crws	25895
surf	1718108
imv	1734768
jwel	1805594
abti	1442999
lvtx	1840748
tatt	808439
self	1031235
nh	1566469
ccel	862692
lrmr	1374690
hgbl	849145
inuv	829323
fusb	717806
surg	1392694
cxdo	1075736
nmtr	1551986
vlns	1859902
ibtn	1379329
uptd	1844417
mack	1274792
invu	862651
lft	1547546
iccc	811641
ndp	1547158
rwlk	1607962
bwen	1120370
botj	1275101
prqr	1612940
fam	1302624
aacg	1420529
jroof	1602380
rnlx	1811115
iroq	1514743
tc	1743340
pnpl	1654672
jmm	838131
agba	1769624
ug	101295
buks	15847
rtto	1646576
bysi	1677940
most	1113513
optt	1378140
bmbn	804563
rskia	84112
mdwt	355379
lnsr	1320350
algs	1799448
aud	1067837
awh	926617
xtnt	1453593
ntip	1065078
stg	1723935
coda	1334325
qyouf	1650287
asnb	1011060
ltrn	1763950
mlss	855683
asm	316888
frd	39092
geg	1831096
buuz	1174891
seed	1321851
culp	723603
grcy	1768910
nm	1333172
hty	1396502
chkr	1524769
fotb	1099668
esoa	1357971
gkin	1509786
gwsn	1592603
mfon	1447380
nick	1000045
asgi	1793855
nwpp	1163389
tmdi	840551
dqws	1652561
bkrrf	1699906
idn	1040896
cuii	1512927
link	828146
pmcb	1157075
mbio	1680048
hwin	1632081
salm	1050606
apm	1734005
nyc	1595527
flux	1083743
shmp	1465470
sera	1534969
apt	884269
neon	87050
snpx	1571934
fury	1514597
eton	1710340
aihs	1711012
tmgi	1434601
derm	1867066
rgs	716643
icch	1681903
reemf	1419806
vapo	1253176
plx	1006281
eng	933738
tren	1740797
harp	1708493
feim	39020
dxr	27367
shwz	1622879
lrfc	1571329
indo	1757840
ectm	1487798
vfl	895574
clev	1414382
asln	1722926
opnt	1385508
ozsc	1679817
cee	860489
aurx	1091596
izea	1495231
janl	1133062
aqst	1398733
gbnh	1735948
lfvn	849146
inab	1740279
otmo	1842498
phas	1169245
nhtc	912061
icgl	1598924
hymc	1718405
pxcly	1888175
mdwd	1593984
aese	1708341
ltcn	1732406
amti	1801777
wwr	839470
pfie	1289636
ocel	1557376
cfoo	1310630
nbo	1178841
admp	887247
rsss	1386301
fgnv	1687919
bdl	12040
sieb	65596
btcy	1630113
uwhr	898171
vmar	1813783
aerc	1872356
kequ	55529
kirk	1056285
rekr	1697851
sntw	1619096
tyme	1537917
eea	791718
grtx	1563577
evfm	1618835
dswl	946936
rgt	1514490
fmy	1319183
tpcs	1328792
wter	1532390
jspr	1788028
notr	1784440
cfms	1305773
bttr	1471727
fgb	1392994
mdna	1807983
atnx	1300699
oesx	1409375
wstrf	1621906
eldn	1404281
avco	1630212
agrx	1261249
akom	1590496
iccm	1584371
trbmf	1406944
onct	1260990
bttx	1832415
grow	754811
wkey	1738699
biof	1549145
iqst	1527702
qnto	1391933
cdtx	1610618
isr	728387
alid	1575295
eman	1046995
gbox	1419275
hvbc	1594555
ior	949961
evax	1828253
omqs	278165
sls	1390478
ainc	1604738
voni	1693690
ktcc	719733
bmra	73290
tcbs	1849466
fpay	1397047
wmllf	1077640
orpb	1216128
cmhf	1792597
heol	1371451
agmh	1705402
mndo	1119083
edi	1551040
cety	1329606
nrxp	1719406
migi	1218683
ganx	1819411
ways	860131
hfbk	1484737
rfl	1713863
drtt	1340476
mtc	1742518
lxeh	1814067
lylt	1870997
kblb	1413119
huiz	1778982
zyne	1621443
hepa	1583771
lotz	1759008
sanw	1477246
pnbk	1098146
snt	896494
pltyf	1433309
stbi	1166362
nmrd	1602078
syrs	1556263
og	1829949
cpsh	814676
coe	1659494
nxn	885731
live	1045742
fco	876717
amam	1836056
jt	1713923
orgs	1460602
wlms	1136294
mtem	1183765
crvs	1626971
novn	1467154
tact	1017303
krkr	1779476
bdsx	1439725
uamy	101538
ppbn	1031233
airt	353184
pyn	1181505
cnnb	1787005
plse	1625101
labp	1785345
ssvff	1397616
lizi	1783407
zdge	1667313
rgf	1871149
ygmz	1782037
litm	1769697
tlltf	1761510
nvve	1836875
vlcn	1829794
pola	1622345
cyad	1637890
cney	1780785
awre	1015739
airrf	1476573
tara	1359931
feo	1363438
aih	1757143
pubc	1575858
frfr	1622408
qli	1779578
achv	949858
nbio	1622057
fsrl	1172102
evgn	1574565
apsi	1553264
nltx	1404644
frsx	1691221
pw	1532619
aitx	1498148
tectp	1766526
alim	1267602
hjgp	1421819
hook	1760542
uavs	8504
ftek	846913
elvt	1651094
bkti	2186
rzlt	1509261
btn	946454
usio	1088034
aumn	1011509
rdhl	1553846
ytfd	1311673
mmmb	1520358
mlyf	1801762
pgtk	1553404
psix	1137091
kanp	1230058
cgtx	1455365
mbot	883975
husa	1156041
sypr	864240
sprb	1683553
egf	1336050
prso	890394
dynr	1111741
msvb	1734875
pcsa	1533743
mxc	66418
wksp	1096275
mbrx	1659617
tess	927355
vev	1834975
jgldf	1686000
ntwk	1039280
shpw	1784851
edry	1731388
rlbd	1084133
dogz	1707303
anav	1491829
boxl	1624512
cvvuf	1023109
spi	1210618
cycn	1755237
acxp	1736243
ocup	1228627
dwsn	799165
lismf	1643715
clir	1434524
hzn	1637655
aamc	1555074
gnrv	1418489
irix	1006045
soho	1301236
genh	1527102
vkin	1102432
verb	1566610
rsrv	83350
aplt	1697532
sntg	1810467
tisi	318833
chci	1299969
rmcf	1616262
sidu	1879726
id	1698113
mhpc	1277998
tphs	724742
hmbl	1119190
any	1591956
dmac	1401040
entx	1638097
vtgn	1411685
viao	1769116
mrkr	1094038
ppbt	1614744
rand	81955
gure	885462
zest	1437491
ltbr	1084554
vero	1409269
mpad	65759
avro	1681087
cmmb	1534248
sctc	1577445
uihc	1401521
gtec	1735041
snmp	1362705
ivc	742112
iclk	1697818
cpix	1087294
inlb	1500123
pmd	806517
tcon	1394319
cspi	356037
xin	1398453
eltp	1053369
pbbk	1849670
itrg	1722387
mark	1368365
ntrb	1676047
scnd	87802
ifbd	1815566
dysl	30831
esp	33533
sabs	1833214
cdbmf	1545634
zgsi	1574186
omex	798528
cyca	1383088
sopa	1817511
huge	1771885
grf	850027
glyc	1253689
sien	1551693
cif	833021
catg	1470129
acst	1444192
ecor	1560258
mxe	863900
nvos	1138978
cocp	1412486
tayd	96536
xpl	917225
blms	1138608
pt	1716338
lpth	889971
mtex	1056358
bmix	1540684
gtim	825324
sskn	1051514
nmtc	1500198
angn	1601485
kins	33992
rcor	1799850
mfv	856128
dtss	1631282
ftft	1066923
wtt	878828
bcda	925741
loncf	1472619
opfi	1818502
utrs	1452965
rgbp	1589150
nymx	1018735
mfbp	1094738
xfor	1501697
trvn	1429560
abio	907654
ugro	1706524
basa	1448705
inti	1042418
cgrn	1009759
fkwl	722572
afib	1522860
ogen	1174940
mdvl	1402479
aptx	1674365
cthr	1015155
bbgi	1099160
actx	1096950
idai	1718939
rnwk	1046327
strn	1872525
qnrx	1671502
kavl	1762239
mcld	1756499
cvv	766792
rvph	1742927
peye	867840
jetmf	1846084
qsep	1103795
piac	845385
clps	1724542
zivo	1101026
aktx	1541157
acrx	1427925
lode	1120970
glbs	1499780
corr	1347652
gjcu	1781726
baos	1811216
cnet	1376321
dlpn	1282224
ppsi	1449792
cntg	1757097
mtr	313364
adad	1593204
cyn	1874097
sgrp	1004989
aim	946644
dwnx	927719
xtgrf	1288770
imcc	1792030
phil	704172
wvvi	838875
wbba	1787412
spro	1701108
egmcf	1199392
imth	1331612
ocg	1776067
bimt	1678848
nukk	1592782
ffhl	1381074
inis	1038277
chek	1610590
cxxif	831609
vjet	1582581
aau	1015647
frln	1810031
vine	1880343
stcb	1539850
vinc	1796129
occ	1000230
ayro	1086745
nsys	722313
fwbi	1604191
msbc	1169483
prka	1297937
imhc	1349706
fngr	1602409
wtrh	1653247
gebrf	1570843
miti	802257
sgma	915358
oncr	1671818
mcvt	1425355
bsgm	1530766
icd	1537028
pgezf	1700157
brn	10048
sngx	812796
snwv	1417663
wbqnl	1785494
veee	1855509
tgen	1537435
beob	1480170
plag	1117057
slgg	1621672
uwhgf	1763543
elev	1783032
hsti	1365388
dcth	872912
dtea	1627606
abvc	1173313
nxtp	1372183
rlby	34285
prsi	79661
hlrtf	1418149
dgly	1342958
svt	89140
nsfdf	1009922
mrm	1819704
glbz	890066
nom	899782
gbny	1823365
daio	351998
dpro	1786286
uuraf	1495651
ikt	1750149
obtx	1730869
bxrx	1780097
pati	1616741
aatc	943034
ubx	1463361
hyre	1713832
igc	1326205
sdig	1856028
atnf	1690080
isun	1634447
umewf	1114936
ekso	1549084
slnh	64463
cpop	1807389
petv	1512922
mrin	1389002
bchg	1732409
fflo	1543652
bpth	1133818
yuanf	1417370
irl	858707
daln	1413898
exdw	1634293
ebet	1829966
usau	27093
cqcq	1569083
eqs	878932
bblr	1873722
mf	1851682
oakv	1865429
hsmd	1418115
otlc	908259
zcmd	1785566
ktta	1841330
sobr	1425627
vvos	1716166
mxrof	1116548
agle	1636282
snrg	1652539
btcm	1517496
crbp	1595097
tlcc	1590695
alrt	1087022
scyyf	1408146
sonx	1407973
ughb	1552743
kdozf	1318482
sbet	1025561
spnd	867038
eryp	1624422
dss	771999
ohcs	1892025
gene	1166272
ards	1614067
edxc	1109486
visl	1565228
modd	1074871
alrn	1420565
agri	1826397
poai	1446159
cvr	19871
sgbx	1023994
athe	1131343
ethof	1482753
srsg	1434737
siox	1636050
caba	1759138
ipw	1830072
jupw	1760903
nubc	1281189
fwp	1604924
aey	874292
ncra	1756180
trib	888721
swgi	1626696
rgrx	707511
cuba	880406
zkin	1687451
ontx	1130598
inrd	719494
bsqr	1054721
ncty	1296774
vcnx	1205922
educ	31667
qtt	1733298
idra	861838
chnr	793628
bsfc	1730773
bkucf	1463978
tsoi	1419051
rshn	1087329
amrr	1488638
pqeff	1561180
eftr	1828522
pypd	1611842
edsa	1540159
hlyk	1680139
meec	728385
prpo	1043961
mhhc	1826135
fsi	1069394
fhs	1786182
cdak	1659352
tlis	1584751
gray	1534133
glgi	1088413
oncs	1444307
seco	1633441
sodi	91668
nnvc	1379006
stly	797465
rnxt	1574094
tait	942126
crmz	315958
astc	1001907
nisn	1603993
enzn	727510
vqs	1777765
ezfl	1817004
hgld	1551206
airi	1009891
eltk	1024672
hall	819913
nrsn	1875091
nuro	1289850
hyex	1630176
vegyf	1508527
dghi	1854368
ampe	1411906
mdjh	1741534
age	1708599
gvp	944480
pvct	315545
tlrs	1288750
kala	1479419
whlm	1013706
ctgl	1498067
creg	721693
sgtx	1821323
iht	82473
indp	1857044
ovtz	1107280
oogi	1160798
xelb	1083220
moxc	1864055
auvi	1811109
vplm	1410738
lowlf	1838128
argc	1698702
rave	718332
arav	1513818
ckx	352955
sdpi	1600422
sfeg	851726
clvr	1819615
acer	1069308
hmmr	1539680
juvaf	1778651
opxs	1397016
qtnt	1596946
mrai	1844392
apwc	1026980
flylf	1403749
wave	1846715
nile	896493
unrv	1451512
mtcr	1634379
igxt	1098880
lsf	1650696
nulgf	1502758
brst	764897
dxyn	29332
vvpr	1681348
mnpr	1645469
bhv	1169034
sqft	1080657
femy	1339005
pure	1006028
kgnr	1593773
onvo	1497253
rcrt	1462223
fbrx	1419041
dhcc	844887
pcst	1406434
lci	57725
lyl	1682241
vaso	839087
kzia	1075880
mdia	1784254
bdco	793306
vynt	1349929
idw	1463833
wyy	1034760
eqos	1790515
fust	1636051
elox	1035354
ewgff	1518561
ttoo	1492674
imbi	870826
ampg	1518461
sif	90168
vgfc	1878835
auud	1554818
tetaa	1444874
gnrs	1790665
sckt	944075
ophc	1288855
altd	1664127
carv	1016178
ayla	1797336
aei	1750106
atif	1755058
tkat	1491487
apre	1781983
cskl	1384437
jctcf	885307
hltt	1307624
evol	1052054
gfasy	1389207
acfn	880984
otic	1493566
alf	1833908
insd	1697587
else	351789
kiq	1161814
kuke	1809158
trka	1021096
ltum	1415332
canf	1536196
axref	1175596
crwe	1103833
lvrlf	1168981
apdn	744452
apvo	1671584
jsda	1083522
mdxl	1601280
tivc	1787740
nnup	888981
crcuf	868822
hqda	1304730
beat	1779372
bvxv	1611747
agtc	1273636
mito	1696396
ymtx	1445283
lnby	1672572
mappf	1493130
kbnt	1729750
nbse	1173281
icnb	1350073
hcti	1839285
ispc	1558569
nexi	1538210
rstn	1760233
cntx	1842952
bon	1816815
tomdf	1645260
bjdx	1704287
ensv	319458
btx	748592
ipix	1355250
sprs	747540
qmci	1101433
inqd	1572565
nvsgf	1415758
vicp	1468639
plxp	1497504
winsf	1640251
lexx	1348362
tbtc	1090396
esbs	1358254
sxtc	1723980
pvnc	1134982
fbpi	1074543
wnw	1787803
pfin	75340
tlgn	1411165
bfri	1858685
tuem	878726
invo	1417926
seac	1019671
pbsv	1304161
rkda	1469443
odii	1781405
cohn	1270436
anldf	1519469
dats	1648960
rdgl	1449349
inpx	1529113
omws	1676852
silo	1514183
opgn	1293818
payd	1017655
trt	732026
blin	1378590
lcgmf	1339688
cnfr	1502292
trux	1578019
rwbyf	1744345
palt	1355839
cann	1477009
reed	1140215
ahi	1815436
xrtx	1729214
hcdi	1784567
mjne	1456857
dtgi	1014052
wavd	803578
evk	943184
iptk	1067873
scia	830616
hannf	1123267
mmtif	1085921
eeiq	1781397
unex	1700844
rcon	1442620
tlss	1463208
rxmd	1402945
bhll	1407583
tpst	1544227
yten	1121702
sfrx	1106213
hlbz	1788841
mitq	1770236
whlr	1527541
bnso	846546
cyth	922247
marps	62362
avct	1704760
tsri	98338
ltry	1673481
chuc	1134765
utme	1789299
ccbc	1365794
gbcs	727346
wbev	1782627
ante	1413745
icct	1408057
avmr	1676163
ycbd	1644903
cycc	1130166
lmfa	1640384
ams	744825
cing	1862150
eqtrf	1635729
lhil	1723187
hfus	1482554
dbs	1383054
gigm	1105101
unxp	1751707
olb	1314196
tanh	1588084
btcs	1436229
fdlb	910492
jsbl	1745931
denr	1435387
oxbr	1584831
psybf	1833371
vrpx	1708331
vivc	1703073
dtst	1419951
drug	1827401
vrvr	1536089
bioc	1044378
inbp	1016504
ndra	1681682
clcs	1569340
rmsl	1412126
gxsff	1299865
nbrv	1641640
phge	1739174
vyne	1566044
sfet	1725332
ucl	1775898
cubt	1400271
cwbr	1522602
pjxrf	1625901
useg	101594
sols	1519177
duot	1396536
rnaz	1829635
pik	1861522
nsav	1432176
btbd	1718224
tomz	314227
curr	1643301
emma	822370
edtk	1782309
aemd	882291
jva	1007019
odyy	1626644
wlyw	1555214
mdgs	1618500
snpw	1343465
pzg	1629210
rzzn	1659559
cemi	1092662
bmnm	1275477
gipr	1651721
fles	1438901
clrd	895665
vblt	1603207
bicx	1443863
nspr	1433607
tevnf	1853718
smit	922612
rayt	1539778
ford	38264
gyro	1589061
prkr	914139
frtm	1589361
lmnl	1351172
reto	1687277
uslg	1536394
ltes	1586495
ssnt	1236275
excl	1786119
clro	840715
mymx	927761
wdlf	1281984
budz	1393772
diah	1692981
fgf	1591890
minm	1467761
obsv	1685316
upc	1809616
sonn	1106838
bimi	1213660
nrrwf	1680637
imte	1668438
ipsi	1591913
bblg	1419554
ptpi	1815903
msn	32621
sofo	1029744
fmfg	1698022
betrf	1464165
ausi	826253
imrn	1660046
qwtr	1487091
yayo	1691077
hgen	1293310
cyan	768408
naov	1326706
bkyi	1019034
exn	1263011
urlof	1115672
scgy	1276531
immx	1873835
rcg	919567
gfgsf	1684888
srre	1083490
elys	1080319
iinn	1837493
dlti	1420368
ipdn	1546296
blyq	1591565
ybgj	895464
vide	758743
ttnp	910267
ilhmf	1522142
lxxgf	1450416
qumu	892482
eawd	1563298
otrk	1136174
cryo	1468679
nuze	1527613
muln	1499961
mdrr	1654595
lgiq	1335112
skky	1546853
agfy	1800637
ngtf	1593001
axas	867665
xeri	1481504
ptco	1609258
nlsp	1783036
leds	1333822
isig	875355
kool	1855255
aspu	1487198
neph	1196298
bosc	1005516
logc	1664106
wafu	1716770
tgcb	1815632
blph	1600132
skye	1516551
tllyf	1689382
awx	1061069
myo	1369290
ocln	1419793
efsh	1599407
wolv	1424404
bwmg	1166708
jzxn	1816172
adru	1169717
nnax	1132509
elvg	1741489
inlx	1081745
cleu	1775085
blbx	1567900
liqt	1307579
enmi	1300781
qlgn	1460702
glmd	1595353
tssi	1320760
gultu	1565146
ccnc	1641398
stal	1614556
vrme	1104038
newh	1371128
axim	1514946
avcrf	1355736
atao	1570937
pbio	830656
uncy	1766140
blcm	1358403
nwcn	934796
vyey	700764
crcw	1688126
taop	1552670
livc	1108630
fzmd	319016
sonm	1178697
ptix	1022899
ionm	1798270
dynt	720875
sql	1605888
garwf	1403870
elio	1531266
brtx	1505497
sint	1269026
usdr	1638911
pnxlf	1689448
smref	1480313
gtmay	1163560
slrx	1615219
suic	1394108
jagx	1585608
reli	1812727
spup	1563227
tops	1296484
qhyg	1423723
mind	926423
wint	946486
sowg	1490161
cscw	1747661
pktx	1128189
cvat	1376793
pbmlf	1319150
pfho	1138476
iaalf	1450894
cdelf	1611831
sipn	1128252
adil	1513525
blnc	1632121
spcb	1291855
allr	1860657
bktpf	1677522
lbcmf	1580110
ithuf	1643154
nlbs	1563463
gril	1701756
phio	1533040
nris	1603793
clxt	1705843
artw	7623
mmnd	1088638
msgm	1821175
rviv	1718500
intv	1520118
cgolf	1759352
dsgt	1413909
irns	723269
bpts	1768946
hoth	1711786
nxgl	1468929
lixt	1335105
trxa	1437750
atms	1062128
xbio	1534525
svvc	1495584
bmmj	1715611
idxg	1054102
hiho	1026785
imh	1000298
cuen	1424657
pstv	1095981
game	1714562
meds	1382574
ambo	1494558
bccef	1807387
rwrdp	1652350
csjt	1451797
rdga	812152
wisa	1682149
snoa	1367083
uuu	102109
tcnt	1602143
avni	1113313
coms	1178727
mmtrs	66496
mobq	1084267
zzll	1365357
dusyf	1551887
lrdc	1442492
bbrw	1641751
byoc	1386049
clsh	1522222
unam	100716
cfrx	1478069
ivfh	312257
lmrmf	1588837
tgiff	1706145
lctc	1493137
govx	832489
crtg	1375195
dxf	1499494
mtp	1643918
lvvv	1421289
stmh	1697834
ufab	1617669
crex	1356093
mcujf	1133519
svfd	1789192
gblx	1165320
mlct	1438943
hpto	1021435
navb	810509
falc	922521
ensc	1716947
nmex	1415744
ryes	1424864
vive	879682
dfco	725394
cphi	1106644
milc	1358656
btb	1543268
unql	1281845
vtna	1280396
sisi	1300734
senr	1576197
exnn	1726744
mtpp	1658521
sscr	1473287
cfic	1448324
nxmr	1088005
ssy	96793
shvlf	1301713
phcf	1729089
tmbr	1504167
ncpl	1414767
gbr	105744
xnda	1624985
mfh	1527762
inkw	1585380
fulo	1092570
gmgt	1816906
zdpy	1279620
worx	1674227
relv	768710
amst	1807166
warm	1399352
luvu	1374567
zvo	1305323
nuwe	1506492
ktra	1498382
prfx	1801834
tofb	730349
acan	1508348
xtlb	1023549
micr	819689
clwt	1026662
rbcn	1410172
pyyx	939930
achn	1672571
sigy	1642159
mkgp	1825270
ezgo	1806904
gmbl	1451448
oblg	746210
gthp	924515
svbl	1031093
cnnc	1410187
sing	1443611
cenbf	1653821
cpmv	836564
jupgf	1684688
bmxi	1122993
gbs	1725430
tetof	1783432
qurt	1549631
rscf	1103090
rkfl	823546
veii	1417664
slgd	88000
tnlx	99106
east	1534708
icco	1103310
cnsp	1729427
clri	1362516
alxef	1603519
enrt	1346022
trck	1045942
ahpi	874710
ejh	1769768
rsls	1427570
jan	862861
fxby	1068897
cosm	1474167
gryn	1443388
nby	1389545
agnpf	1642178
fjhl	850971
ilal	1657214
mgti	1001601
fdit	1593184
ryttf	1706588
tenx	34956
htgm	1169987
sode	1726286
keyr	1832161
dpdw	1110607
moho	1763197
elre	1438461
bylg	1668082
smgi	1426506
gspe	1341726
tppm	1692068
bopo	1510832
grusf	1463000
arao	1083922
ncre	1104462
vper	1133192
towtf	1579026
owpc	1622244
mhtx	1099132
wgmcf	1680962
aytu	1385818
tarsf	1409036
fnhc	1069996
strr	707388
evgdf	1518353
hhse	1069680
rmesf	1358654
yvr	884247
gmpw	1064722
nahd	1485029
vprb	1376231
aiad	743758
dsny	1099369
rhe	1004724
babb	1123596
gfai	1804469
brsf	1662382
tmbxf	1072772
gfoo	1510518
nphc	1119643
ecia	930775
krbp	1792581
ewlu	1517498
ntox	844538
mmmm	66600
cyto	1601936
mysz	1211805
tpia	1763329
qian	1499684
fcic	730669
stab	1318641
pmedf	1868079
ucle	1543623
alds	1755101
cetx	1435064
mnrlf	1555746
henc	1324736
happ	1751876
crvw	1377149
atxi	1644963
nvnt	1282980
gpft	1205181
cvsi	1510964
szlsf	1454263
hbuv	1639068
gegp	1081188
snax	1691936
grom	1662574
adxn	1574232
nugn	1593549
snes	1680378
epgg	1501862
hbis	1489588
drfs	1857910
capc	814926
shrg	1644488
drma	1853816
leai	1561880
fdmsf	1668987
alda	1062506
gdsi	1011662
fwav	1652958
fmnj	1304077
pctl	1119897
gwti	1572386
kdce	1049011
tikk	96885
crkn	1761696
pbla	1029125
mlrkf	1409680
cmxc	1493712
gtvi	1263364
mslp	1415684
wegyf	1008112
ammx	1776048
nvfy	1473334
ctib	1042187
gntof	1346917
paxh	1350156
oilcf	1922639
xito	1651932
abcp	20639
kspn	795212
aehl	1470683
bittf	1028357
thmg	711034
cytr	799698
rnwr	1467913
ivdn	1190370
cowi	1156784
eqmef	1792554
sits	1468978
rgmp	1716324
tcri	1481443
ammj	945617
kgkg	1802546
crdv	1084551
sgni	1023198
frtg	1602813
mjhi	1789330
efoi	924168
acor	1008848
katx	1474558
hrbk	889608
gmvd	1760764
rcar	1016708
celz	1187953
fgco	1377167
phbi	1435181
kern	1755953
ahnr	1304409
sixwf	1515251
nrom	709005
clts	1165639
csui	1680132
ghsi	1642375
plrtf	1639142
fomc	867028
ogaa	1749849
femff	1066130
updc	836937
tptw	1661039
arat	1566243
fnam	884363
coep	1819663
tdcb	1282847
ttcm	1389067
royl	1694617
vnue	1376804
nihk	1084475
lll	1546383
giga	719274
nstm	912544
cloq	1437517
gpox	1673475
iwsh	1279715
lbuy	1643721
rmhb	1670869
gevi	894556
skas	1128281
vyst	1308027
pgid	1061164
cbdy	1586554
igen	1393540
czni	1381871
vs	1701963
seii	819926
admt	849401
lclp	1604930
tnrg	1524872
bdrex	1745846
arth	1537561
maxd	1353499
whlt	1025771
gftx	1338929
wcui	1516887
dvlp	1736865
ccob	1456802
nuvg	1671132
abcff	1284237
whsi	1443089
nxen	1625288
hgyn	1324759
usrm	1388319
agtx	1603345
ybcn	1284454
numd	1543637
amct	1626556
vism	1082733
vyco	1424768
stqn	847942
op	1869467
bntc	1808898
boxs	1487718
afdg	1486297
qron	1689084
vaucf	1723047
tkoi	1094084
kitl	1608092
bzyr	724445
gtll	932021
gsbx	1651166
igpk	1002771
nspdf	1590875
bwmy	1656501
pgol	1080448
nvdef	1398713
reos	1335288
vynd	1745078
spyr	829325
abnaf	873612
ngld	1605481
ldsn	1737193
sils	90391
ipcif	1474835
amty	945828
ltgj	1445175
fecof	849997
jpotf	1061612
rton	1580262
emax	1008653
ciit	1557798
vino	1559998
atyg	1093636
iinx	1528308
eari	1083468
cpwr	827099
vbhi	1490054
brvo	1444839
rnva	931059
syta	1649009
strg	1803096
imci	884650
kprx	1372514
anpc	1786511
ptnyf	1819074
cdjm	1024095
gmer	1454742
luxff	1871928
vlon	1824293
pvnnf	1627480
flxt	925660
mlmn	1100779
xdsl	825322
rahgf	1611852
zyjt	1442101
asft	1651992
xsnx	1039466
pcyn	812306
isco	1355790
sdon	892832
stcc	1555972
gluc	1420108
ifny	822746
csli	1806015
reno	1590383
crtd	1357671
usbl	764630
fifg	1648903
grlf	915661
hitc	1584693
prtm	1874138
etck	1128353
vmcs	1102942
nmgx	891417
nvgt	1089297
thmo	811212
acmb	1598323
fstj	1525306
mojo	1414953
nugs	1290504
amih	1300524
cmdrf	858043
bspk	1409197
trxo	1287900
aunff	1507713
drem	1518336
cbnt	1421636
oxbc	1473086
dbrm	1164256
blpg	1416697
fdct	1722731
bmcs	1384066
repcf	1205059
ioni	1580490
qprc	824416
dxief	1323838
nimu	720762
andr	6494
nauh	1399855
nugl	1096768
suwn	806592
sysx	1737372
bgavf	1500620
pali	1357459
hsdt	1610853
gcfff	863678
bfyw	1852707
aasp	930245
lvca	1874440
jksm	860543
mcoa	1078799
taug	1142790
avoi	1342936
emed	1715819
srmx	841533
flcx	1609988
atds	1068689
wsco	1473490
amhg	1805024
bqst	1568628
sprc	1611746
riii	1574910
lsmg	1319643
tegy	1383394
grst	792935
gdmk	1725911
sgmd	919175
koan	897078
cbds	1360442
ecgi	1856416
bnkl	1508381
gstc	1502152
srco	318299
cmgo	1346655
grhi	894501
sgbi	1104280
elra	1402371
aftc	1182731
plyn	1612851
rebl	1532158
seav	1763660
lbsr	1172178
boty	1407704
laab	1584480
crkr	1162896
eesh	1138867
dbgi	1668010
umhl	1317839
ircc	1373690
ptzh	1627469
nptx	1615999
usaq	856984
lgyv	1616788
vrscf	1422669
usnu	1089815
uvss	1286768
gbux	1169138
snbh	1358633
ntpy	1689490
evoa	728447
evtn	1043894
dpww	1559172
tmrr	1685237
amsu	1142801
stupf	1883898
hadv	1531477
pssr	225628
tcco	96699
plsh	1552189
prps	1855621
dlyt	1125699
kays	1530746
mmmw	1117228
kalo	1389034
sset	1561686
mghl	1162283
pshg	1481241
elst	752294
vrtc	773318
svsn	1099814
alye	946822
tdnt	1421907
mtlk	1098462
chhe	1309057
qtgi	1074663
jrss	1597892
mntr	1599117
corg	837342
sanp	1499275
purt	1126162
ntrr	1512886
zicx	1465311
mdwk	1295514
rmed	1716621
qzmrf	811522
onci	1300867
mage	1515317
thct	1404935
nhle	1552979
dbmm	1127475
neca	1373853
frzt	1485074
ndtp	1329944
bnvif	1647866
uita	1334740
omtk	1404804
msyn	1693687
cpmd	1081938
srus	1761540
wwsg	1342792
duuo	1635136
rlly	1940251
vbix	797542
itdn	51011
avpmf	1445467
gsac	890725
altx	775057
gtxo	1375793
halb	1458631
nfei	1140586
bonz	1439264
mccx	1535079
imahf	1405663
fern	1880419
kssh	1374881
abmc	896747
puge	1540615
wddd	1961
trlff	1698370
afom	1286459
hdvy	1141788
atrx	737207
lfer	1579010
yewb	1548240
glae	1014111
cwnof	1145898
srgz	1401835
forz	1683131
ssok	1559157
cgsi	1552358
mlfb	1308569
itox	1290658
rasp	1582249
gzic	1286648
bisa	918545
admg	1171008
prle	928953
etst	1538495
gfmh	820771
mspc	1488501
amnl	8328
skvi	1085277
imun	1559356
rakr	1872292
aoxy	352991
nhmd	1409446
clis	1393548
btdg	725929
opvs	1571636
tapm	1647170
kens	55234
smme	1301991
equr	1563536
vfrm	1669400
tmll	1065734
abqq	1605331
ehvvf	1653606
dclt	1321828
bant	1704795
qbio	1596062
hstc	797564
esmc	862668
ygyi	1569329
gthr	1017110
mmex	1440799
wesc	1368275
pveg	1163612
wgei	726037
cynxf	1726317
txhg	1133798
ssof	1935435
digp	1502966
rdar	1384365
rspi	849636
tpii	852447
gahc	1138724
endv	1528172
hwke	1750777
qlis	1871181
jfil	1517389
arrt	1530425
gxxm	1681556
lwlw	723533
tkxhf	1598773
bbls	1368637
uryl	1652842
clcn	1394638
dlcr	1614466
pmpg	1301838
gwhp	1598308
rgin	1412659
ghst	1121795
espi	1346526
afpw	1108046
digaf	1369128
xcrt	1138586
nlab	1066719
gcan	1695473
mssv	1760026
emge	1463459
vizc	1506295
bxxy	1680689
dloc	1407878
hybt	1086303
rivu	1079282
lfap	1510247
owvi	1763657
hhhef	825171
cngt	1304741
fkst	1458023
vsym	1075857
xall	1581220
qtxb	820608
wnbd	1114898
hmla	1409624
fssn	1487931
nafs	1409253
vmhg	895287
ifmk	1681941
fccn	1131903
slch	1077319
mxsg	1355677
iwal	1498372
clow	1619227
brgo	1431074
cppxf	852747
abce	1300938
acur	786947
xrxh	1875746
mdex	1318268
bwvi	1310291
wowi	920990
tamg	1158702
omhi	1403674
ferl	1122742
hsta	1813603
dgwr	1637866
nink	1567388
ptam	1431880
scgx	1277575
cydx	1388994
psww	1587476
potn	1746563
logq	768216
sndd	1353406
cbkcq	883943
snwr	1096759
blis	1703625
saml	1530163
grso	1104265
spgx	1500305
both	894560
spowf	1227282
ansu	1168663
pbaj	745543
dwog	869495
gsfi	1437476
trmnf	1574553
pgai	81157
gzcc	1267612
apld	1144879
begi	1483646
roag	1058330
tqlb	1822529
rbsh	1421204
pred	1382943
mapt	1697935
imii	1416090
xtrm	1440280
togl	1378125
aqui	1863976
ivst	1735012
atcc	1372954
cbgl	1413488
pacv	882800
wtii	1715032
pdnla	731245
nnrx	1451433
awon	1432290
cirx	813716
qdmi	1094032
prcx	1572384
knos	1108248
cxdc	1353970
etni	1449447
tghi	225211
chnc	1411057
qban	1427644
adyx	1054274
mblv	1343460
novc	1025953
cglo	842013
cdix	811222
gnolf	1261002
phot	1161582
kwbt	1159275
ghmp	1464865
auri	1092802
grmc	59860
exeo	1528760
fteg	932265
glbd	1524829
owcp	1431934
bdpt	1575142
cyap	1230524
gnbt	1059784
inre	1528985
snho	1927077
rkos	1339205
gmev	1382112
ppcb	1517681
tvog	1098343
btim	1622231
bmmcf	1212787
gspt	1076262
asck	1492091
adgo	869531
ecat	1864843
pcmc	1141964
bzrd	1765826
myhi	1507181
ahix	747435
amlh	1124197
winr	1708410
cmgr	1389518
conc	790273
chno	1530874
gtch	1471781
fmhs	1811999
pacqf	1485922
pgas	1558465
wtkn	1456453
blxx	1428389
mmnt	1653876
fdbl	1414043
ezrg	1543066
brbl	1399306
teum	1084384
nvgi	1500122
crftf	1832761
wtnw	1713909
ecdd	811868
sitoq	1157817
xspt	1628104
fcti	1714379
scrh	831489
znrg	1568875
emdf	894552
cbdl	1776073
spbv	1338672
obcn	789318
lphm	58411
gipl	1837774
blfe	1929281
msto	1407591
alpe	1023298
ecxj	1823635
sfio	1484703
qnta	1691430
webb	1011901
nwyu	1372184
asdn	1425203
fbcd	1370816
efir	1106848
muss	723733
vtxb	1373467
appb	1607549
gsrx	1381240
masn	1486452
gyst	1510524
retc	1627611
bmxc	1613895
cxcq	1073489
clth	1411036
mngg	1406588
bncm	1384939
ltdh	93205
hlwd	1422768
sdvi	1267919
krfg	774415
crxm	772320
agtk	1040850
acca	1001463
vrus	1430523
yumaq	1672326
pxpp	1660839
xcpl	1331275
seci	925661
cybf	1522787
epwcf	1109504
mniz	1532926
pzoo	1533427
tkls	1429393
mdin	1045707
mnvn	1284452
abilf	1652866
syne	1375063
ahfd	1477472
gtor	1499855
eaph	1071272
brrn	1397795
kegs	884380
dpsm	1556753
wkln	1413659
kgji	1089531
plsi	1493212
couv	1450307
aghc	885074
cshef	908256
biio	1575420
legx	1423579
lvpa	831378
eegi	1043150
aggg	1691906
bssp	1495028
anfif	1552448
lrdg	1569568
gteh	1591157
neik	1082027
plyz	1334589
rcmw	1797956
logx	1603494
frli	1223533
aiko	1405660
stsc	1368761
cnna	1368620
rivx	1702541
yuka	1886958
uneq	1572317
msof	766404
mspr	1802450
msth	1790320
mssa	1882464
mtam	1409175
mtek	1872964
mtmt	1036848
mvco	924095
murf	1896212
mtvc	1885754
mwrk	1515139
mysn	1879293
neov	1748137
ndvnq	1659183
neat	1877332
nfnt	1862327
newyy	1726189
netc	1854458
na	1872302
naas	1712178
nams	1936258
naso	1398172
nbad	1847888
nbevq	1579823
nbxg	1843181
nbld	1725516
mnx	1908233
mob	1898643
mntn	1863719
mnao	1710495
mmv	1874074
mnkpf	1567892
modg	837465
mobv	1931691
mond	1828852
mpti	1902314
mpgr	1582962
mlan	1874848
mlxa	1847408
mlst	1846362
mltx	1821586
mkdty	1758736
mkar	1871810
mhua	1835615
mics	923601
mio	1864290
mgom	1697412
mgld	1005101
nyax	1901279
nxl	1527352
nxmh	1811530
nxpl	1058307
nyx	1679379
oaca	1837226
oacc	1845036
oak-pa	1403528
obe	1334388
obtc	1767057
odac	1849074
ocea	1836612
nutx	1479681
nvac	1859807
nvct	1875558
nwtn	1932737
nvx	1859795
nwac	1873504
omcl	926326
omed	1469988
onfa	1541165
onfo	1825452
ons	1810200
opal	1842279
oplf	1628228
ofst	1875931
odv	1431852
odrs	1610718
olit	1866816
oig	1108967
okyo	1849296
novi	1915654
note	1823466
npac	1844004
npct	1835068
npfc	1445831
npfd	1865389
nowg	1761911
nrsax	1663712
nruc	70502
nrgv	1828536
nubi	1881551
nsts	1881592
ntar	1737270
nogn	1841800
nmai	1861115
nmco	1774342
nlsc	1342219
nlon	1922194
nic	1174850
nichx	1736510
nhwk	1476963
nhic	1852931
nftn	1544400
nftt	1868304
rit	1664780
ritm	1556593
rich	1856071
rhdm	1874985
rgti	1838359
rgg	1878628
resi	1555039
revb	1810560
reun	1865482
rfac	1847607
rebn	1707910
rdxx	1849166
reai	1874113
rene	1889112
rciax	1628040
rcpi	1849840
rcs	916183
rdmr	1657249
rdtx	1843548
rbrxf	1278465
rbnk	1535665
rbt	1862068
rcac	1874218
rccc	1665598
rcda	1849801
qtek	1839412
qnzl	1841593
qqcy	1776641
qree	1295961
ramx	1883138
raya	1825875
qvcd	1254699
radd	1847512
racy	1860484
pwup	1847345
pxmd	1811623
qncx	1662774
qngy	1794621
qind	1393781
qbts	1907982
qhi	1792799
ptwo	1930313
prdex	1568905
prdt	1853232
pre	1876431
pr	1658566
ppya	1894057
pplt	1460235
prlh	1856161
prmi	1748669
prok	1850270
proq	1932777
psgi	1880249
psly	1931373
psny	1884082
psoix	1608016
pse	1913715
przo	1916241
pmac	1870068
pmby	1898798
pmec	1891944
pmfax	1723701
pltm	1690842
plur	1158780
plao	1849737
pnyg	1784058
poet	1437424
pol	1810140
port	1865200
pmn	1374339
pmtm	1718271
pnac	1858180
pgrm	1884434
pgru	1873331
pgff	1902700
pet	1842356
perf	1899830
pepg	1835597
pev	1879848
pfflx	1688554
pfhc	1881487
pgy	1883085
phcc	1861681
pics	1841644
pacl	1890897
paci	1853070
oz	1807046
pall	1459862
owscx	1748680
osa	1934064
osaa	1845619
ounz	1546652
ost	1803407
pcm	908187
pdskx	1756404
pdro	1627554
pdpg	1902930
pegy	22701
paxs	1886878
parg	1495648
para	813828
pblt	1771815
loha	1785880
lnzny	1592651
lnw	750004
llap	1835512
lkva	1853790
lre	1888980
lrhc	1879403
lrttf	1547660
lqly	1787123
lpcn	1535955
love	1701758
ltafx	1496254
lspr	1859007
lsta	320017
lsak	1041514
lsdi	1865127
lendx	1658645
ldsp	1898570
ledg	1872206
lfev	1916741
lfac	1851266
lfta	1851991
lfly	1785592
lftru	1822873
lgcp	1661166
licn	1876766
lifd	1391135
livb	1875257
lipo	1347242
lhtv	1296884
lgmk	1566826
lhca	1852084
lgst	1860871
lvvp	1387061
lvvr	1563756
lxam	1740695
lvdw	1503658
lvac	1855450
ltva	1851195
lucy	1808377
matv	1000623
marx	1892922
maia	1878313
madl	1052354
maaq	1870144
mana	1852023
lyt	1816319
lzgi	1126115
mdv	1645873
mdvp	1671077
megi	1855066
megl	1881472
mgam	1886362
mfic	1278752
mfrm	1883331
meta	1326801
mdl	1370496
mdls	1645108
mdlvy	1933644
mcty	1846752
mbsc	1856589
mcac	1895249
kcrd	1696025
ketr	1866907
kbtc	1790727
karx	1729637
kbsg	1631256
kntk	1692787
knw	1074828
knsw	1885444
kitt	1849820
klc	1873529
kpnt	1826202
kscp	1600983
kwe	1889823
kvsd	1847037
kych	1865701
kth	1283464
ktn	1283337
kusa	1467845
kwik	1884164
lab	1162194
laixy	1742056
lanv	1922097
larax	1753712
latg	1868269
lbbb	1867287
lbgj	1896425
lcfy	1875547
lcla	1854906
ivcp	1845123
ivvd	1832038
ixhl	1873875
izm	1854572
jcse	1905511
jewl	1884072
jfbr	1885408
jengq	1538789
jbk	1284143
ivbt	1629205
ivca	1852889
ivcb	1857410
isign	727634
itaq	1841586
ispo	1820566
jggc	1857518
jia	1798255
jjoc	1861733
jppyy	1616291
juns	1679628
kal	1909152
jz	1852440
jwac	1883799
kacl	1865468
gten	1897941
gtac	1848821
gsun	1826376
gwav	1589149
gwghq	1522690
gvac	1867183
gvbt	1571804
gug	1864208
gttj	1368757
gxlm	1761325
gxxy	1481585
gwygu	1812641
hac	1866411
haia	1848861
higr	1807616
hkd	1809691
hkit	1742341
hils	1861657
hlco	1441082
hllk	1331421
hma	1850529
hmac	1894370
hlst	1944977
hlp	1855557
hln	1900304
hdls	1804506
hcvi	1842937
hcma	1845368
hghh	1665421
hgia	1868110
hesp	1907702
gsrm	1901799
gsgg	1668523
grrr	1903145
grta	1600626
grtr	1905312
grouy	1437710
grov	1841761
grnt	1928446
grna	1822691
grfx	1816723
graz	1734237
goxs	1791405
gosc	1858939
golq	1746278
gpat	1834526
gplb	1392449
gpldf	1300050
gmfi	1866547
gmwx	1781929
gnlx	1231457
gns	1847806
gldm	1618181
gle	1908705
gliv	1852024
glnk	1852025
glns	1375348
glxy	1859392
gltr	1483386
glst	1922331
gls	1805087
hmnty	1915760
hlvx	1888012
hnvr	1828588
hnra	1842556
hpac	1871667
hpco	1892480
hoti	1615647
hour	1874875
hous	1398987
hori	1865533
htcr	1892322
hthc	1714420
htia	1561032
hrdg	1816978
hscs	1468492
hs	1867684
hwgc	1623590
hyiv	1849812
hwni	1413891
hwtr	1144546
hypb	1923673
hzag	1900554
hzen	1748945
huda	1853047
huak	1812673
hubc	1905660
ibg	1924482
iaum	1759124
iaux	1853962
iaci	1892876
icg	1895597
icga	1837177
icgc	1878482
icr-pa	1690012
idr	1030192
idiv	1688487
intm	1835104
intr	1864163
ints	1567264
inki	1753373
iraa	1831874
ipx	1898601
ionr	1896084
ilag	1814963
impp	1876581
igacu	1819496
ie	1879016
ifin	1862935
ihita	1698508
igta	1866838
cnrlx	1690996
cnrox	1737936
cnxa	1674440
cohr	820318
cotrp	1283602
corz	1839341
conxu	1823000
cprdx	1762562
cpaq	1873964
clrs	1801956
clrc	1903392
cmca	1865248
clic	1843983
clin	1883984
cmcz	1673362
cmrf	1498547
cmra	1907685
cngl	1867443
cnaq	1842431
cncl	1130889
cslm	1875493
crxtq	1817944
csca	1731899
crec	1851230
crdo	1807794
crgh	1798458
cptn	1498233
crcl	1876042
crbg	1889539
cwpe	1741220
cxac	1856242
cvua	889348
cvmca	1567925
ctkyy	1734262
ctm	1877939
cvf	1871717
efht	1922858
egac	1851366
egio	1391127
eic	1754836
ehab	1803737
eioax	1751156
elgp	1885493
elmsq	1784168
elep	1865833
elbm	1907184
elc	1348952
eawc	1686515
ecec	1909658
ecbk	1914605
edgs	1614826
edbl	1809750
ecoa	1884431
ecox	1144169
ee	1888447
duet	1890671
dycu	1830569
dxyz	1843974
dzgh	1883389
eaas	1737189
drct	1880613
drs	1833756
drts	1871321
dsgr	703604
dsty	1879560
dspk	1917762
dsro	924396
dtla-p	1575311
dpui	1791929
dpac	1864132
do	949039
dmyy	1915380
dnax	1419995
dnify	1487739
dma	1523289
distu	1818605
diam	1825786
dino	1915657
defy	1886015
desr	1882148
deft	1888274
dcre	1854149
deci	1668069
deca	1913577
dcfc	1862490
dc	1852353
dbv	1354730
dave	1841408
daoo	1870471
frgt	1687542
frge	1827821
frprf	1830162
fp	1900035
frza	1901305
fshp	1850059
fspr	1558536
fropx	1732078
frtx	819050
fte	1856114
ftfi	872203
fsrd	1832351
fthwf	1941771
ftii	1889450
flrz	1787462
frbn	1874495
forl	1936255
fogo	1883631
glasf	1848731
gjh	1286405
gjo	1340909
gjp	1343491
gjs	1356284
gjt	1357660
gif	1844512
ggr	1886190
ghix	1894630
ggaa	1865697
gge	1158420
ggi	1842966
gfld	1842327
gfgd	1876714
genq	1853112
gety	1898496
gdst	1858007
gehi	1708441
geex	1856774
gels	1920092
gemz	1104023
gbtt	1657564
gc	1805586
gdnr	1858912
gct	1857816
gctk	1506983
gbtc	1588489
gbtg	1820872
gbat	1852040
gbbk	1894951
gabc	714395
gamc	1841125
gaq	1852061
fxc	1353612
fxco	1817565
fxf	1353615
fwfw	1492617
fxa	1353614
fxb	1353611
epow	1780731
ep	887396
enjyq	1830180
enov	1420800
entf	1870925
eoco	1843917
embc	1872789
emcg	1869601
elsn	1880047
elv	1156039
enac	1860220
emtx	1254348
emwpf	1532981
emld	1889123
endi	1908984
endpq	1593034
encp	1879373
eu	1500881
eubg	1171326
etcg	1705181
eta	1829237
eti-p	1427437
ethe	1725210
evgr	1900402
evex	1823652
eve	1861121
esh	1918661
esab	1877322
esac	1865506
fbox	1747624
faze	1839360
fcrex	1688897
fdha	1868951
fcci	1807689
ezagoo	1752372
exod	1821534
exro	1459762
exac	1844019
evvl	1759424
fepi	726516
fexd	1852407
feam	1888654
fgnb	1844364
fgi	1864943
fgld	1858258
fgmc	1906133
fhlt	1851182
flct	1912287
fllz	1659207
flfv	1912582
figi	1791325
fhp	1785343
fio	1899194
fip	1899883
filg	1852039
aima	1903464
aidg	1702015
aiu	1722380
agxpf	1482436
ahrn	1856696
aib	1882963
aggx	1085596
agss	1514443
afri	1903870
afiiq	1655075
afhif	1539894
affg	1469038
aftx	1792694
alnaq	1624658
aloha	1862733
alor	1883962
alte	1842428
altt	1843079
alr	1159281
alsa	1865111
alwn	1908211
alvo	1898416
akli	1850266
akan	1888014
alef	1887389
aleh	1806905
allg	1874474
aaidx	1754927
aagr	1884334
aaau	1708646
aacp	1822365
acax	1897245
acab	1893219
acac	1914023
aclx	1786205
aclhf	1871737
accv	1847499
acru	1407573
acrg	773717
acmsy	1850767
acnt	95953
acon	1635077
aeae	1852016
adrt	1864032
aeib	1875655
afar	1901886
angh	1871983
antx	1880438
anvi	1570132
aogo	1881741
apca	1862993
aort	784199
apgc	1840632
apgn	1814140
amlx	1658551
amtd	1769731
ampx	1899287
apoa	1862462
arac	1853864
arag	1895812
aqu	1861063
aqub	1863974
aqpw	1317833
apxi	1868573
apxx	1653710
armt	1534155
armv	1625285
ark	1843093
aren	894871
asfh	1828748
asav	1669368
ascix	1716885
ascb	1876716
asca	1868775
aspx	1846459
aspp	1906195
asst	1920406
asns	1141284
aspi	1921865
atek	1882198
ater	1757715
atat	1853717
atak	1883788
atla	1869089
atryx	1740076
atwo	1844940
aust	1817740
aurv	1849006
az	1866030
axin	835662
axh	1855302
awys	1021917
axac	1898019
btcw	1850391
bsec	1724001
brsh	1913210
brni	1851651
brkh	1871638
btmd	1819253
btnc	1844028
btog	1735556
btvc	1846599
btru	1709385
btry	1857971
bttc	1066764
bwv	1782107
bwaq	1878074
bull	1872195
catl	1846470
cblo	1882781
cccp	1696411
bzwr	1830503
byno	1801417
bxt	1750145
byn	1852633
calq	1843080
cadcx	1678124
chmt	1765048
chrd	1486159
chscp	823277
chsn	1825349
chea	1856948
chgi	1284450
chg	1893311
cite	1848437
cinc	1868734
cjoy	1895159
cjax	1763925
cipi	1108645
city	872202
cdmnf	1875236
cdtg	1793895
cdp	1786485
ceai	1795938
cecl	926865
cedax	1722837
ceac	1916148
cead	1482541
cclfx	1735964
cfsb	1879103
cffi	913341
cffs	1839519
cenn	1707919
cets	1936702
bign	1753391
bigz	1836057
bior	1580063
bios	1856653
bitw	1723788
bksw	1759528
blco	1860742
blac	1840425
bleu	1843370
blte	1889109
bngo	1411690
bmnr	1829311
bmr	1899005
bnrg	1901215
bno	1472494
bnow	1877788
boc	1494582
bocn	1856961
brac	1865120
brcnf	1158399
brcc	1891101
brig	1849349
brd	1847351
bouw	1909142
bpac	1856724
bpypm	1545772
bdrl	1000683
bdry	1610940
bds	1831833
beke	1809587
beep	1918056
belp	1749817
bfh	1101215
bfac	1880441
bgac	1916902
bgfdx	1710523
bgtk	1518720
bhm	1903382
biaf	1712762
bgxx	1886799
bbmpy	1553920
bbbt	1409999
bbop	1814102
bcan	1888151
bbuc	1871130
bcgf	1932629
bar	1690437
banl	1914805
ball	9389
aztgy	1921332
back	1729944
baer	1941536
unoo	1849470
unl	1405513
unss	1434389
ukwi	1856659
ueri	1888577
uga	1396878
ufin	1831583
tytp	1873951
tyde	1892492
udn	1383149
ubxg	1888525
urgp	932352
usax	1874981
upow	1469115
upxi	1775194
upyy	1677897
usl	1405528
ussc	1851893
usqix	1691570
usea	1912847
usci	1479247
vbnk	1690639
vboc	1853920
vaqa	1851169
vani	1266806
vacx	1865249
utxou	1793081
uup	1383151
utaa	1879221
vrax	1885827
vnmt	1605057
vnrx	93314
voxr	1907909
vuso	1855011
vvx	1601548
vwfb	1913838
vzla	1796073
vybe	1803977
vxi	1689258
vssa	1897342
vigl	1827087
vmca	1892747
vflex	1681717
vhna	1868640
vemc	1846764
vcor	1627041
vcrrx	1687898
vedu	1892274
vdnt	1862979
vcmix	1515001
vcxb	1848948
syqh	710782
sym	1837240
szzl	1829322
tacq	1852038
tdrk	1282496
tcoa	1846750
tcrt	1107421
tdai	1904958
tcbp	1872812
tcgi	1879270
tbld	1820378
tbmcu	1934945
tf	1882958
tgaa	1847355
tgl	1905956
tfpm	1829726
tfsa	1577134
tenk	1851484
teslu	1877934
tete	1900679
tekc	1847419
ther	1362703
thmb	1846735
thch	1877333
thrd	1923840
tgr	1863099
tiplx	1551047
tirx	1782941
tklf	1836242
tlgy	1879814
tlf	909724
tmna	1648365
tmin	1613685
tnon	1560293
tmq	1543418
tmtc	1879851
tnt	1558924
tony	1735532
top	1848275
tpet	1898766
tpg	1880661
tpta	1674356
trcc	1843875
trde	1111559
tufu	1863032
turo	1514587
twic	1845865
twcg	1845855
twcm	1849512
twen	1926451
tto	1843556
tsifx	1725295
tsio	1900830
tsiv	1892387
trlc	1550453
trlef	1648636
trirf	1819876
trny	1844300
tsgn	1899770
stss	1737995
stry	1884697
strw	1782430
stwy	1897640
suac	1885461
str	1703785
stew	102426
stkh	1828098
stix	1896049
stet	1858684
svni	1849288
svre	1894693
svv	1883313
swgc	1848669
swvl	1875609
svii	1843477
svmb	1647822
sunf	1873885
supb	1925179
sprj	1552164
spty	1840102
spta	1848452
ssic	1843162
srrix	1581005
srts	1494891
sst	1805833
sssc	1870079
stek	1511820
stbx	1914818
sov	811830
spcm	1880968
soun	1840856
sosh	1861785
speyx	1729895
spfx	1807893
spgc	1934245
spgl	1847618
sond	1819395
somc	703699
snti	1854270
smr	1822966
smx	1940674
smxt	1519472
smst	1585389
smtk	1817760
smsa	1824988
sivr	1450922
simpq	1274032
sire	1575051
skyx	1598981
skyh	1823587
skfg	1794942
skgr	1912461
smge	1675426
slta	1880175
smfl	1851860
sldx	1827620
slna	1909417
sed	1883037
sdwl	1753931
scrm	1893325
scth	1703157
scps	1772028
sdch	1022505
sdec	1734669
scua	1853594
scit	1803487
scem	1851091
sbig	1801602
sbp	1884713
sflm	1086313
sfco	1831523
ses	1819142
sesg	1837824
sesu	1885336
sfwj	1919847
sgly	1422892
sge	1893448
sghc	1878057
sghl	1855885
sgol	1450923
shap	1881462
shel	1306965
shmy	1766267
shua	1886268
shph	1757499
siaf	1488419
sigq	1867955
rjac	1852495
rlty	1866874
rmmz	1870833
rmri	1556179
rmpl	1644771
roar	1890380
rocl	1885998
rotk	1858721
rpgl	1912884
rsmxf	1858994
sanb	1856689
sasi	788611
satl	1874315
satt	1661600
satx	1915403
rytm	1649904
saayy	1062750
sai	1847075
saddp	313151
sagn	1639953
saga	1855351
rto	930157
rtgn	1836295
rtl	1568162
rtva	1817241
rvlp	1739426
rwod	1907223
rtpx	1828109
rvsn	1743905
rvr	1891367
rynl	1283789
yswy	1859836
yjgj	1699709
ygf	1875496
ymat	1875016
yosh	1898604
yota	1907730
zhec	1594114
zghb	1778404
zfox	1823575
zcsh	1720265
zaag	1893124
xptfx	1677615
xwamx	1555085
xwmfx	1525355
xsiax	1124959
xygj	1885840
ybzn	1885514
ycqh	1794276
ycrm	1624517
xgeix	1642563
xilsx	1616037
xpon	1894954
xpdb	1855474
xpasx	1682662
zrfy	1285543
zsanq	1587221
ztek	1904501
ztop	1869901
zlme	1489300
zmeny	1838937
zimv	1876588
zing	1844270
wlgs	1899658
wlds	1887673
west	1806347
weth	1826660
wds	844551
wdtk	1422296
wdi	1819559
wefcx	1586009
weib	1717976
weidy	1734902
weix	1771951
wel	1877557
wcap	1800044
wavs	1868419
wbd	1437107
xesp	1709542
xfci	1379043
xalcx	1729678
xcapx	1467631
wxt	1905448
wytc	1560143
wtma	1866226
wpa	1927596
woda	1862681
wnft	1528188
wnlv	1558740
wnnr	1843714
wly	107140
wrnt	1900564
wrpt	1842138
bmymp	14272
usb-pa	36104
brk-a	1067983
jpm-pc	19617
jpm-pd	19617
lvmhf	824046
bml-ph	70858
bml-pg	70858
bml-pl	70858
bac-pb	70858
bac-pk	70858
babaf	1577552
bac-pl	70858
nonof	353278
bac-pe	70858
bml-pj	70858
rydaf	1306965
toyof	1094517
wfc-pl	72971
wfc-pq	72971
wfc-py	72971
asmlf	937966
wfc-pr	72971
nvsef	1114448
azncf	901832
tbb	732717
c-pj	831001
ptr	1108329
pccyf	1108329
ttfnf	879764
bhplf	811809
hbcyf	1089113
unlyf	217410
stohf	1140625
bpaqf	313807
snynf	1121404
sapgf	1000184
dgeaf	835403
ciljf	1268896
cmxhf	1274152
ry-pt	1000275
jdcmf	1549802
btaff	1303523
wfc-pc	72971
lfc	1268896
snejf	313838
rtppf	863064
cta-pb	30554
ebbnf	895728
budff	1668717
pbr-a	1119639
usb-ph	36104
glaxf	1131399
glcnf	1524684
usb-pp	36104
eadsf	1378580
snpmf	1123658
snp	1123658
gs-pa	886982
pbcrf	1463157
idexf	1438656
ms-pa	895421
ms-pi	895421
goog	1652044
gs-pd	886982
ms-pk	895421
gs-pj	886982
amovf	1129137
mbfjf	67088
ms-pf	895421
ms-pe	895421
gs-pk	886982
sftbf	1065521
duk-pa	1326160
ccz	1166691
stz-b	16918
spg-pj	1063761
amx	1129137
ablzf	1091587
amxvf	1129137
webnf	719245
rbgpf	1420798
sfbqf	1065521
schw-pd	316709
pdrdf	899108
esocf	1096200
rlxxf	929869
bamgf	1001085
ppwlo	75594
pldgp	1045609
bcdrf	891478
aig-pa	5272
met-pe	1099219
eipaf	1002242
smfnf	1022837
tkphf	1395064
hndaf	715153
nggtf	1004315
met-pa	1099219
enba	895728
psa-ph	1393311
lldtf	1160106
ingvf	1039765
psa-pk	1393311
fngd	927971
vodpf	839923
rbspf	844150
bf-b	14693
rsmdf	943819
ifnnf	1107457
stmef	932787
all-pb	899051
bbvxf	842180
bclyf	312069
bbdo	1160330
all-pg	899051
gpdnf	1048515
pukpf	1116578
mzhof	1335730
expgf	1376891
hcmlf	948696
all-ph	899051
baesf	895564
pperf	1461748
crhcf	849395
nokbf	924613
rwnff	939380
fnctf	1038143
arzgf	1379235
cajff	16988
dlr-pk	1297996
cixpf	1535628
dlr-pj	1297996
sojc	92122
gnmsf	1434265
rciaf	733099
ckhuf	1637120
vwsyf	1330306
erixf	717826
lnngf	1447629
tefof	814052
wopef	844551
tscdf	885834
codgf	1012037
rstrf	1618755
amsyf	1243429
amccf	1748790
poahf	1450346
orxcf	1070304
atasf	1437775
lngpf	1554394
pcrff	63271
tckrf	886986
stt-pd	93751
ptaif	1455967
stt-pg	93751
fox	1754301
hig-pg	874766
fitbi	35527
nuan	1002517
annsf	1700171
tsgtf	939383
hei-a	46619
key-pk	91576
fmcct	1026214
tnrsf	1190723
nmk-pc	1004315
fxflf	915191
lbrdb	1611983
lbrdk	1611983
fwonb	1560385
hrnnf	1712356
snmrf	1437771
key-pj	91576
btgof	756620
rylpf	313216
bio-b	12208
hldvf	811250
key-pi	91576
swmaf	1011218
lsxmb	1560385
lsxmk	1560385
cfg-pd	759944
sjrwf	932872
frc-pi	1132979
ryaof	1038683
nistf	1140471
nholf	1669414
nmk-pb	1004315
fwonk	1560385
frfff	915191
npscy	1140471
sgiof	1281721
jronf	1438077
ebr-b	1439124
fwona	1560385
ptpif	1547873
chkif	1041668
trpcf	1269238
frc-ph	1132979
cocsf	910631
ncsyf	1003935
rf-pc	1281761
rf-pb	1281761
bpypp	1545772
prdsf	1527249
maa-pi	912595
acglo	947484
kkoyf	1436794
csgkf	1159510
nly-pf	1043219
atlcy	748954
pnc-pp	713676
snnuf	845982
nly-pg	1043219
bntgf	1533945
jxhgf	1487533
pbnnf	1504122
sasof	314590
rklif	930157
vno-pl	899689
tap-a	24545
rtoky	930157
bouyf	1445475
cukpf	1125259
nrscf	1163653
adttf	1158838
uhid	352915
vno-pm	899689
zhshf	1495686
wso-b	105016
lbtyk	1570585
lbtyb	1570585
jhiuf	1159152
fmcqf	1333141
ni-pb	1111711
nws	1564708
nipnf	72127
tevjf	818686
almmf	1161611
wppgf	806968
asxff	1449003
ach	1161611
dqjcf	1446597
aegof	769218
bolif	1446457
frt-pc	34903
ncmgy	899611
ncmgf	899611
gifof	1438569
z	1617640
lnvgf	932477
uepeo	100826
remyf	1552653
asekf	1450206
vivef	1127055
rza	898174
svysf	930826
rzb	898174
dispf	1671750
glpef	1385535
amh-ph	1562401
hungf	929058
agncn	1423689
gikly	1438569
amh-pg	1562401
uepen	100826
uepem	100826
prymf	1437672
fcncb	798941
aomff	1062066
kim-pm	879101
oscuf	1600520
ath-pa	1527469
agncm	1423689
gs-pc	886982
gpaef	1347557
kim-pl	879101
mdibf	1379481
auoty	1172494
msuxf	1710864
mtlhf	1446418
voya-pb	1535929
psorf	938323
rnr-pf	913144
giflf	1438569
rkunf	1294591
gfiof	1172724
slg-pi	1040971
miuff	1447113
dcp-pc	1338065
cmsa	811156
acrff	1049183
oukpf	1436786
oak-pb	1403528
asrmf	1123452
atgff	1695519
dsecf	1481045
jsm	1593538
sptjf	908732
cig-c	1157557
jscpf	1449005
bkgff	1447137
cxmsf	1076378
vnorp	899689
eenef	1455626
jsgcf	1442656
aulgf	1067428
ccoef	1551467
unma	5513
pcg-pa	1004980
mndt	1370880
qbcaf	1156831
bpopo	763901
phtcf	78150
pcg-pb	1004980
gab-pg	794685
padef	1489079
bhfal	1685040
lyscf	1444327
bnmdf	1667075
snv-pd	18349
epr-pg	1045450
evotf	1412558
qbcrf	1156831
pcg-pd	1004980
pcg-ph	1004980
adrzf	1436467
mituf	1446420
pcg-pe	1004980
mcfe	1783317
fnmas	310522
iivi	820318
pcg-pi	1004980
psb-py	866368
psb-px	866368
fnmat	310522
fnmaj	310522
cwen-a	1567683
axs-pe	1214816
ubsff	1446702
mgddf	896076
fnmah	310522
gsabf	887153
dtw	936340
mxchf	1435969
fnmfn	310522
bhfap	1685040
nycb-pa	910073
sr-pa	1126956
rexr-pb	1571283
ely	837465
fnmam	310522
eqc-pd	803649
dfilf	871459
mant	892537
al-pa	1487712
sf-pb	720672
ua	1336917
vlpnf	1378984
src-pa	1308606
shi	908732
slmbp	1032033
slfpf	1370418
fnman	310522
fnmai	310522
tkayf	1792627
qrteb	1355096
fnmag	310522
grpff	912892
wbs-pf	801337
bcucf	1581285
fnmfm	310522
fnmak	310522
fnmal	310522
wtfcm	1015328
esgro	1363829
isbc	1594012
pbi-pb	78814
fnb-pe	37808
esgrp	1363829
glpgf	1421876
dlkgf	1361371
asmvf	1447945
fnmfo	310522
fnmao	310522
egfef	1437178
abczf	1492074
vlypo	714310
chfff	1601994
gef-b	43920
pegrf	1455633
dfryf	1546354
peb-pe	1474098
asb-pe	7789
two-pc	1465740
regi	1463258
vlypp	714310
gaerf	1378239
hovnp	357294
ltsl	1029730
peb-pf	1474098
iijif	1090633
cim-pa	1409493
cim-pb	1409493
ns-pa	1110805
two-pa	1465740
two-pb	1465740
rlj-pa	1511337
rushb	1012019
cim-pd	1409493
cim-pc	1409493
ns-pb	1110805
wrb-pe	11544
mog-b	67887
ailli	18654
sfrgf	1571549
irdef	1530256
mfa-pb	1055160
trolb	98677
fmckj	1026214
nsa-pa	1618563
idbhf	1044896
fmckl	1026214
socgp	92108
trtn-pa	1660734
thnpf	1792045
lgf-b	929351
hgmcf	1023514
albkf	902102
ns-pc	1110805
sitc-pa	894315
fmcko	1026214
fmckm	1026214
srg-pa	1628063
azihf	1533475
lnznf	1592651
fisk	1553079
aillo	18654
lxp-pc	910108
fmckn	1026214
cdev	1658566
fmccs	1026214
fmcki	1026214
fmccj	1026214
fmcci	1026214
mcfuf	1359711
fmccl	1026214
fmccn	1026214
fmcck	1026214
fmcco	1026214
grouf	1437710
centa	887733
fmcch	1026214
ogcp	1553079
fmccg	1026214
fmccm	1026214
nss	1110805
plltl	1728205
fmccp	1026214
codql	1770561
swghf	1816901
fmckp	1026214
fregp	1026214
apely	900278
ako-b	925261
gngyf	1012139
gdv-ph	1260729
tcmff	932470
tgsnf	1437731
frejn	1026214
poly	914025
trfpf	1829726
gtn-a	43196
vtscf	1880866
atgsf	1437541
ivr-pb	1437071
slcjf	1523541
ivr-pc	1437071
batrb	1560385
sauhf	1451279
bfs-pd	907254
lilak	1712184
lilab	1712184
batra	1560385
gab-ph	794685
crnzf	909531
batrk	1560385
pmt-pb	1464423
ltghf	1569963
gnl-pa	1526113
pmt-pa	1464423
ubeof	1446626
apelf	900278
glog-pa	1534126
miccf	912958
mhla	1412100
hcxy	1280784
oxsql	1259429
bpirf	1437441
ahl-pd	1267395
codi-pb	1345126
codi-pa	1345126
glop-pa	1598655
hl-pb	719413
stsff	1585389
ahh-pa	1569187
dniyy	1487739
eai	7323
ihrtb	1400891
nymtn	1273685
glop-pb	1598655
glop-pc	1598655
kbsr	1482430
brchf	1887388
tanni	1378453
agm-a	845877
tannz	1378453
tannl	1378453
iipr-pa	1677576
sboef	1578955
gsky	1712923
conx	1823000
cmctp	908311
ngl-pc	1504461
bcomf	1402606
inn-pe	1497645
celjf	1385145
gam-pb	40417
cwgrp	1466538
ngl-pb	1504461
sapmf	1062750
saflf	1538028
clvlf	1290877
agm-pc	845877
oibrq	1160846
etac	1814728
star-pg	1095651
fpac-un	1829426
star-pd	1095651
star-pi	1095651
nvnxf	1859795
hltc	1561032
agm-pd	845877
ubp-ph	1029800
acbi	1461755
ht-pe	1063344
ht-pc	1063344
cmre-pb	1503584
ht-pd	1063344
aic	1209028
enmpf	1630511
vslcx	853180
vslyx	853180
uba	1029800
cubi-pe	1488813
glp-pa	1323468
cubi-pf	1488813
cmre-pc	1503584
rtlr	1748773
ggn-pb	1313510
kelyb	55135
cmre-pd	1503584
cmre-pe	1503584
ggt-pe	921671
mpsyf	1340243
ivevf	1756594
bdsi	1103021
ellkf	1437194
hmlp-pa	1603016
umh-pd	752642
adgi	1832038
ctt	1341141
drdgf	1023512
artnb	863110
hvt-a	216085
oxlcm	1495222
arcxf	1243429
brcpf	1499849
ncv-pa	1214935
eccx	1604174
taco	1585583
seneb	88948
gss	903571
ltrpb	1606745
dgicb	800457
mbinp	1629019
enrff	1904856
bvhbb	315858
cio-pa	1593222
oxlco	1495222
gmre-pa	1533615
kor	1507964
muds	1820727
landm	1495240
mitt-pa	1514281
bh	1726173
igac	1819496
splp-pa	1452857
gut-pc	1080720
apyp	1568969
corr-pa	1347652
mitt-pb	1514281
bhr-pd	1574085
vrrmw	1682745
meobf	1345099
thptf	1556898
hawln	46207
ncz-pa	1227857
dhbcu	1838176
warr	1820209
aht-pd	1232582
cnthp	23426
belfb	729580
brg-pd	1442626
cnlpl	23426
aht-ph	1232582
ncacu	1849475
brg-pc	1442626
dsx-pb	1318885
bhr-pb	1574085
gne-pa	1528356
nev	1469392
tnp-pe	1166663
aht-pf	1232582
tnp-pd	1166663
tnp-pf	1166663
aht-pg	1232582
aht-pi	1232582
cntho	23426
amsiy	1243429
hawen	46207
crd-b	25475
hbp	1093082
vwtr	830122
hmtv	1567345
usak	883945
vtiqu	1823884
iqmdu	1850051
cnlho	23426
hovvb	357294
chmi-pb	1571776
cnltp	23426
bbi	819050
uonek	1041657
atrob	8063
ds-pb	1175483
lftr	1822873
mudsu	1820727
cnlpm	23426
chmi-pa	1571776
cnpwm	23426
fgprb	922358
ds-pd	1175483
cnltn	23426
ds-pc	1175483
srnw	1321517
slncf	1479615
vsogf	1762506
lhc-un	1824153
iphyf	1598599
rvrf	1641601
acaq-un	1869141
jco	1686142
itepf	1579597
encuf	1500881
eno	71508
bkep	1392091
plym-pa	1515816
arbkf	1841675
privx	1557265
gwin	1515114
prlvx	1557265
sb-pc	1434754
asepf	1882607
sb-pd	1434754
aixn	835662
acrdf	1762359
eyes	1266806
qtrhf	1518419
smm	1534293
moyff	1850453
dlng-pa	1578453
bbxib	1814974
rlftf	1854078
gsv	1321847
oibzq	1160846
adre	1169717
ecf-pa	793040
claq	1849820
ezoo	1752372
bcv-pa	9521
fatbb	1705012
lmpx	1731727
nm-ph	1333172
sohob	1301236
rtnxf	1811115
mtbcp	1582982
fbiop	1429260
nm-pg	1333172
sohoo	1301236
pkkff	1558924
sohon	1301236
cdr-pc	761648
pkdc	76321
glu-pb	1282957
rdib	716634
khdhf	16859
knwn	1074828
cdr-pb	761648
voxcf	1907909
gsl-pb	1430725
gpl	1300050
cssep	1679063
cxkj	1502557
rsf	1644771
exrof	1459762
dlng-pb	1578453
aljj	1438731
pei-pb	77281
pei-pd	77281
pei-pc	77281
mkd	1758736
ihta	1698508
ofstf	1875931
uboh	1087456
axu	1364128
scxlb	93676
innd	1609139
argq	1533106
cheaf	1030475
fflwf	1811942
taoff	1898601
kfsyf	1072627
gntlf	1166272
deftf	1888274
wcrs	1363958
mh-pa	1412100
mh-pd	1412100
brcn	1158399
nexcf	1737270
mh-pc	1412100
emms	783005
tlfa	909724
pxclf	1888175
bctf	1668340
assf	1719435
avlnf	1362898
ottw	1675192
wvvip	838875
edtxu	1817153
snipf	1613092
wskef	1738699
wondf	1882839
efbi	1699018
iehc	50292
wvfc	910679
clbs	320017
lbtd	1520592
mbbc	1835385
snnf	1708715
pwco	733337
impho	1000298
lndzf	1617765
hnckf	1336364
afgc	1042046
tbba	1751700
sgldf	1318482
sume	1396633
ftlf	1374328
erkh	1769725
ocgsf	942149
ctek	1011432
norzf	910569
oasmy	1607245
mindp	926423
solcf	1710138
osto	74925
wstl	1002135
nbev	1579823
cyccp	1130166
pngzf	1725750
sinc	1532595
whlrd	1527541
admq	1588014
tvc	1376986
tve	1376986
vmnt	1605057
sgoo	1387998
milof	1898643
sbsaa	927720
brll	1631463
emhtf	1768225
sgic	1446371
zyqg	1790368
ucasu	1723517
alna	1624658
gkit	1398529
kwemf	1889823
gads	711665
prpi	1395445
auto	1023364
lazyw	1721741
atvk	1530185
cetxp	1435064
isgn	727634
abmt	1385799
qbak	758938
hrst	1361937
ucix	1358099
rhe-pa	1004724
kngw	1797762
regrf	925541
mllof	1370496
grnf	1492448
verbw	1566610
sdttu	1509228
whlrp	1527541
igex	1442853
iwsy	941685
bleg	1794311
btzi	1525852
unir	1172706
ftrs	1041633
lmdcf	1177167
imuc	822411
vxit	1591615
rgus	922330
encc	1119807
askh	1511161
prki	1579717
zpas	1635748
inct	1099234
drop	842722
genn	1351051
ewll	1550020
jshg	1475430
gttnq	1315255
ptoi	1381105
bomh	1678746
biei	1515740
mrge	1623360
rgbpp	1589150
vbvt	1575975
oxbrw	1584831
ghav	1514113
grpx	1472998
crxt	1817944
ottv	1479000
phcg	1351573
trnx	1353487
wsrc	42050
accr	1041588
strz	1043156
ltsc	1108891
nuvi	1410708
kmgh	1471302
brsyf	1505065
gnca	1457612
ndvn	1659183
rpnrf	1833089
gvsi	1068618
exla	1388174
ctam	18172
vitx	1609472
dcac	1498291
ftxp	1503458
hdii	1058828
psyc	1771077
artlw	1621221
acmc	934543
trnf	1778805
yriv	1487843
evio	715788
dgtw	1065598
zeno	1458581
pfsf	1400732
pdnlb	731245
leas	822655
tgro	1386018
favo	1795851
psru	1415306
destq	896985
grwc	1448558
find	1089061
tldn	1101246
kldo	1751299
nbco	1383637
vhld	1916879
sigo	1689066
gnal	1398137
onov	1877939
omps	1103640
kegx	318996
nich	772263
clnv	1391426
chcr	22872
igldf	1090159
essi	1490873
stwc	1400683
nbgv	726293
tchc	1639874
cbtc	1429859
lith	1396054
syev	1056757
gbpt	1527844
nann	1685570
rbnw	1608430
cdxi	1544238
rpt-pd	842183
rrac-un	1860879
rrac-wt	1860879
rrvcs	1768126
ross-un	1841610
ross-wt	1841610
roseu	1870129
rosew	1870129
roni-un	1845437
roni-wt	1845437
rollp	1324948
roclu	1885998
roclw	1885998
roivw	1635088
rocar	1884516
rocau	1884516
rkly-wt	1852117
rjac-un	1852495
rnr-pg	913144
rnwww	1848763
rnerw	1854461
rmpl-p	1644771
rmgcu	1838108
rmgcw	1838108
rsvrw	1824403
rsscs	1768126
rssks	1768126
rtlpo	1568162
rtlpp	1568162
rvic	1735184
rvphw	1742927
rvsnw	1743905
rwayl	1653384
rwodu	1907223
rwodw	1907223
rxrau	1840463
rxraw	1840463
sach-pa	1682220
sabsw	1833214
sacc	1682220
sabrp	1597033
sagar	1855351
sagau	1855351
saitw	1847075
samau	1843100
samaw	1843100
sasbq	1451167
satlw	1874315
sat	1377936
sapmd	1062750
sanbw	1856689
sigip	230557
silod	1514183
sierw	1844135
shuaw	1886268
shpw-wt	1784851
shqau	1844908
shqaw	1844908
shacw	1820160
shap-un	1881462
shap-wt	1881462
shcau	1848558
shcaw	1848558
shiiy	908732
shcrw	1816233
sho-ph	1295810
sho-pi	1295810
sghlu	1855885
sghlw	1855885
sghc-wt	1878057
sgiiu	1869824
sgiiw	1869824
sfuny	1294404
sgg	312070
sggff	312070
ses-wt	1819142
sf-pc	720672
sf-pd	720672
sfb	720672
sdrdw	1349436
sdrww	1349436
seatw	1856031
seal-pa	1308106
seal-pb	1308106
sedrf	1737706
sect	925661
seda-un	1846975
seda-wt	1846975
scua-un	1853594
scua-wt	1853594
sdacu	1837248
sdacw	1837248
scobu	1838431
scobw	1838431
scoau	1828985
scoaw	1828985
scrmu	1893325
scrmw	1893325
sbug	312070
sbnyp	1288784
sbev-wt	1553788
sbba	1483934
sbigw	1801602
sbii-un	1842729
sbii-wt	1842729
sbfmw	1402328
schw-pj	316709
sco	1415311
scleu	1838697
sclew	1838697
sce-pg	92103
sce-ph	92103
sce-pj	92103
sce-pk	92103
sce-pl	92103
scdl	1114446
scaqu	1821812
scaqw	1821812
sccb	1682220
sccc	1682220
sccd	1682220
scce	1682220
sccf	1682220
sccg	1682220
suac-un	1885461
suac-wt	1885461
stssw	1737995
stry-wt	1884697
strrp	707388
str-wt	1703785
strnw	1872525
strcw	1826681
stre-un	1838361
stre-wt	1838361
stet-un	1858684
stet-wt	1858684
stixw	1896049
surgw	1392694
svnau	1850699
svnaw	1850699
svix	1793497
svfau	1828478
svfaw	1828478
swetu	1828914
swetw	1828914
svxy	1415311
svrew	1894693
swvlw	1875609
swssu	1838000
swssw	1838000
swt	93556
srsau	1821682
srsaw	1821682
srznw	1824893
ssaau	1830547
ssaaw	1830547
sspcf	1576018
ssssl	1509470
sst-wt	1805833
ssu-wt	1869858
spkar	1848097
spkbu	1826435
spkbw	1826435
spkx	1817218
spky	1817218
spnt-pb	1576018
sptku	1836259
sptkw	1836259
srea	1032208
srcu	1126956
sqllw	1605888
sqftp	1080657
sqftw	1080657
si-pa	1312109
sivbp	719739
sjij	91928
sjiv	91928
slamu	1838162
slamw	1838162
skyh-wt	1823587
slac-un	1834755
slac-wt	1834755
skgru	1912461
skgrw	1912461
skil-wt	1774675
skyau	1847152
skyaw	1847152
smihu	1839185
smihw	1839185
smfrw	1818331
smhb	1114446
slvru	1842644
slvrw	1842644
slvo	1053092
smapu	1863990
smapw	1863990
slnhp	64463
slhgf	1839730
slhpf	1839730
slgcw	1837412
sldpw	1844862
slcru	1826553
slcrw	1826553
solow	1637736
sondw	1819395
sojd	92122
soje	92122
socgm	92108
spgs-un	1839127
spgs-wt	1839127
spir-wt	1816017
spe-pc	897802
sounw	1840856
soyb	1471824
spcmu	1880968
spcmw	1880968
snnc	1313938
snv-pe	18349
snrhw	1823854
snaxw	1691936
sncrl	1131554
smr-wt	1822966
syf-pa	1601712
szzlu	1829322
szzlw	1829322
sytaw	1649009
tbcpu	1815753
tbcpw	1815753
tbc	732717
talkw	1803901
tags	1471824
takax	1725472
takix	1725472
taklx	1725472
takmx	1725472
taknx	1725472
takyx	1725472
tcbpw	1872812
tcbio	1077428
tbsau	1843764
tbsaw	1843764
tbkcp	1539638
tbltw	1668370
tblaw	1840502
tcoa-un	1846750
tcoa-wt	1846750
tds-pu	1051512
tds-pv	1051512
tdw-wt	98222
tdw-wta	98222
tdw-wtb	98222
tdgmw	98222
tgr-un	1863099
tgr-wt	1863099
tgvcu	1865191
tgvcw	1865191
thcaw	1773087
thacu	1863685
thacw	1863685
thcpu	1843993
thcpw	1843993
tioau	1846163
tioaw	1846163
thwww	1712189
tinv-un	1820144
tinv-wt	1820144
tihe	1066719
tfc-pi	92230
tfc-po	92230
tfc-pr	92230
tgaau	1847355
tgaaw	1847355
tgh-pa	1413159
tgh-pb	1413159
tekku	1822027
tekkw	1822027
teteu	1900679
tetew	1900679
tetcu	1840920
tetcw	1840920
tetar	1444874
telz	61398
tlga-un	1827871
tlga-wt	1827871
tlgyu	1879814
tlgyw	1879814
tmac-wt	1835236
tmcww	1798562
tmkru	1821606
tmkrw	1821606
tmpmu	1823524
tmpmw	1823524
trinl	1786108
trca-un	1840353
trca-wt	1840353
traq-un	1841230
traq-wt	1841230
t-pa	732717
toacu	1860482
toacw	1860482
tpgy-un	1819399
tpgy-wt	1819399
tpbau	1847090
tpbaw	1847090
toiiw	1799191
t-pc	732717
tronw	1847513
trirw	1819876
trkaw	1021096
tris-un	1852736
tris-wt	1852736
trtn-pb	1660734
trtn-pc	1660734
trtn-pd	1660734
trtn-pe	1660734
trtl-un	1847112
trtl-wt	1847112
trtx-pc	1630472
tvixf	1053092
twcbu	1845618
twcbw	1845618
twnd-un	1814215
twnd-wt	1814215
twlvu	1819498
twlvw	1819498
tsibu	1832737
tsibw	1832737
tspq-un	1838219
tspq-wt	1838219
vhaq-wt	1823857
vhnau	1868640
vhnaw	1868640
viasp	1606268
vieww	1811856
venar	1800392
venaw	1800392
velou	1832371
velow	1832371
vcxb-un	1848948
vcxb-wt	1848948
vckaw	1820190
vcxau	1848898
vcxaw	1848898
vixm	1415311
vixy	1415311
vincu	1796129
viiau	1826011
viiaw	1826011
viixf	1053092
vmgau	1875943
vmgaw	1875943
vmcau	1892747
vmcaw	1892747
vlta-wt	1819584
vln-wt	1863006
vlatu	1843091
vlatw	1843091
vldrw	1745317
vld-wt	1825079
vxx	312070
vxz	312070
vygg-un	1822877
vygg-wt	1822877
vweww	1834045
vsacu	1883983
vsacw	1883983
vssyw	1701963
vtaqr	1822145
vtaqu	1822145
vtaqw	1822145
vtiqw	1823884
vno-pn	899689
vno-po	899689
vpcbu	1840792
vorbw	1843388
vrssf	1701963
vrmew	1104038
vbocu	1853920
vbixd	797542
val-wt	314808
vaniw	1266806
utaau	1879221
utaaw	1879221
uvxy	1415311
uvix	1793497
uwmc-wt	1783398
uzd	821130
uze	821130
uzf	821130
uph-wt	1770141
up-wt	1819516
uptdw	1844417
usb-pq	36104
usb-pr	36104
usb-ps	36104
usctu	1860514
usctw	1860514
usoi	1053092
uswsw	1670349
uslvf	1053092
usml	1114446
unov	1538495
untcw	798949
ugazf	1053092
ugic	884614
ugl	1415311
ugldf	1053092
ukomw	1821424
ule	1415311
ucbio	857855
ucib	1114446
uco	1415311
ubp-pk	1029800
twni-un	1835881
twni-wt	1835881
tzpsu	1823481
tzpsw	1823481
tzup	1853825
zgn-wt	1877787
zfoxw	1823575
zev-wt	1802749
yotar	1907730
yotau	1907730
yotaw	1907730
ygyip	1569329
xpaxu	1853397
xpaxw	1853397
xosww	1819493
xpdbu	1855474
xpdbw	1855474
xpngf	1810997
xomao	791908
xomap	791908
xone	1561627
ycs	1415311
ycbd-pa	1644903
ycl	1415311
xsicx	1124959
xsiix	1124959
xsiwx	1124959
xslcx	853180
xprtx	853180
ztaqu	1849408
ztaqw	1849408
zsl	1415311
zingu	1844270
zingw	1844270
zionl	109380
ziono	109380
zionp	109380
zivow	1101026
zivzf	1053092
zwrku	1828438
zwrkw	1828438
wtfcp	1015328
wtmar	1866226
wtmau	1866226
wtre	1601669
wwacu	1853044
wwacw	1853044
xbiow	1534525
xflt-pa	1703079
xfinu	1852749
xfinw	1852749
xcwpx	1467631
xdmbx	1565381
xelap	1620179
wolvd	1424404
wpca-un	1836075
wpca-wt	1836075
wlyb	107140
wnnr-un	1843714
wnnr-wt	1843714
wscrd	795800
wsbcp	203596
wrb-pf	11544
wrb-pg	11544
wrb-ph	11544
wpcb-un	1836074
wpcb-wt	1836074
wqga-un	1851174
wqga-wt	1851174
wfc-pz	72971
wftuf	1603923
wfc-pd	72971
wfc-pa	72971
we-wt	1813756
westw	1806347
wesfx	1586009
wldsw	1887673
wllaw	1255474
wllbw	1255474
wkspw	1096275
winrw	1708410
wil	312070
winvu	1854463
winvw	1854463
wcc-pa	929008
weat	1471824
wejow	1864448
welpm	107815
wel-wt	1877557
weifx	1586009
wafdp	936528
waldw	1840199
wal-pa	1212545
wbs-pg	801337
wbhc	885275
wbx-wt	1866501
wavsw	1868419
ceadw	1482541
cefd	1114446
cdrow	1866782
ccnep	736772
cctsu	1865861
cctsw	1865861
cdaqu	1851909
cdaqw	1851909
ccvi-un	1828250
ccvi-wt	1828250
ccv-un	1812234
ccv-wt	1812234
ceqp-p	1136352
celg-ri	14272
celuw	1752828
cenqw	1841425
cfg-pe	759944
cfivu	1825249
cfivw	1825249
cffsu	1839519
cffsw	1839519
cffeu	1839530
cffew	1839530
cfr-pb	39263
cgabl	1527166
cfviu	1830081
cfviw	1830081
cgccx	1676197
chekz	1610590
cheau	1856948
cheaw	1856948
chaa-un	1838293
chaa-wt	1838293
chkel	895126
chkew	895126
chkez	895126
chrb	1730346
chscl	823277
chscm	823277
chscn	823277
chsco	823277
clbtw	1854587
clbr-un	1847064
clbr-wt	1847064
clayw	1855467
clas-un	1826855
clas-wt	1826855
claqr	1849820
claqu	1849820
claqw	1849820
claa-un	1837739
citeu	1848437
citew	1848437
cingw	1862150
cifrw	1819989
ciigu	1841338
ciigw	1841338
cbrgu	1845149
cbrgw	1845149
ccaiu	1846968
ccaiw	1846968
capd	312070
cape	312070
cano-wt	1800682
cane	1471824
cade-pa	1299939
bxdif	1001085
bynou	1801417
bynow	1801417
byn-un	1852633
byn-wt	1852633
bytsu	1842566
bzfdw	1828972
brshw	1913210
brogw	1774983
brliw	1787518
brmk-wt	1784797
brivu	1844211
brivw	1844211
brkhu	1871638
brkhw	1871638
bsx-pa	885725
bsgar	1853084
bsgau	1853084
bskyu	1846804
bskyw	1846804
btwnu	1815086
btwnw	1815086
btbdw	1718224
btmdw	1819253
bwacw	1821146
bulz	927971
bwaqr	1878074
bwaqu	1878074
bwbbp	1341317
bwcaw	1854863
bwnb	1630805
bw-pa	1630805
bwsn	1630805
biosw	1856653
biotu	1825413
biotw	1825413
bite-un	1831270
bite-wt	1831270
biph	1788348
bipi	1406234
bip-pa	1406234
bip-pb	1406234
bkkt-wt	1820302
bksy-wt	1753539
bleur	1843370
bleuu	1843370
bleuw	1843370
blblf	1723690
bldew	1779128
blnkw	1429764
blngu	1842384
blngw	1842384
bmaqr	1867757
bmaqu	1867757
bmaqw	1867757
bmac-un	1848020
bmac-wt	1848020
blua-un	1831006
blua-wt	1831006
bltsu	1827328
bltsw	1827328
body-wt	1826889
boh-pa	46195
boil	1415311
bocnu	1856961
bocnw	1856961
boas-un	1838544
boas-wt	1838544
boac-un	1818089
boac-wt	1818089
bnnru	1852332
bnnrw	1852332
bnku	927971
bnixr	1845942
bnixw	1845942
bnkd	927971
bngow	1411690
bmtx-wt	1725872
bracu	1865120
brezr	1817640
brezw	1817640
brds-wt	1861449
brd-un	1847351
brd-wt	1847351
bpacw	1856724
bpopm	763901
bpypn	1545772
bpypo	1545772
boxd-wt	1828672
bctxw	1610820
bdcx	1114446
bdcz	1114446
beatw	1779372
bdvc	1490927
bdxb	10795
beemw	1398805
bener	1828735
benew	1828735
bfac-un	1880441
bfac-wt	1880441
bfagy	1712641
berz	927971
beph	1533232
bepi	1533232
bep-pa	1533232
bhseu	1759186
bhsew	1759186
biafw	1712762
bhil-wt	1830210
bhacu	1851612
bhacw	1851612
bhfam	1685040
bhfan	1685040
bhfao	1685040
bfs-pe	907254
bfriw	1858685
bfly-wt	1804176
bfiiw	1723580
bgsx-un	1838666
bgsx-wt	1838666
bgryw	1824734
bcdaw	925741
bcm	312070
bc-pa	14930
bc-pb	14930
bc-pc	14930
bcsau	1873441
bcsaw	1873441
bbcpw	1703956
bbai-wt	1836981
bblgw	1419554
baswq	1109189
bark-wt	1819574
bamh	1001085
bami	1001085
banfp	760498
baltf	312070
bal	312070
baca-un	1869673
baca-wt	1869673
bac-pm	70858
bac-pn	70858
bac-po	70858
bac-pp	70858
bac-pq	70858
bac-ps	70858
aconw	1635077
acqru	1837393
acqrw	1837393
acp-pa	1503290
acro-un	1847891
acro-wt	1847891
acr-pc	1332551
acr-pd	1332551
acut	1850767
adilw	1513525
adnww	1744494
adra-wt	1823584
adalw	1853928
adc-pa	917251
adex-un	1830029
adex-wt	1830029
aderu	1822912
aderw	1822912
adocr	1824884
adocw	1824884
adrt-wt	1864032
adsew	1879248
adthw	1838672
aeaeu	1852016
aeaew	1852016
aefc	769218
aeacu	1827392
aeacw	1827392
advww	1776661
aehaw	1869974
ael-pa	1039828
ael-pb	1039828
aeppz	4904
afaru	1901886
afarw	1901886
afaqu	1841661
afaqw	1841661
aeva-wt	1789029
aesc	874761
acbau	1844389
acbaw	1844389
acaq-wt	1869141
acaxr	1897245
acaxu	1897245
acaxw	1897245
acacu	1914023
acacw	1914023
acahu	1836274
acahw	1836274
acabu	1893219
acabw	1893219
acdi-un	1850316
acdi-wt	1850316
acgln	947484
acevu	1813658
acevw	1813658
acii-un	1838614
acii-wt	1838614
achhy	1161611
achr-wt	1824502
abr-pd	1253986
abr-pe	1253986
abr-pf	1253986
aac-un	1829432
aac-wt	1829432
aaciw	1844817
aaic-pb	1209028
aaic-pc	1209028
aain	1209028
aatrl	1004434
aam-pa	1411494
aam-pb	1411494
aaqc-un	1838883
aaqc-wt	1838883
alorw	1883962
alpau	1842939
alpaw	1842939
alsar	1865111
alsau	1865111
alsaw	1865111
altg-pa	1759824
amciu	1843724
amciw	1843724
ambp-wt	1845097
ambc-wt	874501
alvow	1898416
altuu	1822366
altuw	1822366
amaow	1843656
allg-wt	1874474
all-pi	899051
alfiw	1833908
aizn	1267238
ajxa	1614806
akicu	1826574
akicw	1826574
afgd	1042046
afge	1042046
afgb	1042046
afriw	1903870
agbar	1769624
agbaw	1769624
agcgv	1423689
aftr-un	1865975
aftr-wt	1865975
agac-un	1833909
agac-wt	1833909
agm-pe	845877
agm-pf	845877
agm-pg	845877
agilw	1790625
aggru	1842329
aggrw	1842329
agncl	1423689
agnco	1423689
agncp	1423689
agriw	1826397
agq	1415311
aimau	1903464
aimaw	1903464
aimbu	1903464
aivi	1471302
aimdw	1014763
aibbr	1882963
aibbu	1882963
ahrnu	1856696
ahrnw	1856696
ahpau	1846253
ahpaw	1846253
ahl-pe	1267395
arteu	1839990
artew	1839990
arrwu	1835972
arrww	1835972
arsnd	1624517
arqqw	1859690
arrkf	1855743
arr-pc	1428205
argo-pa	1091748
arguu	1865377
arguw	1865377
arhys	1768126
arkow	1823794
arizr	1882078
arizu	1882078
arizw	1882078
arcku	1855886
arckw	1855886
argd	1091748
arecw	1590715
arebw	1648087
arbkl	1841675
arbgu	1823826
arbgw	1823826
arbew	1861841
apxiu	1868573
apxiw	1868573
aqna	1174169
aqnb	1174169
aqnu	1174169
apmiu	1857662
apmiw	1857662
ap-wt	6176
apts	1481832
aptv-pa	1521332
apphw	1807707
aptmu	1845550
aptmw	1845550
ampx-wt	1899287
amps-wt	1828723
amtr	1114446
amub	1114446
anac-un	1829558
anac-wt	1829558
amj	19617
amnd	1114446
amna	1114446
ampgw	1518461
ampi-un	1835636
ampi-wt	1835636
ape	1411579
apcxw	1070050
apgnw	1814140
apgb-un	1838337
apgb-wt	1838337
apca-wt	1862993
apacu	1844981
apacw	1844981
anghw	1871983
aogou	1881741
aogow	1881741
anzuu	1840877
anzuw	1840877
aspaw	1854583
astlw	1860805
astsw	1780312
asb-pf	7789
ascbr	1876716
ascbu	1876716
ascbw	1876716
ascar	1868775
ascau	1868775
ascaw	1868775
asaxu	1824211
asaxw	1824211
asaq-un	1819510
asaq-wt	1819510
atnfw	1690080
atlpf	748954
atmp	312070
ath-pb	1527469
ath-pc	1527469
ath-pd	1527469
atlcl	1464343
atlcp	1464343
atip-wt	1815849
atcol	1794846
atco-pd	1794846
atco-ph	1794846
atco-pi	1794846
asz-un	1839191
asz-wt	1839191
ata-wt	1825254
atakr	1883788
ataku	1883788
atakw	1883788
aubap	883948
aurow	1828108
aurcu	1835856
aurcw	1835856
aus-un	1838207
aus-wt	1838207
auudw	1554818
auvip	1811109
avacw	1836478
avan-un	1819608
avan-wt	1819608
avctw	1704760
avgop	1730168
awca	1021917
avptw	1777921
axac-ri	1898019
axac-wt	1898019
gbtg-wt	1820872
gbrgr	1822792
gbrgu	1822792
gbrgw	1822792
gbbkr	1894951
gbbkw	1894951
gaz	312070
gateu	1838513
gatew	1838513
gdnrw	1858912
gdl-pc	1378701
gcc	1379606
gbug	312070
gb-wt	1799983
gcmgw	1819796
gcrex	1597634
gacqu	1846288
gacqw	1846288
fzt-un	1839824
fzt-wt	1839824
gab-pk	794685
gainn	1321741
gainz	1321741
gamcu	1841125
gamcw	1841125
gaq-un	1852061
gaq-wt	1852061
gapa-un	1839121
gapa-wt	1839121
fxcor	1817565
fxcow	1817565
fviv-un	1828183
fviv-wt	1828183
fvt-un	1824434
fvt-wt	1824434
ghbwf	1848731
giacu	1853314
giacw	1853314
gia-un	1844505
gia-wt	1844505
ghixu	1894630
ghixw	1894630
ghacu	1806156
ggt-pg	921671
ggrow	1886190
ggmcu	1818880
ggmcw	1818880
ggaau	1865697
ggaaw	1865697
giixu	1841080
giixw	1841080
giprw	1651721
girex	1597634
giwww	1848795
geni-wt	1834489
geexu	1856774
geexw	1856774
geggl	1831096
gdv-pk	1260729
gdxd	927971
gdxu	927971
gdstr	1858007
gdstu	1858007
gdstw	1858007
geccm	1675033
geccn	1675033
gecco	1675033
gflu	1780232
gfor-un	1845459
gfor-wt	1845459
gfx-un	1823896
gfx-wt	1823896
gety-wt	1898496
genqu	1853112
genqw	1853112
gfgdr	1876714
gfgdu	1876714
gfaiw	1804469
fue	352960
ftviu	1844336
ftviw	1844336
fultp	700564
ftrsf	1666175
ftpau	1839972
ftpaw	1839972
ftiiu	1889450
ftiiw	1889450
fsrdw	1832351
fsrxu	1834336
fsrxw	1834336
fssiw	1822862
fsun	1709442
ftaau	1832696
ftaaw	1832696
ftev-un	1839569
ftev-wt	1839569
ftcvu	1829328
ftcvw	1829328
ftain	1590364
ftaio	1590364
ftaip	1590364
frejp	1026214
freew	1753706
frfzf	915191
frgap	1528930
frey-wt	1844224
frlaw	1849294
frmep	712534
fronu	1842223
fronw	1842223
frwau	1825739
frwaw	1825739
frxb-un	1840161
frxb-wt	1840161
frsgu	1842883
frsgw	1842883
fsnb-un	1840225
fsnb-wt	1840225
fpac-wt	1829426
foxwu	1829999
foxww	1829999
fosll	883569
frc-pj	1132979
frc-pk	1132979
frc-pl	1132979
frc-pm	1132979
frc-pn	1132979
frbnu	1874495
f-pb	37996
f-pc	37996
f-pd	37996
fnmap	310522
fnvtu	1857855
fnvtw	1857855
foa-wt	1828937
fmckk	1026214
fmivu	1842916
fmivw	1842916
fngo	927971
fngs	927971
fngu	927971
flya-un	1844650
flya-wt	1844650
flyd	927971
flyu	927971
et-pc	1276187
et-pd	1276187
et-pe	1276187
eucru	1822929
eucrw	1822929
euo	1415311
etwo-wt	1800347
evgru	1900402
evgow	1821159
evojw	1834342
evlvw	1805385
evex-wt	1823652
eve-un	1861121
eve-wt	1861121
eresu	1814287
eresw	1814287
eqrxw	1843762
eqd-un	1818221
eqha-un	1826729
eqha-wt	1826729
eqh-pa	1333986
eqh-pc	1333986
esm-un	1841420
esm-wt	1841420
esacu	1865506
esacw	1865506
espid	1346526
esscr	1760683
esscu	1760683
esscw	1760683
estwf	1716586
esus	1114446
encpu	1879373
encpw	1879373
encud	1500881
enerr	1855555
eneru	1855555
enerw	1855555
emldu	1889123
emldw	1889123
empw	1822928
emgc	1494448
emcgu	1869601
embkw	1827980
enpc-un	1816261
enpc-wt	1816261
enjwq	1830180
enscw	1716947
entfu	1870925
entfw	1870925
entxw	1638097
eqcdx	1582138
ep-pc	1506307
epr-pc	1045450
epr-pe	1045450
ephyu	1827248
ephyw	1827248
eosew	1805077
eocw-un	1843862
eocw-wt	1843862
fihd	1114446
fiiso	862831
finmu	1823855
fitbo	35527
fitbp	35527
fiacu	1854480
fiacw	1854480
ficvu	1855693
ficvw	1855693
fiee	1114446
flfvr	1912582
flfvu	1912582
flfvw	1912582
flme-un	1831481
flme-wt	1831481
flrap	1124198
flacu	1828326
flacw	1828326
ffiew	1805521
fgfpp	1591890
fgbip	1408534
fhltw	1851182
fhn-pb	36966
fhn-pc	36966
fhn-pd	36966
fhn-pe	36966
fhn-pf	36966
fgmcu	1906133
fgmcw	1906133
fgiww	1864943
fedl	1114446
fexdr	1852407
fexdw	1852407
fazew	1839360
fbrt-pe	1562528
fatbp	1705012
fatbw	1705012
fbds	926844
fbgx	1114446
fath-wt	1836176
fatpu	1865045
fatpw	1865045
fcnco	798941
fcncp	798941
fcax-un	1823733
fcax-wt	1823733
fcelb	886128
fcrx	1464963
evtl-wt	1867102
faca-un	1839550
faca-wt	1839550
fact-un	1838987
fact-wt	1838987
clnnw	1822791
cloer	1849058
cloeu	1849058
clinr	1883984
clinu	1883984
clinw	1883984
clim-un	1835713
clim-wt	1835713
cldt-pa	1476045
clrcr	1903392
clrcu	1903392
clrcw	1903392
clrmu	1831937
clrmw	1831937
cmcau	1865248
cmaxw	1813914
clvt-pa	1764046
clvrw	1819615
cnglu	1867443
cnglw	1867443
cnfrl	1502292
cndb-un	1851961
cndb-wt	1851961
cnd-un	1824301
cnd-wt	1824301
cnda-un	1851959
cnda-wt	1851959
cmraw	1907685
cmsc	811156
cmsd	811156
cms-pc	811156
cmpow	1823144
cnobp	712771
cno-pa	1224608
cntqu	1847986
cntqw	1847986
cntww	1704715
codi-pc	1345126
cof-pi	927628
cof-pj	927628
cof-pk	927628
cof-pl	927628
cof-pn	927628
comsp	1178727
comsw	1178727
coliu	1829615
coliw	1829615
corn	1471824
coolu	1829953
coolw	1829953
conxw	1823000
corzw	1839341
cors-un	1842659
cors-wt	1842659
covau	1837160
covaw	1837160
cow	312070
c-pk	831001
c-pn	831001
cpptl	1837671
cpaqu	1873964
cpaqw	1873964
cpaau	1841137
cpaaw	1841137
cper	1479247
cparu	1848410
cparw	1848410
cownl	1466538
cowtf	312070
ctaqw	1823634
csta-un	1834032
csta-wt	1834032
cslmu	1875493
cssel	1679063
cssen	1679063
csr-pc	798359
crxtw	1817944
crxwq	1817944
crznu	1844635
crznw	1844635
crtdw	1357671
cru-un	1825497
cru-wt	1825497
crcqw	1609253
cptnw	1498233
cpuh-un	1828608
cpuh-wt	1828608
cptk-un	1827899
cptk-wt	1827899
crhc-un	1818212
crhc-wt	1818212
crexw	1356093
crdtx	1676197
cresw	1034957
crecu	1851230
crecw	1851230
crlkp	1059262
cxac-un	1856242
cxac-wt	1856242
ctdd	68622
ctbb	68622
ctlpp	896429
ctv-wt	1835378
cto-pa	23795
ctos-wt	1709682
cubb	1488813
cuenw	1424657
curiw	1776909
cvii-un	1828248
cvii-wt	1828248
decau	1913577
decaw	1913577
ddt	28917
dc-wt	1852353
dcrdu	1848959
dcrdw	1848959
dgp	1159508
dhbcw	1838176
dgz	1159508
dhacu	1864531
dhacw	1864531
dhcau	1838163
dhcaw	1838163
dhcni	1075415
dhcnl	1075415
dhhcu	1830188
dhhcw	1830188
dcgow	1822359
dcfcw	1862490
dcp-pb	1338065
dcomp	846617
dbrg-ph	1679688
dbrg-pi	1679688
dbrg-pj	1679688
daoou	1870471
daoow	1870471
davew	1841408
datsw	1648960
dbgiw	1668010
czoo-wt	1859639
djp	312070
dlcau	1831928
dlcaw	1831928
dlr-pl	1297996
divvz	825788
djcb	1114446
dkdcu	1849380
dkdcw	1849380
disau	1838831
disaw	1838831
dhr-pb	313616
dilau	1843608
dilaw	1843608
didiy	1764757
dnz-un	1830374
dnz-wt	1830374
dms-wt	1725134
dmaqr	1857086
dmys-un	1858327
dmys-wt	1858327
dna-wt	1830214
dpcsu	1857803
dpcsw	1857803
dnrrw	945764
dodrw	949039
doma-wt	1722438
dslvf	1053092
dtb	936340
dtg	936340
drctw	1880613
drayu	1852771
drayw	1852771
drh-pa	1298946
drmaw	1853816
dsaq-wt	1871745
drtsw	1871321
eacpw	1832765
dzz	1159508
dx-pc	826675
duetu	1890671
duetw	1890671
dunew	1817232
dukb	1326160
dtstw	1419951
dtrtu	1865537
dtrtw	1865537
dtocu	1839998
dtocw	1839998
dtp	936340
dwin-un	1812360
dwin-wt	1812360
dwacu	1849635
dwacw	1849635
efc-pa	1411342
efc-pb	1411342
efhtu	1922858
efscp	1025835
eftrw	1828522
elat	1739104
eica	1754836
edtxw	1817153
edncw	1864891
edblw	1809750
ebrgf	895728
eccc	1604174
ecc-pd	1604174
eccv	1604174
eccw	1604174
ebacu	1841258
ebacw	1841258
ebbgf	895728
ebgef	895728
rbot-wt	1812173
rbspd	844150
rcb	1527590
rcc	1527590
rcfa-wt	1870143
rcacu	1874218
rcacw	1874218
rbt-wt	1862068
rca	1527590
rcrtw	1462223
rchgu	1817786
rchgw	1817786
rclfu	1833498
rclfw	1833498
rc-pc	1527590
rc-pe	1527590
qtekw	1839412
qsiaw	1816431
qsmgd	1627554
qull	1114446
qrtep	1355096
qprcd	824416
racyw	1860484
qvcc	1254699
rammu	1841867
rammw	1841867
rf-pe	1281761
rgtiw	1838359
rilyg	1464790
rilyk	1464790
rilyl	1464790
rilym	1464790
rilyn	1464790
rilyo	1464790
rilyp	1464790
rilyt	1464790
rilyz	1464790
ritm-pa	1556593
ritm-pb	1556593
ritm-pc	1556593
ritm-pd	1556593
rja	352960
riv-pa	1501072
rjf-pa	720005
rjf-pb	720005
rji	352960
rjn	352960
rjz	352960
reliw	1812727
reneu	1889112
renew	1889112
rdw-wt	1819810
reeaw	1843588
rexr-pc	1571283
rfacr	1847607
rfacu	1847607
rfacw	1847607
revew	1853651
revhu	1841389
revhw	1841389
revbu	1810560
revbw	1810560
ptocu	1838238
ptocw	1838238
ptixw	1022899
pticu	1821075
pticw	1821075
ptrcy	1108329
ptwou	1930313
pucku	1836100
puckw	1836100
pv-un	1818787
pv-wt	1818787
qbts-wt	1907982
qegyd	1295961
qfta-un	1830795
qfta-wt	1830795
qngy-wt	1794621
pwupu	1847345
pwupw	1847345
pw-pa	1532619
pwppw	1777835
pxpc	1785493
pxsap	1640043
pxsaw	1640043
pype	1114446
prh	1137774
prif-pd	1554625
prif-pf	1554625
prif-pg	1554625
prif-ph	1554625
prif-pi	1554625
prif-pj	1554625
prif-pk	1554625
prif-pl	1554625
prmff	1880319
prlhu	1856161
prlhw	1856161
procw	1863362
prenw	1876431
pre-pj	911421
prbm-un	1848165
prbm-wt	1848165
ppyau	1894057
ppyaw	1894057
pplaf	1546066
pphpu	1863460
pphpw	1863460
prpc-un	1821329
prpc-wt	1821329
prs	1137774
prsru	1825473
prsrw	1825473
prthw	1653558
psa-pi	1393311
psa-pj	1393311
psagu	1833235
psagw	1833235
psa-pf	1393311
psa-pg	1393311
psb-pz	866368
psec-pa	1287032
psa-pl	1393311
psa-pm	1393311
psa-pn	1393311
psa-po	1393311
psa-pp	1393311
psa-pq	1393311
psa-pr	1393311
psa-ps	1393311
pspc-un	1843716
pspc-wt	1843716
psnyw	1884082
psfe-wt	1833835
pdot-un	1841845
pdot-wt	1841845
pearw	1835567
pegru	1847241
pegrw	1847241
peb-pg	1474098
peb-ph	1474098
pcpc-un	1824993
pcpc-wt	1824993
pcttu	1830033
pcttw	1830033
pcxcw	1855751
pbnaf	1546066
pcgu	1004980
pbug	312070
pcctu	1844149
pcctw	1844149
pbaxu	1870404
pbaxw	1870404
payow	1845815
paraa	813828
parap	813828
paqcu	1830531
paqcw	1830531
pavmz	1624326
otrkp	1136174
otmow	1842498
ostru	1834226
ostrw	1834226
otecu	1846809
otecw	1846809
osi-un	1832136
osi-wt	1832136
orgnw	1802457
oriau	1847416
oriaw	1847416
oxlcl	1495222
oxlcp	1495222
oxlcz	1495222
oxlcn	1495222
oxsqz	1259429
oxsqg	1259429
oxusu	1852973
oxusw	1852973
oxy-wt	797468
owlt-wt	1816708
oxacw	1861622
oust-wt	1816581
pafor	1858414
pacxu	1829797
pacxw	1829797
pacwp	1102112
ozkap	1569650
paasf	771992
paci-un	1853070
paci-wt	1853070
pflex	1688554
pfxnl	1490349
pfxnz	1490349
pftau	1853580
pgm	312070
peplu	1873324
peplw	1873324
petvw	1512922
petww	1842356
pfh	1137774
pfdrw	1839132
pffl	1114446
pfalx	1688554
phge-un	1739174
phge-wt	1739174
phicu	1825724
phicw	1825724
pgss-un	1861541
pgss-wt	1861541
pgyww	1883085
pgrwu	1833213
pgrww	1833213
phunw	1665300
phyt-un	1848756
phyt-wt	1848756
picc-un	1835800
picc-wt	1835800
piai-un	1819175
piai-wt	1819175
piiiw	1832511
pipp-wt	1822835
pnfpp	1115055
pntm-un	1830392
pntm-wt	1830392
pmt-pc	1464423
pnacr	1858180
pnacu	1858180
pnacw	1858180
pmvc-un	1807765
pmvc-wt	1807765
ponow	1855631
powru	1829427
powrw	1829427
powwp	1015383
port-un	1865200
port-wt	1865200
pol-wt	1810140
pond-wt	1842430
plaou	1849737
plaow	1849737
plmiu	1840317
plmiw	1840317
pl-wt	1836833
pmflx	1723701
pmgmu	1837607
pmgmw	1837607
naacu	1830063
naacw	1830063
ncacw	1849475
ncplw	1414767
nblwf	1458891
nbrwf	1163739
nbstu	1831978
nbstw	1831978
ndacu	1837067
ndacw	1837067
neovw	1748137
nee-pn	753308
nee-pp	753308
nee-pq	753308
nfnt-un	1862327
nfnt-wt	1862327
newen	1004315
newtl	1587987
newtz	1587987
netc-un	1854458
netc-wt	1854458
nesrw	1698514
mtb-ph	36270
mtcn	1243429
mtbco	1582982
mtekw	1872964
mtryu	1860663
mtryw	1860663
mssar	1882464
mssau	1882464
mssaw	1882464
mtacu	1826667
mtacw	1826667
mtal-un	1853021
mtal-wt	1853021
msprw	1802450
msprz	1802450
ms-pl	895421
ms-po	895421
ms-pp	895421
mudsw	1820727
mtvc-un	1885754
mtvc-wt	1885754
murfu	1896212
murfw	1896212
mvrl	1114446
mvstw	1760689
mytg	1617351
mypsw	1823878
mprau	1853436
mpln-wt	1793229
mpacr	1841993
mpacw	1841993
msfix	1523289
msdau	1846996
msdaw	1846996
msbip	1466026
msacw	1836517
mondw	1828852
moncu	1828325
moncw	1828325
mobvu	1931691
mobqw	1084267
mnqff	1086888
mntn-un	1863719
mntn-wt	1863719
mnsbp	1693577
mobbw	1898643
mntsw	1781162
mlaiw	1851625
mktww	1805651
mlacu	1807594
mlacw	1807594
mkfg-wt	1816613
mit-wt	1826058
ml-wt	1807846
mlpb	1114446
mlpo	1053092
mlpr	1114446
mmtlp	1431959
mimo-wt	1823882
mir-wt	1809987
mitt-pc	1514281
mit-un	1826058
mitau	1847440
mitaw	1847440
mhnc	1412100
mgr	1004434
mgrb	1004434
mgrd	1004434
nib	312070
nile-pd	896493
nimc	1111711
nhicu	1852931
nhicw	1852931
nfys-un	1850502
nfys-wt	1850502
ngc-un	1843249
ngc-wt	1843249
nmmcu	1819157
nmmcw	1819157
nly-pi	1043219
nlspw	1783036
nlitu	1854963
nlitw	1854963
nnavw	1865631
nognw	1841800
nnupd	888981
noacu	1824888
noacw	1824888
note-wt	1823466
novvr	1858028
novvu	1858028
novvw	1858028
npabw	1837929
nrxpw	1719406
nrscx	1663712
nrsnw	1875091
nrszx	1663712
nref-pa	1786248
nrgd	927971
nrgu	927971
nrdy-wt	1819404
nracu	1831964
nracw	1831964
nstc-un	1835817
nstc-wt	1835817
nstd-un	1835814
nstd-wt	1835814
nsrwf	813639
nstb-un	1834518
nstb-wt	1834518
nubiu	1881551
nubiw	1881551
ntrso	73124
ntrbw	1676047
onbpo	707179
onbpp	707179
onfow	1825452
opinl	1456772
opp-pa	1678130
opp-pb	1678130
opalw	1842279
opa-un	1843121
opa-wt	1843121
opfi-wt	1818502
opad-wt	1825024
onyxu	1849548
onyxw	1849548
ofssh	1487918
ohaau	1870778
ohaaw	1870778
ogsrw	75208
ogmbs	1767876
ogpks	1767876
odtc	1717452
oepwu	1824677
oepww	1824677
ohpau	1836129
ohpaw	1836129
oild	927971
oilnf	312070
oilu	927971
oil	312070
olitw	1866816
nuvb-wt	1811063
nvacr	1859807
nvnow	1661053
nxdt-pa	1356115
nvvew	1836875
nvsau	1838433
nvsaw	1838433
oaspw	1486159
ocaxw	1820175
occin	1716951
occio	1716951
ocfcp	1004702
nymtz	1273685
nycb-pu	910073
nymtl	1273685
nymtm	1273685
nxplw	1058307
nxglw	1468929
lionw	1845991
livbu	1875257
livbw	1875257
littu	1848364
littw	1848364
lixtw	1335105
lidrw	1818644
lilmw	1855756
lhc-wt	1824153
libyu	1880151
libyw	1880151
lgv-un	1832300
lgv-wt	1832300
lgstu	1860871
lgstw	1860871
lgndz	886163
lgnxz	886163
lgnyz	886163
lgnzz	886163
lgvcu	1879297
lgvcw	1879297
lgtou	1883814
lgtow	1883814
ldhau	1842373
ldhaw	1842373
lev-wt	1834974
legau	1834628
legaw	1834628
lgacu	1836337
lgacw	1836337
lftrw	1822873
lghlw	1806524
lgl-wt	61004
lfchy	1268896
lfacu	1851266
lfacw	1851266
lexxw	1348362
lflyw	1785592
lfmdp	948320
lft-pa	1547546
lmdxw	1685428
lmaou	1831868
lmaow	1831868
llap-wt	1835512
lmacu	1831992
lmacw	1831992
ljaqu	1825437
ljaqw	1825437
locl-wt	1840780
locc-un	1848323
locc-wt	1848323
lokm-un	1841585
lokm-wt	1841585
ltifx	1496254
ltcfx	1496254
ltchw	1826000
lsprw	1859007
lseaw	1721386
lotzw	1759008
ltsh	1029730
lucyw	1808377
ltryw	1673481
ltsa	1029730
luxau	1823767
luxaw	1823767
lvacw	1855450
lvoxu	1723648
lvoxw	1723648
mbac-wt	1839175
mbnkp	1000209
mbgaf	1067318
mbinn	1629019
mbino	1629019
maqcw	1844419
mapsw	1779474
maaqr	1870144
maaqw	1870144
macc-un	1840148
macc-wt	1840148
macau	1835416
macaw	1835416
metcl	1687187
mfa-pc	1055160
met-pf	1099219
metxw	1796514
meoaw	1859310
mer-pk	70858
mdv-pa	1645873
meacw	1849902
mcaer	1853775
mcafr	1853774
mcafu	1853774
mcacr	1895249
mcacu	1895249
mcacw	1895249
mcaau	1856995
mcaaw	1856995
mbsc-un	1856589
mbsc-wt	1856589
mbtcr	1837344
mbtcu	1837344
mdh-wt	1823143
mdgsw	1618500
mdrrp	1654595
mcldw	1756499
mcle	1827855
kcgi-un	1865407
kcgi-wt	1865407
kcac-un	1899287
kcac-wt	1899287
kca-un	1899287
kbntw	1729750
kiiiu	1826059
kiiiw	1826059
kernw	1755953
key-pl	91576
klr-wt	1719489
klaqw	1823323
kmpb	860748
kinzu	1820875
kinzw	1820875
kkr-pc	1404912
kkrs	1404912
kittw	1849820
knsw-un	1885444
knsw-wt	1885444
kold	1415311
kore-wt	1855457
lbbbr	1867287
lbbbu	1867287
lbbbw	1867287
latgu	1868269
latgw	1868269
laxxu	1847846
laxxw	1847846
lando	1495240
laaof	1791706
lcaau	1841024
lcaaw	1841024
lcahu	1844642
lcahw	1844642
lbrdp	1611983
lcfyw	1875547
lcw-wt	1847577
ld	312070
kxinw	1713539
kwacw	1823086
kychu	1865701
kychw	1865701
laaau	1848000
laaaw	1848000
kttaw	1841330
ksicu	1843351
ksicw	1843351
kpltw	1785424
krnlu	1832950
krnlw	1832950
kref-pa	1631596
juggu	1842609
juggw	1842609
jupww	1760903
jun-un	1838814
jun-wt	1838814
jtkwy	1792627
jsprw	1788028
kaclu	1865468
kaclw	1865468
jwacr	1883799
jwsm-un	1831359
jwsm-wt	1831359
kahc-un	1843212
kahc-wt	1843212
kaiiu	1825962
kaiiw	1825962
kairu	1824171
kairw	1824171
kalww	1909152
jjoff	312070
jjp	312070
jjs	312070
jjt	312070
jju	312070
jggcr	1857518
jggcu	1857518
jggcw	1857518
jiref	835271
jja	312070
jjatf	312070
jjc	312070
jjctf	312070
jje	312070
jjetf	312070
jjg	312070
jjgtf	312070
jjm	312070
jjn	312070
jjntf	312070
jpm-pj	19617
jpm-pk	19617
jpm-pl	19617
jpm-pm	19617
jpuhf	835271
jrjcy	1297830
jmacu	1874259
jmacw	1874259
jo	312070
joby-wt	1819848
joffu	1824149
joffw	1824149
ixaqu	1852019
ixaqw	1852019
ivcpu	1845123
ivcpw	1845123
ivdaw	1397183
iwdl	1114446
iwfl	1114446
iwml	1114446
jfbrw	1885408
jcicu	1822312
jcicw	1822312
jatt-wt	1855644
jaqcu	1817868
jaqcw	1817868
ispow	1820566
itaqu	1841586
itaqw	1841586
iscb	51548
irs-wt	933267
irnt-wt	1777946
irrx-un	1854795
irrx-wt	1854795
itqru	1805508
itqrw	1805508
ivcbu	1857410
ivcau	1852889
ivcaw	1852889
igtar	1866838
igtaw	1866838
ignyu	1834645
ignyw	1834645
iiiiu	1829889
iiiiw	1829889
iinnw	1837493
igacw	1819496
igicw	1794338
igld	1090159
ieaww	1652362
ifin-un	1862935
ifin-wt	1862935
ifed	1114446
indiw	1841925
inaq-un	1862463
inaq-wt	1862463
inbkz	1562463
imtxw	1809196
imppp	1876581
imrnw	1660046
impx-un	1820191
impx-wt	1820191
imlp	312070
iivip	820318
imbil	870826
imaqu	1846235
imaqw	1846235
imacw	1729944
inno	1281845
inn-pf	1497645
inkaw	1826671
intew	1850262
ioacu	1854275
ioacw	1854275
invzw	1835654
ipof-un	1818873
ipof-wt	1818873
ipod-un	1818876
ipod-wt	1818876
ionq-wt	1824920
iogpq	866609
ipaxu	1844452
ipaxw	1844452
iqmdw	1850051
ipwlk	874761
ipwlo	874761
ipva-un	1839608
ipva-wt	1839608
ipvf-wt	1839610
ipviu	1839611
ipviw	1839611
iraau	1831874
iraaw	1831874
hrowl	1360214
hscsw	1468492
hpx-un	1809353
hpx-wt	1809353
htpa-un	1828817
htpa-wt	1828817
htoow	1819794
htlfp	920112
htibp	1561032
htfb	1487428
htfc	1487428
htaq-un	1850038
htaq-wt	1850038
hnnaz	1145255
hnpiy	929058
hnra-wt	1842556
hmcou	1829042
hmcow	1829042
hmdcf	1648257
hpp-pc	1482512
hpltu	1863181
hpltw	1863181
hpkew	1792849
horiu	1865533
hofvw	1708176
homu	1819006
hwelw	1845013
hwcpz	750577
htzww	1657853
hugs-un	1836894
hugs-wt	1836894
humaw	1818382
hyznw	1716583
hymcl	1718405
hymcw	1718405
hymcz	1718405
hwm-p	4281
hwkz-wt	1841144
icnc-un	1858351
hzon-un	1821788
hzon-wt	1821788
iber-un	1835205
iber-wt	1835205
gnacu	1832250
gnacw	1832250
gmvdw	1760764
gmlpf	1415916
gmfiu	1866547
gmfiw	1866547
gmblp	1451448
gmblw	1451448
gmblz	1451448
gnt-pa	1438893
gogn-un	1852940
gogn-wt	1852940
goevw	1750153
gnl-pb	1526113
gnrsu	1790665
gnrsw	1790665
glblw	1838615
gleeu	1843248
gleew	1843248
gldi	1053092
gll	1415311
glhau	1841734
glhaw	1841734
gllir	1888734
glliu	1888734
glliw	1888734
glu-pa	1282957
gls-wt	1805087
glta-un	1852767
glta-wt	1852767
glp-pb	1323468
gl-pd	320335
gru	352960
grrrw	1903145
groy-wt	1834026
grov-wt	1841761
gromw	1662574
gsevu	1828096
gsevw	1828096
gsce	886982
gsmgw	1738758
gsp	312070
gsqb-un	1849280
gsqb-wt	1849280
gsrmr	1901799
gsrmu	1901799
gsrmw	1901799
gpacu	1831979
gpacw	1831979
gpcor	1841209
gpcou	1841209
gpcow	1841209
goodn	1234006
govxw	832489
grbk-pa	1373670
grabw	1855612
gpmt-pa	1703644
grfxf	1816723
greel	1844971
grcyw	1768910
grnaw	1822691
grmwf	1876945
grn	312070
hdlb	1114446
hcviu	1842937
hcviw	1842937
hcneu	1843205
hcnew	1843205
hcmau	1845368
hcmaw	1845368
hciiw	1823033
hgty-wt	1840776
hfro-pa	1710680
herau	1842912
heraw	1842912
hlncf	1900304
hlly-wt	1822928
hmacr	1894370
hmacu	1894370
hmacw	1894370
hma-un	1850529
hma-wt	1850529
hlgn-wt	1840292
hlbzw	1788841
hlahu	1826814
hlahw	1826814
higa-wt	1823776
hhla-un	1824185
hhla-wt	1824185
hhgcr	1822886
hhgcu	1822886
hhgcw	1822886
hiiiu	1823034
hiiiw	1823034
hipo-wt	1828105
gvciu	1858503
gvciw	1858503
gwh-wt	1819438
gwiiw	1850487
gtxap	1735707
gtpau	1837105
gtpaw	1837105
gtpbu	1842046
gtpbw	1842046
guer	1832487
gsqd-un	1837207
gsqd-wt	1837207
gtacu	1848821
gtacw	1848821
gtbt	1346526
gxiiu	1826669
gxiiw	1826669
haiau	1848861
haiaw	1848861
haacu	1824013
haacw	1824013
hcaru	1824846
hcarw	1824846
hcdip	1784567
hcdiw	1784567
hcdiz	1784567
hcicu	1829455
hcicw	1829455
hawlm	46207
hbanm	49196
hbanp	49196
"""